# Input Data Processing

## Set Up Code

In [ ]:
import os
import pandas as pd
import re
import requests
from urllib.parse import urlencode
import time
import xlwings as xw
import io
import certifi
import numpy as np
import zipfile

# Set User-Defined Inputs
base_year = 2024 
start_year = 2014
end_year = 2050
atb_data_folder = 'ATB_Data'
atb_data_file = f'{base_year}_v3_Workbook.xlsx'
cambium_data_folder = 'Cambium_Data'
cambium_data_workbook = f'Cambium{base_year - 2000}_Workbook.xlsx'
cambium_data_file = f'Cambium{base_year - 2000}_allScenarios_annual_balancingArea.csv'
agent_data_folder = 'Agent_Data'
original_agent_data_folder = 'Agent_Data/original_agent_files'
updated_agent_data_folder = 'Agent_Data/updated_urdb_agent_files'
rate_data_folder = 'Rate_Data'
rate_json_data_folder = 'Rate_Data/json_files'
rate_state_data_folder = 'Rate_Data/state_urdb_labels'
state = 'texas' # Change this each time the 'Electricity Rates' code is being run if the state is different 
agents_original_file = f'agents_rider_{state}.parquet' # Update if needed
eia_api_key = 'daAlnp6Ye8riIV3svv4wSSdeGnleVxqCoERfFAQ7' # Replace if needed
urdb_api_key ='Ro9gfEvVHefX5C7TxRSrvi8x5ieLHLBMCAtES70t' # Replace if needed

## NOTES
# change 'atb24' to 'atb{future_year}' [control-F]
# change 'aeo25' to 'aeo{future_year}' [control-F]

# Overall ATB Assumptions
# Chosen Financial Case: Market
# Chosen Capital Recovery Period: TechLife

## Electricity Load

In [139]:
# Input Data Processing: Electricity Load [Load Projections and Load Growth Projections]

# Input Folder: load_growth
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'Load Growth Scenario'
# Data Source: EIA Annual Electricity Outlook
#              Table 54. Electric Power Projections by Electricity Market Module Region (62)
#              Series (4 sectors x 26 regions = 104 series): 
#                   1. 'Electricity: Electricity Demand: Residential'
#                   2. 'Electricity: Electricity Demand: Commercial/Other'
#                   3. 'Electricity: Electricity Demand: Industrial'
#                   4. 'Electricity: Electricity Demand: Transportation'
#--------------------#

# List of Scenarios
# AEO2023 Reference case
# AEO2025 Alternative Transportation 
# AEO2025 High Oil and Gas Supply
# AEO2025 High Oil Price
# AEO2025 High Zero-Carbon Technology Cost
# AEO2025 High Economic Growth
# AEO2025 Low Economic Growth
# AEO2025 Low Oil and Gas Supply
# AEO2025 Low Oil Price
# AEO2025 Low Zero-Carbon Technology Cost
# AEO2025 Alternative Electricity
# AEO2025 Reference case
scenario_list = ['aeo2023ref','alttrnp','highogs','highprice','highZTC','hm2025','lm2025','lowogs','lowprice','lowZTC','nocaa111','ref2025']

# List of Regions
# Florida Reliability Coordinating Council (FRCC)
# Midcontinent / Central (MISC)
# Midcontinent / East (MISE)
# Midcontinent / South (MISS)
# Midcontinent / West (MISW)
# Northeast Power Coordinating Council / New England (ISNE)
# Northeast Power Coordinating Council / New York City and Long Island (NYCW)
# Northeast Power Coordinating Council / Upstate New York (NYUP)
# PJM / Commonwealth Edison (PJMC)
# PJM / Dominion (PJMD)
# PJM / East (PJME)
# PJM / West (PJMW)
# SERC Reliability Corporation / Central (SRCE)
# SERC Reliability Corporation / East (SRCA)
# SERC Reliability Corporation / Southeastern (SRSE)
# Southwest Power Pool / Central (SPPC)
# Southwest Power Pool / North (SPPN)
# Southwest Power Pool / South (SPPS)
# Texas Reliability Entity (TRE)
# Western Electricity Coordinating Council / Basin (BASN)
# Western Electricity Coordinating Council / California North (CANO)
# Western Electricity Coordinating Council / California South (CASO)
# Western Electricity Coordinating Council / Northwest Power Pool Area (NWPP)
# Western Electricity Coordinating Council / Rockies (RMRG)
# Western Electricity Coordinating Council / Southwest (SRSG)
nerc_region_mapping = {
    'Florida Reliability Coordinating Council': 'FRCC',
    'Midcontinent / Central': 'MISC',
    'Midcontinent / East': 'MISE',
    'Midcontinent / South': 'MISS',
    'Midcontinent / West': 'MISW',
    'Northeast Power Coordinating Council / New England': 'ISNE',
    'Northeast Power Coordinating Council / New York City and Long Island': 'NYCW',
    'Northeast Power Coordinating Council / Upstate New York': 'NYUP',
    'PJM / Commonwealth Edison': 'PJMC',
    'PJM / Dominion': 'PJMD',
    'PJM / East': 'PJME',
    'PJM / West': 'PJMW',
    'SERC Reliability Corporation / Central': 'SRCE',
    'SERC Reliability Corporation / East': 'SRCA',
    'SERC Reliability Corporation / Southeastern': 'SRSE',
    'Southwest Power Pool / Central': 'SPPC',
    'Southwest Power Pool / North': 'SPPN',
    'Southwest Power Pool / South': 'SPPS',
    'Texas Reliability Entity': 'TRE',
    'Western Electricity Coordinating Council / Basin': 'BASN',
    'Western Electricity Coordinating Council / California North': 'CANO',
    'Western Electricity Coordinating Council / California South': 'CASO',
    'Western Electricity Coordinating Council / Northwest Power Pool Area': 'NWPP',
    'Western Electricity Coordinating Council / Rockies': 'RMRG',
    'Western Electricity Coordinating Council / Southwest': 'SRSG',
}

# Initialize the eia_data_all DataFrame (contains all scenarios)
load_projections_raw = pd.DataFrame()

##### Extract API Data for Each Scenarios
for scenario in scenario_list:

    # From EIA API Dashboard website: https://www.eia.gov/opendata/browser/aeo/2025
    base_url = 'https://api.eia.gov/v2/aeo/2025/data/?frequency=annual&data[0]=value&facets[tableId][]=62&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_NA_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_flrc_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_mcc_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_mce_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_mcs_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_mcw_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_nenycli_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_npccne_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_npccupny_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_pjmce_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_pjmd_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_pjme_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_pjmw_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_serccnt_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_serce_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_sercsoes_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_swppc_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_swppno_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_swppso_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_tre_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_weccb_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_wecccan_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_wecccas_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_weccrks_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_weccsw_blnkwh&facets[seriesId][]=cnsm_NA_comm_NA_elc_NA_wenwpp_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_NA_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_flrc_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_mcc_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_mce_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_mcs_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_mcw_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_nenycli_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_npccne_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_npccupny_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_pjmce_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_pjmd_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_pjme_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_pjmw_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_serccnt_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_serce_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_sercsoes_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_swppc_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_swppno_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_swppso_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_tre_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_weccb_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_wecccan_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_wecccas_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_weccrks_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_weccsw_blnkwh&facets[seriesId][]=cnsm_NA_idal_NA_elc_NA_wenwpp_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_NA_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_flrc_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_mcc_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_mce_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_mcs_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_mcw_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_nenycli_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_npccne_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_npccupny_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_pjmce_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_pjmd_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_pjme_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_pjmw_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_serccnt_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_serce_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_sercsoes_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_swppc_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_swppno_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_swppso_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_tre_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_weccb_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_wecccan_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_wecccas_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_weccrks_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_weccsw_blnkwh&facets[seriesId][]=cnsm_NA_resd_NA_elc_NA_wenwpp_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_NA_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_flrc_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_mcc_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_mce_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_mcs_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_mcw_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_nenycli_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_npccne_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_npccupny_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_pjmce_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_pjmd_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_pjme_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_pjmw_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_serccnt_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_serce_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_sercsoes_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_swppc_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_swppno_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_swppso_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_tre_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_weccb_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_wecccan_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_wecccas_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_weccrks_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_weccsw_blnkwh&facets[seriesId][]=cnsm_NA_trn_NA_elc_NA_wenwpp_blnkwh&sort[0][column]=period&sort[0][direction]=asc&offset=0&length=5000'
    api_key = eia_api_key
    full_url = base_url + '&facets[scenario][]=' + str(scenario) + '&api_key=' + api_key
    #print(full_url)
    
    # Query the API
    response = requests.get(full_url)
    #display(response.json())
    if response.status_code == 200:
        try:
            data = response.json()
            print('API call successful. Data retrieved.')
            if 'response' in data and 'data' in data['response']:
                aeo_data = data['response']['data']
                load_projections_scenario_raw = pd.DataFrame(aeo_data)
            else:
                print('No data found in the API response.')
        except requests.exceptions.JSONDecodeError:
            print('Error decoding JSON response.')
            print(response.text)
    else:
        print(f'API call failed with status code: {response.status_code}')
        print(response.text)

    # Add the data for the scenario into a master data frame
    load_projections_raw = pd.concat([load_projections_raw, load_projections_scenario_raw], ignore_index = True)
    
    time.sleep(5)  # Add a delay after each API call

API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.


In [141]:
##### SCENARIO: All [Load Projections]
#display(load_projections_raw)

### Clean up the data frame
load_projections_aeo25 = load_projections_raw.copy()

# Drop columns from the data frame
columns_to_drop = ['history', 'scenario', 'tableId', 'tableName', 'seriesId', 'unit']
load_projections_aeo25.drop(columns=columns_to_drop, inplace=True)

# Re-name the columns to align AEO output with dGen input
load_projections_aeo25.rename(columns={'period': 'year', 'scenarioDescription': 'scenario', 'seriesName': 'sector_abbr', 'value': 'billions_kwh', 'regionId': 'nerc_region_ID', 'regionName': 'nerc_region_desc'}, inplace=True)

# Drop rows from the data frame (year is less than base_year and region is United States)
load_projections_aeo25['year'] = pd.to_numeric(load_projections_aeo25['year'], errors='coerce').astype('Int64')
load_projections_aeo25 = load_projections_aeo25[load_projections_aeo25['year'] >= base_year]
load_projections_aeo25 = load_projections_aeo25[load_projections_aeo25['nerc_region_desc'] != 'United States']

# Add columns to the data frame
load_projections_aeo25['fuel_type'] = 'electricity'
load_projections_aeo25['nerc_region_abbr'] = load_projections_aeo25['nerc_region_desc'].map(nerc_region_mapping)

# Modify colummns in the data frame
# Re-name sectors
load_projections_aeo25.loc[load_projections_aeo25['sector_abbr'].str.contains('Residential', case=False), 'sector_abbr'] = 'res'
load_projections_aeo25.loc[load_projections_aeo25['sector_abbr'].str.contains('Commercial', case=False), 'sector_abbr'] = 'com'
load_projections_aeo25.loc[load_projections_aeo25['sector_abbr'].str.contains('Industrial', case=False), 'sector_abbr'] = 'ind'
load_projections_aeo25.loc[load_projections_aeo25['sector_abbr'].str.contains('Transportation', case=False), 'sector_abbr'] = 'trans'

# Modify scenario name
mask = load_projections_aeo25['scenario'] != 'AEO2023 Reference case'
prepend_string = f'AEO{base_year+1} '
load_projections_aeo25.loc[mask, 'scenario'] = prepend_string + load_projections_aeo25.loc[mask, 'scenario'].astype(str)

# Re-order the columns in the data frame
new_column_order = ['year', 'billions_kwh', 'sector_abbr', 'nerc_region_ID', 'nerc_region_abbr', 'nerc_region_desc', 'fuel_type', 'scenario']
load_projections_aeo25 = load_projections_aeo25[new_column_order]

### Retrieve some statistics on the EIA Data

# Year
unique_year_list = load_projections_aeo25['year'].unique().tolist()
unique_year_count = str(len(unique_year_list))
print('Number of Years: ' + unique_year_count)
print(unique_year_list)
print('')

# Scenario
unique_scenario_list = load_projections_aeo25['scenario'].unique().tolist()
unique_scenario_count = str(len(unique_scenario_list))
print('Number of Scenarios: ' + unique_scenario_count)
print(unique_scenario_list)
print('')

# Sector
unique_sector_list = load_projections_aeo25['sector_abbr'].unique().tolist()
unique_sector_count = str(len(unique_sector_list))
print('Number of Sectors: ' + unique_sector_count)
print(unique_sector_list)
print('')

# Region
unique_region_list = load_projections_aeo25['nerc_region_abbr'].unique().tolist()
unique_region_count = str(len(unique_region_list))
print('Number of Regions: ' + unique_region_count)
print(unique_region_list)
print('')

# Save the csv files
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_projections_aeo25.csv')
load_projections_aeo25.to_csv(output_path, index=False)
display(load_projections_aeo25)

##### SCENARIO: AEO2023 Reference case (aeo23reference) [Load Projections]
load_projections_aeo25_aeo23reference = load_projections_aeo25[load_projections_aeo25['scenario'] == 'AEO2023 Reference case']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_projections_aeo25_aeo2023reference.csv')
load_projections_aeo25_aeo23reference.to_csv(output_path, index=False)
#display(load_projections_aeo25_aeo23reference)

##### SCENARIO: AEO2025 Alternative Transportation (alttrans) [Load Projections]
load_projections_aeo25_alttrans = load_projections_aeo25[load_projections_aeo25['scenario'] == 'AEO2025 Alternative Transportation']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_projections_aeo25_alttrans.csv')
load_projections_aeo25_alttrans.to_csv(output_path, index=False)
#display(load_projections_aeo25_alttrans)

##### SCENARIO: AEO2025 High Oil and Gas Supply (highogs) [Load Projections]
load_projections_aeo25_highogs = load_projections_aeo25[load_projections_aeo25['scenario'] == 'AEO2025 High Oil and Gas Supply']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_projections_aeo25_highogs.csv')
load_projections_aeo25_highogs.to_csv(output_path, index=False)
#display(load_projections_aeo25_highogs)

##### SCENARIO: AEO2025 High Oil Price (highoilprice) [Load Projections]
load_projections_aeo25_highoilprice = load_projections_aeo25[load_projections_aeo25['scenario'] == 'AEO2025 High Oil Price']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_projections_aeo25_highoilprice.csv')
load_projections_aeo25_highoilprice.to_csv(output_path, index=False)
#display(load_projections_aeo25_highoilprice)

##### SCENARIO: AEO2025 High Zero-Carbon Technology Cost (highZTC) [Load Projections]
load_projections_aeo25_highZTC = load_projections_aeo25[load_projections_aeo25['scenario'] == 'AEO2025 High Zero-Carbon Technology Cost']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_projections_aeo25_highZTC.csv')
load_projections_aeo25_highZTC.to_csv(output_path, index=False)
#display(load_projections_aeo25_highZTC)

##### SCENARIO: AEO2025 High Economic Growth (highecongrowth) [Load Projections]
load_projections_aeo25_highecongrowth = load_projections_aeo25[load_projections_aeo25['scenario'] == 'AEO2025 High Economic Growth']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_projections_aeo25_highecongrowth.csv')
load_projections_aeo25_highecongrowth.to_csv(output_path, index=False)
#display(load_projections_aeo25_highecongrowth)

##### SCENARIO: AEO2025 Low Economic Growth (lowecongrowth) [Load Projections]
load_projections_aeo25_lowecongrowth = load_projections_aeo25[load_projections_aeo25['scenario'] == 'AEO2025 Low Economic Growth']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_projections_aeo25_lowecongrowth.csv')
load_projections_aeo25_lowecongrowth.to_csv(output_path, index=False)
#display(load_projections_aeo25_lowecongrowth)

##### SCENARIO: AEO2025 Low Oil and Gas Supply (lowogs) [Load Projections]
load_projections_aeo25_lowogs = load_projections_aeo25[load_projections_aeo25['scenario'] == 'AEO2025 Low Oil and Gas Supply']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_projections_aeo25_lowogs.csv')
load_projections_aeo25_lowogs.to_csv(output_path, index=False)
#display(load_projections_aeo25_lowogs)

##### SCENARIO: AEO2025 Low Oil Price (lowoilprice) [Load Projections]
load_projections_aeo25_lowoilprice = load_projections_aeo25[load_projections_aeo25['scenario'] == 'AEO2025 Low Oil Price']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_projections_aeo25_lowoilprice.csv')
load_projections_aeo25_lowoilprice.to_csv(output_path, index=False)
#display(load_projections_aeo25_lowoilprice)

##### SCENARIO: AEO2025 Low Zero-Carbon Technology Cost (lowZTC) [Load Projections]
load_projections_aeo25_lowZTC = load_projections_aeo25[load_projections_aeo25['scenario'] == 'AEO2025 Low Zero-Carbon Technology Cost']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_projections_aeo25_lowZTC.csv')
load_projections_aeo25_lowZTC.to_csv(output_path, index=False)
#display(load_projections_aeo25_lowZTC)

##### SCENARIO: AEO2025 Alternative Electricity (altelec) [Load Projections]
load_projections_aeo25_altelec = load_projections_aeo25[load_projections_aeo25['scenario'] == 'AEO2025 Alternative Electricity']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_projections_aeo25_altelec.csv')
load_projections_aeo25_altelec.to_csv(output_path, index=False)
#display(load_projections_aeo25_altelec)

##### SCENARIO: AEO2025 Reference case (reference) [Load Projections]
load_projections_aeo25_reference = load_projections_aeo25[load_projections_aeo25['scenario'] == 'AEO2025 Reference case']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_projections_aeo25_reference.csv')
load_projections_aeo25_reference.to_csv(output_path, index=False)
#display(load_projections_aeo25_reference)


Number of Years: 27
[2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050]

Number of Scenarios: 12
['AEO2023 Reference case', 'AEO2025 Alternative Transportation', 'AEO2025 High Oil and Gas Supply', 'AEO2025 High Oil Price', 'AEO2025 High Zero-Carbon Technology Cost', 'AEO2025 High Economic Growth', 'AEO2025 Low Economic Growth', 'AEO2025 Low Oil and Gas Supply', 'AEO2025 Low Oil Price', 'AEO2025 Low Zero-Carbon Technology Cost', 'AEO2025 Alternative Electricity', 'AEO2025 Reference case']

Number of Sectors: 4
['ind', 'trans', 'com', 'res']

Number of Regions: 25
['PJME', 'SRSE', 'SRCA', 'PJMD', 'NYUP', 'NYCW', 'ISNE', 'PJMC', 'PJMW', 'NWPP', 'CASO', 'SPPN', 'SPPC', 'CANO', 'SRSG', 'MISS', 'TRE', 'SPPS', 'SRCE', 'MISE', 'MISC', 'BASN', 'RMRG', 'MISW', 'FRCC']



,year,billions_kwh,sector_abbr,nerc_region_ID,nerc_region_abbr,nerc_region_desc,fuel_type,scenario
105,2024,47.322182,ind,5-10,PJME,PJM / East,electricity,AEO2023 Reference case
106,2024,60.860062,ind,5-15,SRSE,SERC Reliability Corporation / Southeastern,electricity,AEO2023 Reference case
107,2024,49.395676,ind,5-14,SRCA,SERC Reliability Corporation / East,electricity,AEO2023 Reference case
108,2024,12.308199,ind,5-13,PJMD,PJM / Dominion,electricity,AEO2023 Reference case
109,2024,17.252844,ind,5-9,NYUP,Northeast Power Coordinating Council / Upstate...,electricity,AEO2023 Reference case
...,...,...,...,...,...,...,...,...
33795,2050,0.386071,trans,5-7,ISNE,Northeast Power Coordinating Council / New Eng...,electricity,AEO2025 Reference case
33796,2050,27.075485,com,5-19,SPPN,Southwest Power Pool / North,electricity,AEO2025 Reference case
33797,2050,0.853199,trans,5-8,NYCW,Northeast Power Coordinating Council / New Yor...,electricity,AEO2025 Reference case
33798,2050,1.015111,trans,5-9,NYUP,Northeast Power Coordinating Council / Upstate...,electricity,AEO2025 Reference case


In [153]:
##### SCENARIO: All [Load Growth Projections]

### Clean up the data frame
load_growth_projections_aeo25 = load_projections_aeo25.copy()

# Add columns to the data frame
load_growth_projections_aeo25['load_multiplier'] = None
load_growth_projections_aeo25.loc[load_growth_projections_aeo25['sector_abbr'].str.contains('res', case=False), 'sector'] = 'Residential'
load_growth_projections_aeo25.loc[load_growth_projections_aeo25['sector_abbr'].str.contains('com', case=False), 'sector'] = 'Commercial'
load_growth_projections_aeo25.loc[load_growth_projections_aeo25['sector_abbr'].str.contains('ind', case=False), 'sector'] = 'Industrial'
load_growth_projections_aeo25.loc[load_growth_projections_aeo25['sector_abbr'].str.contains('trans', case=False), 'sector'] = 'Transportation'

# Add in rows for start_year to base_year
rows_to_duplicate = load_growth_projections_aeo25[load_growth_projections_aeo25['year'] == base_year].copy()
years_to_add = list(range(start_year, base_year))
dataframes_to_add = []
for year in years_to_add:
    new_dataframe = rows_to_duplicate.copy()
    new_dataframe['year'] = year
    dataframes_to_add.append(new_dataframe)
load_growth_projections_aeo25 = pd.concat(([load_growth_projections_aeo25] + dataframes_to_add), ignore_index=True)
load_growth_projections_aeo25 = load_growth_projections_aeo25.sort_values(by='year')

# Populate the load_multiplier column
load_growth_projections_aeo25['year'] = pd.to_numeric(load_growth_projections_aeo25['year'], errors='coerce')
load_growth_projections_aeo25['billions_kwh'] = pd.to_numeric(load_growth_projections_aeo25['billions_kwh'], errors='coerce')

def calculate_load_multiplier(row):
    year = row['year']
    sector = row['sector_abbr']
    region = row['nerc_region_abbr']
    scenario = row['scenario']
    kwh = row['billions_kwh']

    # Find the billions_kwh for the same sector, region, scenario, but for the base_year
    baseline_kwh = load_growth_projections_aeo25.loc[
        (load_growth_projections_aeo25['year'] == base_year) &
        (load_growth_projections_aeo25['sector_abbr'] == sector) &
        (load_growth_projections_aeo25['nerc_region_abbr'] == region) &
        (load_growth_projections_aeo25['scenario'] == scenario),
        'billions_kwh'
    ]
    return kwh / baseline_kwh.iloc[0]

load_growth_projections_aeo25['load_multiplier'] = load_growth_projections_aeo25.apply(calculate_load_multiplier, axis=1)

# Drop columns from the data frame
columns_to_drop = ['fuel_type', 'billions_kwh']
load_growth_projections_aeo25.drop(columns=columns_to_drop, inplace=True)

# Save the csv files
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_growth_projections_aeo25.csv')
load_growth_projections_aeo25.to_csv(output_path, index=False)
display(load_growth_projections_aeo25)

,year,sector_abbr,nerc_region_ID,nerc_region_abbr,nerc_region_desc,scenario,load_multiplier,sector
33598,2014,res,5-4,MISC,Midcontinent / Central,AEO2025 Reference case,1.000000,Residential
33201,2014,res,5-1,TRE,Texas Reliability Entity,AEO2025 Low Oil Price,1.000000,Residential
33200,2014,res,5-2,FRCC,Florida Reliability Coordinating Council,AEO2025 Low Oil Price,1.000000,Residential
33199,2014,ind,5-4,MISC,Midcontinent / Central,AEO2025 Low Oil and Gas Supply,1.000000,Industrial
33198,2014,ind,5-5,MISE,Midcontinent / East,AEO2025 Low Oil and Gas Supply,1.000000,Industrial
...,...,...,...,...,...,...,...,...
5394,2050,ind,5-24,RMRG,Western Electricity Coordinating Council / Roc...,AEO2025 Alternative Transportation,1.932421,Industrial
5393,2050,ind,5-23,NWPP,Western Electricity Coordinating Council / Nor...,AEO2025 Alternative Transportation,1.967218,Industrial
5392,2050,res,5-3,MISW,Midcontinent / West,AEO2025 Alternative Transportation,1.185172,Residential
18802,2050,res,5-20,SRSG,Western Electricity Coordinating Council / Sou...,AEO2025 Low Economic Growth,1.497752,Residential


In [155]:
##### SCENARIO: AEO2023 Reference case (aeo23reference) [Load Growth Projections]
load_growth_projections_aeo25_aeo23reference = load_growth_projections_aeo25[load_growth_projections_aeo25['scenario'] == 'AEO2023 Reference case']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_growth_projections_aeo25_aeo23reference.csv')
load_growth_projections_aeo25_aeo23reference.to_csv(output_path, index=False)
#display(load_growth_projections_aeo25_aeo23reference)

##### SCENARIO: AEO2025 Alternative Transportation (alttrans) [Load Growth Projections]
load_growth_projections_aeo25_alttrans = load_growth_projections_aeo25[load_growth_projections_aeo25['scenario'] == 'AEO2025 Alternative Transportation']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_growth_projections_aeo25_alttrans.csv')
load_growth_projections_aeo25_alttrans.to_csv(output_path, index=False)
#display(load_growth_projections_aeo25_alttrans)

##### SCENARIO: AEO2025 High Oil and Gas Supply (highogs) [Load Growth Projections]
load_growth_projections_aeo25_highogs = load_growth_projections_aeo25[load_growth_projections_aeo25['scenario'] == 'AEO2025 High Oil and Gas Supply']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_growth_projections_aeo25_highogs.csv')
load_growth_projections_aeo25_highogs.to_csv(output_path, index=False)
#display(load_growth_projections_aeo25_highogs)

##### SCENARIO: AEO2025 High Oil Price (highoilprice) [Load Growth Projections]
load_growth_projections_aeo25_highoilprice = load_growth_projections_aeo25[load_growth_projections_aeo25['scenario'] == 'AEO2025 High Oil Price']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_growth_projections_aeo25_highoilprice.csv')
load_growth_projections_aeo25_highoilprice.to_csv(output_path, index=False)
#display(load_growth_projections_aeo25_highoilprice)

##### SCENARIO: AEO2025 High Zero-Carbon Technology Cost (highZTC) [Load Growth Projections]
load_growth_projections_aeo25_highZTC = load_growth_projections_aeo25[load_growth_projections_aeo25['scenario'] == 'AEO2025 High Zero-Carbon Technology Cost']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_growth_projections_aeo25_highZTC.csv')
load_growth_projections_aeo25_highZTC.to_csv(output_path, index=False)
#display(load_growth_projections_aeo25_highZTC)

##### SCENARIO: AEO2025 High Economic Growth (highecongrowth) [Load Growth Projections]
load_growth_projections_aeo25_highecongrowth = load_growth_projections_aeo25[load_growth_projections_aeo25['scenario'] == 'AEO2025 High Economic Growth']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_growth_projections_aeo25_highecongrowth.csv')
load_growth_projections_aeo25_highecongrowth.to_csv(output_path, index=False)
#display(load_growth_projections_aeo25_highecongrowth)

##### SCENARIO: AEO2025 Low Economic Growth (lowecongrowth) [Load Growth Projections]
load_growth_projections_aeo25_lowecongrowth = load_growth_projections_aeo25[load_growth_projections_aeo25['scenario'] == 'AEO2025 Low Economic Growth']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_growth_projections_aeo25_lowecongrowth.csv')
load_growth_projections_aeo25_lowecongrowth.to_csv(output_path, index=False)
#display(load_growth_projections_aeo25_lowecongrowth)

##### SCENARIO: AEO2025 Low Oil and Gas Supply (lowogs) [Load Growth Projections]
load_growth_projections_aeo25_lowogs = load_growth_projections_aeo25[load_growth_projections_aeo25['scenario'] == 'AEO2025 Low Oil and Gas Supply']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_growth_projections_aeo25_lowogs.csv')
load_growth_projections_aeo25_lowogs.to_csv(output_path, index=False)
#display(load_growth_projections_aeo25_lowogs)

##### SCENARIO: AEO2025 Low Oil Price (lowoilprice) [Load Growth Projections]
load_growth_projections_aeo25_lowoilprice = load_growth_projections_aeo25[load_growth_projections_aeo25['scenario'] == 'AEO2025 Low Oil Price']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_growth_projections_aeo25_lowoilprice.csv')
load_growth_projections_aeo25_lowoilprice.to_csv(output_path, index=False)
#display(load_growth_projections_aeo25_lowoilprice)

##### SCENARIO: AEO2025 Low Zero-Carbon Technology Cost (lowZTC) [Load Growth Projections]
load_growth_projections_aeo25_lowZTC = load_growth_projections_aeo25[load_growth_projections_aeo25['scenario'] == 'AEO2025 Low Zero-Carbon Technology Cost']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_growth_projections_aeo25_lowZTC.csv')
load_growth_projections_aeo25_lowZTC.to_csv(output_path, index=False)
#display(load_growth_projections_aeo25_lowZTC)

##### SCENARIO: AEO2025 Alternative Electricity (altelec) [Load Growth Projections]
load_growth_projections_aeo25_altelec = load_growth_projections_aeo25[load_growth_projections_aeo25['scenario'] == 'AEO2025 Alternative Electricity']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_growth_projections_aeo25_altelec.csv')
load_growth_projections_aeo25_altelec.to_csv(output_path, index=False)
#display(load_growth_projections_aeo25_altelec)

##### SCENARIO: AEO2025 Reference case (reference) [Load Growth Projections]
load_growth_projections_aeo25_reference = load_growth_projections_aeo25[load_growth_projections_aeo25['scenario'] == 'AEO2025 Reference case']
output_path = os.path.join(os.getcwd(), '..', 'load_growth', 'load_growth_projections_aeo25_reference.csv')
load_growth_projections_aeo25_reference.to_csv(output_path, index=False)
#display(load_growth_projections_aeo25_reference)

## Retail Electricity Prices

In [3]:
# Input Data Processing: Retail Electricity Prices [Rate Projections and Rate Escalation Projections]

# Input Folder: elec_prices
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'Retail Electricity Price Escalation Scenario'
# Data Source: EIA Annual Electricity Outlook
#              Table 54. Electric Power Projections by Electricity Market Module Region (62)
#              Series (4 sectors x 26 regions = 104 series): 
#                   1. 'Electricity: End-Use Prices: Residential'
#                   2. 'Electricity: End-Use Prices: Commercial'
#                   3. 'Electricity: End-Use Prices: Industrial'
#                   4. 'Electricity: End-Use Prices: Transportation'
#--------------------#

# List of Scenarios
# AEO2023 Reference case
# AEO2025 Alternative Transportation 
# AEO2025 High Oil and Gas Supply
# AEO2025 High Oil Price
# AEO2025 High Zero-Carbon Technology Cost
# AEO2025 High Economic Growth
# AEO2025 Low Economic Growth
# AEO2025 Low Oil and Gas Supply
# AEO2025 Low Oil Price
# AEO2025 Low Zero-Carbon Technology Cost
# AEO2025 Alternative Electricity
# AEO2025 Reference case
scenario_list = ['aeo2023ref','alttrnp','highogs','highprice','highZTC','hm2025','lm2025','lowogs','lowprice','lowZTC','nocaa111','ref2025']

# List of Regions
# Florida Reliability Coordinating Council (FRCC)
# Midcontinent / Central (MISC)
# Midcontinent / East (MISE)
# Midcontinent / South (MISS)
# Midcontinent / West (MISW)
# Northeast Power Coordinating Council / New England (ISNE)
# Northeast Power Coordinating Council / New York City and Long Island (NYCW)
# Northeast Power Coordinating Council / Upstate New York (NYUP)
# PJM / Commonwealth Edison (PJMC)
# PJM / Dominion (PJMD)
# PJM / East (PJME)
# PJM / West (PJMW)
# SERC Reliability Corporation / Central (SRCE)
# SERC Reliability Corporation / East (SRCA)
# SERC Reliability Corporation / Southeastern (SRSE)
# Southwest Power Pool / Central (SPPC)
# Southwest Power Pool / North (SPPN)
# Southwest Power Pool / South (SPPS)
# Texas Reliability Entity (TRE)
# Western Electricity Coordinating Council / Basin (BASN)
# Western Electricity Coordinating Council / California North (CANO)
# Western Electricity Coordinating Council / California South (CASO)
# Western Electricity Coordinating Council / Northwest Power Pool Area (NWPP)
# Western Electricity Coordinating Council / Rockies (RMRG)
# Western Electricity Coordinating Council / Southwest (SRSG)
nerc_region_mapping = {
    'Florida Reliability Coordinating Council': 'FRCC',
    'Midcontinent / Central': 'MISC',
    'Midcontinent / East': 'MISE',
    'Midcontinent / South': 'MISS',
    'Midcontinent / West': 'MISW',
    'Northeast Power Coordinating Council / New England': 'ISNE',
    'Northeast Power Coordinating Council / New York City and Long Island': 'NYCW',
    'Northeast Power Coordinating Council / Upstate New York': 'NYUP',
    'PJM / Commonwealth Edison': 'PJMC',
    'PJM / Dominion': 'PJMD',
    'PJM / East': 'PJME',
    'PJM / West': 'PJMW',
    'SERC Reliability Corporation / Central': 'SRCE',
    'SERC Reliability Corporation / East': 'SRCA',
    'SERC Reliability Corporation / Southeastern': 'SRSE',
    'Southwest Power Pool / Central': 'SPPC',
    'Southwest Power Pool / North': 'SPPN',
    'Southwest Power Pool / South': 'SPPS',
    'Texas Reliability Entity': 'TRE',
    'Western Electricity Coordinating Council / Basin': 'BASN',
    'Western Electricity Coordinating Council / California North': 'CANO',
    'Western Electricity Coordinating Council / California South': 'CASO',
    'Western Electricity Coordinating Council / Northwest Power Pool Area': 'NWPP',
    'Western Electricity Coordinating Council / Rockies': 'RMRG',
    'Western Electricity Coordinating Council / Southwest': 'SRSG',
}

# Initialize the eia_data_all DataFrame (contains all scenarios)
rate_projections_raw = pd.DataFrame()

##### Extract API Data for Each Scenarios
for scenario in scenario_list:

    # From EIA API Dashboard website: https://www.eia.gov/opendata/browser/aeo/2025
    base_url = 'https://api.eia.gov/v2/aeo/2025/data/?frequency=annual&data[0]=value&facets[tableId][]=62&facets[seriesId][]=prce_NA_comm_NA_elc_NA_NA_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_flrc_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_mcc_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_mce_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_mcs_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_mcw_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_nenycli_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_npccne_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_npccupny_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_pjmce_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_pjmd_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_pjme_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_pjmw_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_serccnt_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_serce_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_sercsoes_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_swppc_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_swppno_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_swppso_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_tre_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_weccb_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_wecccan_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_wecccas_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_weccrks_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_weccsw_ncntpkwh&facets[seriesId][]=prce_NA_comm_NA_elc_NA_wenwpp_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_NA_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_flrc_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_mcc_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_mce_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_mcs_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_mcw_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_nenycli_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_npccne_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_npccupny_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_pjmce_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_pjmd_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_pjme_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_pjmw_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_serccnt_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_serce_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_sercsoes_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_swppc_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_swppno_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_swppso_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_tre_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_weccb_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_wecccan_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_wecccas_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_weccrks_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_weccsw_ncntpkwh&facets[seriesId][]=prce_NA_idal_NA_elc_NA_wenwpp_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_NA_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_flrc_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_mcc_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_mce_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_mcs_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_mcw_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_nenycli_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_npccne_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_npccupny_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_pjmce_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_pjmd_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_pjme_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_pjmw_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_serccnt_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_serce_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_sercsoes_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_swppc_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_swppno_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_swppso_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_tre_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_weccb_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_wecccan_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_wecccas_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_weccrks_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_weccsw_ncntpkwh&facets[seriesId][]=prce_NA_resd_NA_elc_NA_wenwpp_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_NA_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_flrc_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_mcc_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_mce_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_mcs_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_mcw_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_nenycli_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_npccne_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_npccupny_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_pjmce_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_pjmd_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_pjme_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_pjmw_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_serccnt_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_serce_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_sercsoes_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_swppc_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_swppno_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_swppso_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_tre_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_weccb_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_wecccan_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_wecccas_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_weccrks_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_weccsw_ncntpkwh&facets[seriesId][]=prce_NA_trn_NA_elc_NA_wenwpp_ncntpkwh&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000'
    api_key = eia_api_key
    full_url = base_url + '&facets[scenario][]=' + str(scenario) + '&api_key=' + api_key
    #print(full_url)
    
    # Query the API
    response = requests.get(full_url)
    #display(response.json())
    if response.status_code == 200:
        try:
            data = response.json()
            print('API call successful. Data retrieved.')
            if 'response' in data and 'data' in data['response']:
                aeo_data = data['response']['data']
                rate_projections_scenario_raw = pd.DataFrame(aeo_data)
            else:
                print('No data found in the API response.')
        except requests.exceptions.JSONDecodeError:
            print('Error decoding JSON response.')
            print(response.text)
    else:
        print(f'API call failed with status code: {response.status_code}')
        print(response.text)

    # Add the data for the scenario into a master data frame
    rate_projections_raw = pd.concat([rate_projections_raw, rate_projections_scenario_raw], ignore_index = True)
    
    time.sleep(5)  # Add a delay after each API call

API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.
API call successful. Data retrieved.


In [15]:
##### SCENARIO: All [Rate Projections]
#display(rate_projections_raw)

### Clean up the data frame
rate_projections_aeo25 = rate_projections_raw.copy()

# Drop columns from the data frame
columns_to_drop = ['history', 'scenario', 'tableId', 'tableName', 'seriesId', 'unit']
rate_projections_aeo25.drop(columns=columns_to_drop, inplace=True)

# Re-name the columns to align AEO output with dGen input
rate_projections_aeo25.rename(columns={'period': 'year', 'scenarioDescription': 'scenario', 'seriesName': 'sector_abbr', 'regionId': 'nerc_region_ID', 'regionName': 'nerc_region_desc', 'value': 'cents_per_kwh'}, inplace=True)

# Drop rows from the data frame (year is less than base_year and region is United States)
rate_projections_aeo25['year'] = pd.to_numeric(rate_projections_aeo25['year'], errors='coerce').astype('Int64')
rate_projections_aeo25 = rate_projections_aeo25[rate_projections_aeo25['year'] >= base_year]
rate_projections_aeo25 = rate_projections_aeo25[rate_projections_aeo25['nerc_region_desc'] != 'United States']

# Add columns to the data frame
rate_projections_aeo25['fuel_type'] = 'electricity'
rate_projections_aeo25['nerc_region_abbr'] = rate_projections_aeo25['nerc_region_desc'].map(nerc_region_mapping)

# Modify colummns in the data frame
# Re-name sectors
rate_projections_aeo25.loc[rate_projections_aeo25['sector_abbr'].str.contains('Residential', case=False), 'sector_abbr'] = 'res'
rate_projections_aeo25.loc[rate_projections_aeo25['sector_abbr'].str.contains('Commercial', case=False), 'sector_abbr'] = 'com'
rate_projections_aeo25.loc[rate_projections_aeo25['sector_abbr'].str.contains('Industrial', case=False), 'sector_abbr'] = 'ind'
rate_projections_aeo25.loc[rate_projections_aeo25['sector_abbr'].str.contains('Transportation', case=False), 'sector_abbr'] = 'trans'

# Modify scenario name
mask = rate_projections_aeo25['scenario'] != 'AEO2023 Reference case'
prepend_string = f'AEO{base_year+1} '
rate_projections_aeo25.loc[mask, 'scenario'] = prepend_string + rate_projections_aeo25.loc[mask, 'scenario'].astype(str)

# Re-order the columns in the data frame
new_column_order = ['year', 'cents_per_kwh', 'sector_abbr', 'nerc_region_ID', 'nerc_region_abbr', 'nerc_region_desc', 'fuel_type', 'scenario']
rate_projections_aeo25 = rate_projections_aeo25[new_column_order]

### Retrieve some statistics on the EIA Data

# Year
unique_year_list = rate_projections_aeo25['year'].unique().tolist()
unique_year_count = str(len(unique_year_list))
print('Number of Years: ' + unique_year_count)
print(unique_year_list)
print('')

# Scenario
unique_scenario_list = rate_projections_aeo25['scenario'].unique().tolist()
unique_scenario_count = str(len(unique_scenario_list))
print('Number of Scenarios: ' + unique_scenario_count)
print(unique_scenario_list)
print('')

# Sector
unique_sector_list = rate_projections_aeo25['sector_abbr'].unique().tolist()
unique_sector_count = str(len(unique_sector_list))
print('Number of Sectors: ' + unique_sector_count)
print(unique_sector_list)
print('')

# Region
unique_region_list = rate_projections_aeo25['nerc_region_abbr'].unique().tolist()
unique_region_count = str(len(unique_region_list))
print('Number of Regions: ' + unique_region_count)
print(unique_region_list)
print('')

# Save the csv files
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_projections_aeo25.csv')
rate_projections_aeo25.to_csv(output_path, index=False)
display(rate_projections_aeo25)

##### SCENARIO: AEO2023 Reference case (aeo23reference) [Rate Projections]
rate_projections_aeo25_aeo23reference = rate_projections_aeo25[rate_projections_aeo25['scenario'] == 'AEO2023 Reference case']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_projections_aeo25_aeo2023reference.csv')
rate_projections_aeo25_aeo23reference.to_csv(output_path, index=False)
#display(rate_projections_aeo25_aeo23reference)

##### SCENARIO: AEO2025 Alternative Transportation (alttrans) [Rate Projections]
rate_projections_aeo25_alttrans = rate_projections_aeo25[rate_projections_aeo25['scenario'] == 'AEO2025 Alternative Transportation']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_projections_aeo25_alttrans.csv')
rate_projections_aeo25_alttrans.to_csv(output_path, index=False)
#display(rate_projections_aeo25_alttrans)

##### SCENARIO: AEO2025 High Oil and Gas Supply (highogs) [Rate Projections]
rate_projections_aeo25_highogs = rate_projections_aeo25[rate_projections_aeo25['scenario'] == 'AEO2025 High Oil and Gas Supply']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_projections_aeo25_highogs.csv')
rate_projections_aeo25_highogs.to_csv(output_path, index=False)
#display(rate_projections_aeo25_highogs)

##### SCENARIO: AEO2025 High Oil Price (highoilprice) [Rate Projections]
rate_projections_aeo25_highoilprice = rate_projections_aeo25[rate_projections_aeo25['scenario'] == 'AEO2025 High Oil Price']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_projections_aeo25_highoilprice.csv')
rate_projections_aeo25_highoilprice.to_csv(output_path, index=False)
#display(rate_projections_aeo25_highoilprice)

##### SCENARIO: AEO2025 High Zero-Carbon Technology Cost (highZTC) [Rate Projections]
rate_projections_aeo25_highZTC = rate_projections_aeo25[rate_projections_aeo25['scenario'] == 'AEO2025 High Zero-Carbon Technology Cost']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_projections_aeo25_highZTC.csv')
rate_projections_aeo25_highZTC.to_csv(output_path, index=False)
#display(rate_projections_aeo25_highZTC)

##### SCENARIO: AEO2025 High Economic Growth (highecongrowth) [Rate Projections]
rate_projections_aeo25_highecongrowth = rate_projections_aeo25[rate_projections_aeo25['scenario'] == 'AEO2025 High Economic Growth']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_projections_aeo25_highecongrowth.csv')
rate_projections_aeo25_highecongrowth.to_csv(output_path, index=False)
#display(rate_projections_aeo25_highecongrowth)

##### SCENARIO: AEO2025 Low Economic Growth (lowecongrowth) [Rate Projections]
rate_projections_aeo25_lowecongrowth = rate_projections_aeo25[rate_projections_aeo25['scenario'] == 'AEO2025 Low Economic Growth']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_projections_aeo25_lowecongrowth.csv')
rate_projections_aeo25_lowecongrowth.to_csv(output_path, index=False)
#display(rate_projections_aeo25_lowecongrowth)

##### SCENARIO: AEO2025 Low Oil and Gas Supply (lowogs) [Rate Projections]
rate_projections_aeo25_lowogs = rate_projections_aeo25[rate_projections_aeo25['scenario'] == 'AEO2025 Low Oil and Gas Supply']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_projections_aeo25_lowogs.csv')
rate_projections_aeo25_lowogs.to_csv(output_path, index=False)
#display(rate_projections_aeo25_lowogs)

##### SCENARIO: AEO2025 Low Oil Price (lowoilprice) [Rate Projections]
rate_projections_aeo25_lowoilprice = rate_projections_aeo25[rate_projections_aeo25['scenario'] == 'AEO2025 Low Oil Price']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_projections_aeo25_lowoilprice.csv')
rate_projections_aeo25_lowoilprice.to_csv(output_path, index=False)
#display(rate_projections_aeo25_lowoilprice)

##### SCENARIO: AEO2025 Low Zero-Carbon Technology Cost (lowZTC) [Rate Projections]
rate_projections_aeo25_lowZTC = rate_projections_aeo25[rate_projections_aeo25['scenario'] == 'AEO2025 Low Zero-Carbon Technology Cost']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_projections_aeo25_lowZTC.csv')
rate_projections_aeo25_lowZTC.to_csv(output_path, index=False)
#display(rate_projections_aeo25_lowZTC)

##### SCENARIO: AEO2025 Alternative Electricity (altelec) [Rate Projections]
rate_projections_aeo25_altelec = rate_projections_aeo25[rate_projections_aeo25['scenario'] == 'AEO2025 Alternative Electricity']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_projections_aeo25_altelec.csv')
rate_projections_aeo25_altelec.to_csv(output_path, index=False)
#display(rate_projections_aeo25_altelec)

##### SCENARIO: AEO2025 Reference case (reference) [Rate Projections]
rate_projections_aeo25_reference = rate_projections_aeo25[rate_projections_aeo25['scenario'] == 'AEO2025 Reference case']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_projections_aeo25_reference.csv')
rate_projections_aeo25_reference.to_csv(output_path, index=False)
#display(rate_projections_aeo25_reference)

Number of Years: 27
[2050, 2049, 2048, 2047, 2046, 2045, 2044, 2043, 2042, 2041, 2040, 2039, 2038, 2037, 2036, 2035, 2034, 2033, 2032, 2031, 2030, 2029, 2028, 2027, 2026, 2025, 2024]

Number of Scenarios: 12
['AEO2023 Reference case', 'AEO2025 Alternative Transportation', 'AEO2025 High Oil and Gas Supply', 'AEO2025 High Oil Price', 'AEO2025 High Zero-Carbon Technology Cost', 'AEO2025 High Economic Growth', 'AEO2025 Low Economic Growth', 'AEO2025 Low Oil and Gas Supply', 'AEO2025 Low Oil Price', 'AEO2025 Low Zero-Carbon Technology Cost', 'AEO2025 Alternative Electricity', 'AEO2025 Reference case']

Number of Sectors: 4
['res', 'com', 'ind', 'trans']

Number of Regions: 25
['ISNE', 'MISW', 'FRCC', 'MISE', 'MISC', 'TRE', 'SRCE', 'SRSE', 'SRCA', 'PJMC', 'PJMW', 'PJMD', 'PJME', 'NYUP', 'MISS', 'NYCW', 'BASN', 'RMRG', 'CANO', 'SRSG', 'NWPP', 'CASO', 'SPPN', 'SPPC', 'SPPS']



,year,cents_per_kwh,sector_abbr,nerc_region_ID,nerc_region_abbr,nerc_region_desc,fuel_type,scenario
0,2050,45.683788,res,5-7,ISNE,Northeast Power Coordinating Council / New Eng...,electricity,AEO2023 Reference case
1,2050,17.941492,com,5-3,MISW,Midcontinent / West,electricity,AEO2023 Reference case
2,2050,16.385078,com,5-2,FRCC,Florida Reliability Coordinating Council,electricity,AEO2023 Reference case
3,2050,20.394299,com,5-5,MISE,Midcontinent / East,electricity,AEO2023 Reference case
4,2050,17.442791,com,5-4,MISC,Midcontinent / Central,electricity,AEO2023 Reference case
...,...,...,...,...,...,...,...,...
33795,2024,12.592728,trans,5-10,PJME,PJM / East,electricity,AEO2025 Reference case
33796,2024,12.101444,trans,5-11,PJMW,PJM / West,electricity,AEO2025 Reference case
33797,2024,22.244051,com,5-21,CANO,Western Electricity Coordinating Council / Cal...,electricity,AEO2025 Reference case
33798,2024,25.059662,com,5-22,CASO,Western Electricity Coordinating Council / Cal...,electricity,AEO2025 Reference case


In [21]:
##### SCENARIO: All [Rate Escalation Projections]

### Clean up the data frame
rate_escalation_projections_aeo25 = rate_projections_aeo25.copy()

# Add columns to the data frame
rate_escalation_projections_aeo25['escalation_factor'] = None
rate_escalation_projections_aeo25.loc[rate_escalation_projections_aeo25['sector_abbr'].str.contains('res', case=False), 'sector'] = 'Residential'
rate_escalation_projections_aeo25.loc[rate_escalation_projections_aeo25['sector_abbr'].str.contains('com', case=False), 'sector'] = 'Commercial'
rate_escalation_projections_aeo25.loc[rate_escalation_projections_aeo25['sector_abbr'].str.contains('ind', case=False), 'sector'] = 'Industrial'
rate_escalation_projections_aeo25.loc[rate_escalation_projections_aeo25['sector_abbr'].str.contains('trans', case=False), 'sector'] = 'Transportation'

# Add in rows for start_year to base_year
rows_to_duplicate = rate_escalation_projections_aeo25[rate_escalation_projections_aeo25['year'] == base_year].copy()
years_to_add = list(range(start_year, base_year))
dataframes_to_add = []
for year in years_to_add:
    new_dataframe = rows_to_duplicate.copy()
    new_dataframe['year'] = year
    dataframes_to_add.append(new_dataframe)
rate_escalation_projections_aeo25 = pd.concat(([rate_escalation_projections_aeo25] + dataframes_to_add), ignore_index=True)
rate_escalation_projections_aeo25 = rate_escalation_projections_aeo25.sort_values(by='year')

# Populate the escalation_factor column
rate_escalation_projections_aeo25['year'] = pd.to_numeric(rate_escalation_projections_aeo25['year'], errors='coerce')
rate_escalation_projections_aeo25['cents_per_kwh'] = pd.to_numeric(rate_escalation_projections_aeo25['cents_per_kwh'], errors='coerce')

def calculate_escalation_factor(row):
    year = row['year']
    sector = row['sector_abbr']
    region = row['nerc_region_abbr']
    scenario = row['scenario']
    rate = row['cents_per_kwh']

    # Find the cents_per_kwh for the same sector, region, scenario, but for the base_year
    baseline_rate = rate_escalation_projections_aeo25.loc[
        (rate_escalation_projections_aeo25['year'] == base_year) &
        (rate_escalation_projections_aeo25['sector_abbr'] == sector) &
        (rate_escalation_projections_aeo25['nerc_region_abbr'] == region) &
        (rate_escalation_projections_aeo25['scenario'] == scenario),
        'cents_per_kwh'
    ]
    return rate / baseline_rate.iloc[0]

rate_escalation_projections_aeo25['escalation_factor'] = rate_escalation_projections_aeo25.apply(calculate_escalation_factor, axis=1)

# Drop columns from the data frame
columns_to_drop = ['cents_per_kwh']
rate_escalation_projections_aeo25.drop(columns=columns_to_drop, inplace=True)

# Save the csv files
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_escalation_projections_aeo25.csv')
rate_escalation_projections_aeo25.to_csv(output_path, index=False)
display(rate_escalation_projections_aeo25)

,year,sector_abbr,nerc_region_ID,nerc_region_abbr,nerc_region_desc,fuel_type,scenario,escalation_factor,sector
32401,2014,com,5-4,MISC,Midcontinent / Central,electricity,AEO2023 Reference case,1.000000,Commercial
33203,2014,ind,5-3,MISW,Midcontinent / West,electricity,AEO2025 Low Oil Price,1.000000,Industrial
33202,2014,ind,5-2,FRCC,Florida Reliability Coordinating Council,electricity,AEO2025 Low Oil Price,1.000000,Industrial
33201,2014,ind,5-1,TRE,Texas Reliability Entity,electricity,AEO2025 Low Oil Price,1.000000,Industrial
33200,2014,ind,5-5,MISE,Midcontinent / East,electricity,AEO2025 Low Oil Price,1.000000,Industrial
...,...,...,...,...,...,...,...,...,...
13538,2050,trans,5-3,MISW,Midcontinent / West,electricity,AEO2025 High Economic Growth,1.419294,Transportation
13539,2050,trans,5-2,FRCC,Florida Reliability Coordinating Council,electricity,AEO2025 High Economic Growth,1.926780,Transportation
13540,2050,ind,5-19,SPPN,Southwest Power Pool / North,electricity,AEO2025 High Economic Growth,2.028720,Industrial
13527,2050,res,5-13,PJMD,PJM / Dominion,electricity,AEO2025 High Economic Growth,1.605255,Residential


In [23]:
##### SCENARIO: AEO2023 Reference case (aeo23reference) [Rate Escalation Projections]
rate_escalation_projections_aeo25_aeo23reference = rate_escalation_projections_aeo25[rate_escalation_projections_aeo25['scenario'] == 'AEO2023 Reference case']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_escalation_projections_aeo25_aeo23reference.csv')
rate_escalation_projections_aeo25_aeo23reference.to_csv(output_path, index=False)
#display(rate_escalation_projections_aeo25_aeo23reference)

##### SCENARIO: AEO2025 Alternative Transportation (alttrans) [Rate Escalation Projections]
rate_escalation_projections_aeo25_alttrans = rate_escalation_projections_aeo25[rate_escalation_projections_aeo25['scenario'] == 'AEO2025 Alternative Transportation']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_escalation_projections_aeo25_alttrans.csv')
rate_escalation_projections_aeo25_alttrans.to_csv(output_path, index=False)
#display(rate_escalation_projections_aeo25_alttrans)

##### SCENARIO: AEO2025 High Oil and Gas Supply (highogs) [Rate Escalation Projections]
rate_escalation_projections_aeo25_highogs = rate_escalation_projections_aeo25[rate_escalation_projections_aeo25['scenario'] == 'AEO2025 High Oil and Gas Supply']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_escalation_projections_aeo25_highogs.csv')
rate_escalation_projections_aeo25_highogs.to_csv(output_path, index=False)
#display(rate_escalation_projections_aeo25_highogs)

##### SCENARIO: AEO2025 High Oil Price (highoilprice) [Rate Escalation Projections]
rate_escalation_projections_aeo25_highoilprice = rate_escalation_projections_aeo25[rate_escalation_projections_aeo25['scenario'] == 'AEO2025 High Oil Price']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_escalation_projections_aeo25_highoilprice.csv')
rate_escalation_projections_aeo25_highoilprice.to_csv(output_path, index=False)
#display(rate_escalation_projections_aeo25_highoilprice)

##### SCENARIO: AEO2025 High Zero-Carbon Technology Cost (highZTC) [Rate Escalation Projections]
rate_escalation_projections_aeo25_highZTC = rate_escalation_projections_aeo25[rate_escalation_projections_aeo25['scenario'] == 'AEO2025 High Zero-Carbon Technology Cost']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_escalation_projections_aeo25_highZTC.csv')
rate_escalation_projections_aeo25_highZTC.to_csv(output_path, index=False)
#display(rate_escalation_projections_aeo25_highZTC)

##### SCENARIO: AEO2025 High Economic Growth (highecongrowth) [Rate Escalation Projections]
rate_escalation_projections_aeo25_highecongrowth = rate_escalation_projections_aeo25[rate_escalation_projections_aeo25['scenario'] == 'AEO2025 High Economic Growth']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_escalation_projections_aeo25_highecongrowth.csv')
rate_escalation_projections_aeo25_highecongrowth.to_csv(output_path, index=False)
#display(rate_escalation_projections_aeo25_highecongrowth)

##### SCENARIO: AEO2025 Low Economic Growth (lowecongrowth) [Rate Escalation Projections]
rate_escalation_projections_aeo25_lowecongrowth = rate_escalation_projections_aeo25[rate_escalation_projections_aeo25['scenario'] == 'AEO2025 Low Economic Growth']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_escalation_projections_aeo25_lowecongrowth.csv')
rate_escalation_projections_aeo25_lowecongrowth.to_csv(output_path, index=False)
#display(rate_escalation_projections_aeo25_lowecongrowth)

##### SCENARIO: AEO2025 Low Oil and Gas Supply (lowogs) [Rate Escalation Projections]
rate_escalation_projections_aeo25_lowogs = rate_escalation_projections_aeo25[rate_escalation_projections_aeo25['scenario'] == 'AEO2025 Low Oil and Gas Supply']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_escalation_projections_aeo25_lowogs.csv')
rate_escalation_projections_aeo25_lowogs.to_csv(output_path, index=False)
#display(rate_escalation_projections_aeo25_lowogs)

##### SCENARIO: AEO2025 Low Oil Price (lowoilprice) [Rate Escalation Projections]
rate_escalation_projections_aeo25_lowoilprice = rate_escalation_projections_aeo25[rate_escalation_projections_aeo25['scenario'] == 'AEO2025 Low Oil Price']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_escalation_projections_aeo25_lowoilprice.csv')
rate_escalation_projections_aeo25_lowoilprice.to_csv(output_path, index=False)
#display(rate_escalation_projections_aeo25_lowoilprice)

##### SCENARIO: AEO2025 Low Zero-Carbon Technology Cost (lowZTC) [Rate Escalation Projections]
rate_escalation_projections_aeo25_lowZTC = rate_escalation_projections_aeo25[rate_escalation_projections_aeo25['scenario'] == 'AEO2025 Low Zero-Carbon Technology Cost']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_escalation_projections_aeo25_lowZTC.csv')
rate_escalation_projections_aeo25_lowZTC.to_csv(output_path, index=False)
#display(rate_escalation_projections_aeo25_lowZTC)

##### SCENARIO: AEO2025 Alternative Electricity (altelec) [Rate Escalation Projections]
rate_escalation_projections_aeo25_altelec = rate_escalation_projections_aeo25[rate_escalation_projections_aeo25['scenario'] == 'AEO2025 Alternative Electricity']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_escalation_projections_aeo25_altelec.csv')
rate_escalation_projections_aeo25_altelec.to_csv(output_path, index=False)
#display(rate_escalation_projections_aeo25_altelec)

##### SCENARIO: AEO2025 Reference case (reference) [Rate Escalation Projections]
rate_escalation_projections_aeo25_reference = rate_escalation_projections_aeo25[rate_escalation_projections_aeo25['scenario'] == 'AEO2025 Reference case']
output_path = os.path.join(os.getcwd(), '..', 'elec_prices', 'rate_escalation_projections_aeo25_reference.csv')
rate_escalation_projections_aeo25_reference.to_csv(output_path, index=False)
#display(rate_escalation_projections_aeo25_reference)

## Wholesale Electricity Prices

In [3]:
# Input Data Processing: Wholesale Electricity Prices

# Input Folder: wholesale_electricity_prices
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'Wholesale Electricity Price Scenario'
# Data Source: Cambium_Data

#--------------------#

# Import the Cambium data
cambium_file_path = os.path.join(cambium_data_folder, cambium_data_file)
cambium_data = pd.read_csv(cambium_file_path, skiprows = 3)

# Filter the Cambium data
columns_to_keep = ["Scenario", "Geography", "Time", "End use cost.3"] # Keep select columns
cambium_data = cambium_data[columns_to_keep].copy()
cambium_data = cambium_data.iloc[2:] # Remove certain rows
cambium_data['End use cost.3'] = pd.to_numeric(cambium_data['End use cost.3'], errors='coerce')
cambium_data['End use cost.3'] = cambium_data['End use cost.3'] / 1000 # Convert the units in this column from $/MWh to $/kWh
cambium_data = cambium_data.rename(columns={"End use cost.3": "Wholesale Electricity Price ($/kWh)", "Geography": "ba", "Time": "Year"}) # Re-name certain columns
#display(cambium_data)

# Separate out the data by scenario
wholesale_electricity_price_data_LowRECost = cambium_data[cambium_data['Scenario'] == 'LowRECost'].copy() # Scenario: LowRECost
#display(wholesale_electricity_price_data_LowRECost)
wholesale_electricity_price_data_MidCase = cambium_data[cambium_data['Scenario'] == 'MidCase'].copy() # Scenario: MidCase
#display(wholesale_electricity_price_data_MidCase)
wholesale_electricity_price_data_HighRECost = cambium_data[cambium_data['Scenario'] == 'HighRECost'].copy() # Scenario: HighRECost
#display(wholesale_electricity_price_data_HighRECost)
years_to_fill = [str(year) for year in range(2014, 2025)] # '2014' through '2024'

# Re-format the data frames: LowRECost
Cambium24_LowRECost_wholesale = wholesale_electricity_price_data_LowRECost.pivot_table(
    index='ba',
    columns='Year',
    values='Wholesale Electricity Price ($/kWh)'
)
Cambium24_LowRECost_wholesale = Cambium24_LowRECost_wholesale.reset_index()
Cambium24_LowRECost_wholesale.columns = [str(col) for col in Cambium24_LowRECost_wholesale.columns]
all_years_range = list(range(2014, 2051)) # 2014 to 2050 inclusive
desired_columns = ['ba'] + [str(year) for year in all_years_range]
Cambium24_LowRECost_wholesale = Cambium24_LowRECost_wholesale.set_index('ba')
Cambium24_LowRECost_wholesale = Cambium24_LowRECost_wholesale.reindex(columns=[str(year) for year in all_years_range])
Cambium24_LowRECost_wholesale = Cambium24_LowRECost_wholesale.reset_index()
for col in years_to_fill:
    Cambium24_LowRECost_wholesale[col] = Cambium24_LowRECost_wholesale['2025']
#display(Cambium24_LowRECost_wholesale)

# Re-format the data frames: MidCase
Cambium24_MidCase_wholesale = wholesale_electricity_price_data_MidCase.pivot_table(
    index='ba',
    columns='Year',
    values='Wholesale Electricity Price ($/kWh)'
)
Cambium24_MidCase_wholesale = Cambium24_MidCase_wholesale.reset_index()
Cambium24_MidCase_wholesale.columns = [str(col) for col in Cambium24_MidCase_wholesale.columns]
all_years_range = list(range(2014, 2051)) # 2014 to 2050 inclusive
desired_columns = ['ba'] + [str(year) for year in all_years_range]
Cambium24_MidCase_wholesale = Cambium24_MidCase_wholesale.set_index('ba')
Cambium24_MidCase_wholesale = Cambium24_MidCase_wholesale.reindex(columns=[str(year) for year in all_years_range])
Cambium24_MidCase_wholesale = Cambium24_MidCase_wholesale.reset_index()
for col in years_to_fill:
    Cambium24_MidCase_wholesale[col] = Cambium24_MidCase_wholesale['2025']
#display(Cambium24_MidCase_wholesale)

# Re-format the data frames: HighRECost
Cambium24_HighRECost_wholesale = wholesale_electricity_price_data_HighRECost.pivot_table(
    index='ba',
    columns='Year',
    values='Wholesale Electricity Price ($/kWh)'
)
Cambium24_HighRECost_wholesale = Cambium24_HighRECost_wholesale.reset_index()
Cambium24_HighRECost_wholesale.columns = [str(col) for col in Cambium24_HighRECost_wholesale.columns]
all_years_range = list(range(2014, 2051)) # 2014 to 2050 inclusive
desired_columns = ['ba'] + [str(year) for year in all_years_range]
Cambium24_HighRECost_wholesale = Cambium24_HighRECost_wholesale.set_index('ba')
Cambium24_HighRECost_wholesale = Cambium24_HighRECost_wholesale.reindex(columns=[str(year) for year in all_years_range])
Cambium24_HighRECost_wholesale = Cambium24_HighRECost_wholesale.reset_index()
for col in years_to_fill:
    Cambium24_HighRECost_wholesale[col] = Cambium24_HighRECost_wholesale['2025']
#display(Cambium24_HighRECost_wholesale)

### Interpolate missing values: 2026-2029, 2031-2034, 2036-2039, 2041-2044, 2046-2049

# Identify all year columns to interpolate (excluding 'ba')
columns_to_interpolate = [col for col in Cambium24_LowRECost_wholesale.columns if col != 'ba']

# Apply linear interpolation across the year columns (axis=1) for each row
Cambium24_LowRECost_wholesale[columns_to_interpolate] = \
    Cambium24_LowRECost_wholesale[columns_to_interpolate].interpolate(method='linear', axis=1)
display(Cambium24_LowRECost_wholesale)

Cambium24_MidCase_wholesale[columns_to_interpolate] = \
    Cambium24_MidCase_wholesale[columns_to_interpolate].interpolate(method='linear', axis=1)
display(Cambium24_MidCase_wholesale)

Cambium24_HighRECost_wholesale[columns_to_interpolate] = \
    Cambium24_HighRECost_wholesale[columns_to_interpolate].interpolate(method='linear', axis=1)
display(Cambium24_HighRECost_wholesale)

# Save the csv files
output_path = os.path.join(os.getcwd(), '..', 'wholesale_electricity_prices', 'wholesale_price_projections_cambium24_lowrecosts.csv')
Cambium24_LowRECost_wholesale.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'wholesale_electricity_prices', 'wholesale_price_projections_cambium24_midcase.csv')
Cambium24_MidCase_wholesale.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'wholesale_electricity_prices', 'wholesale_price_projections_cambium24_highrecosts.csv')
Cambium24_HighRECost_wholesale.to_csv(output_path, index=False)

,ba,2014,2015,2016,2017,2018,2019,2020,2021,2022,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,p1,0.04262,0.04262,0.04262,0.04262,0.04262,0.04262,0.04262,0.04262,0.04262,...,0.042112,0.041444,0.040776,0.040108,0.03944,0.038902,0.038364,0.037826,0.037288,0.03675
1,p10,0.07253,0.07253,0.07253,0.07253,0.07253,0.07253,0.07253,0.07253,0.07253,...,0.061538,0.061056,0.060574,0.060092,0.05961,0.059278,0.058946,0.058614,0.058282,0.05795
2,p100,0.06319,0.06319,0.06319,0.06319,0.06319,0.06319,0.06319,0.06319,0.06319,...,0.057440,0.057880,0.058320,0.058760,0.05920,0.058416,0.057632,0.056848,0.056064,0.05528
3,p101,0.07282,0.07282,0.07282,0.07282,0.07282,0.07282,0.07282,0.07282,0.07282,...,0.056308,0.056016,0.055724,0.055432,0.05514,0.054878,0.054616,0.054354,0.054092,0.05383
4,p102,0.06835,0.06835,0.06835,0.06835,0.06835,0.06835,0.06835,0.06835,0.06835,...,0.059738,0.059436,0.059134,0.058832,0.05853,0.058044,0.057558,0.057072,0.056586,0.05610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,p96,0.06954,0.06954,0.06954,0.06954,0.06954,0.06954,0.06954,0.06954,0.06954,...,0.047918,0.046896,0.045874,0.044852,0.04383,0.043174,0.042518,0.041862,0.041206,0.04055
129,p97,0.06575,0.06575,0.06575,0.06575,0.06575,0.06575,0.06575,0.06575,0.06575,...,0.053986,0.053612,0.053238,0.052864,0.05249,0.051756,0.051022,0.050288,0.049554,0.04882
130,p98,0.06932,0.06932,0.06932,0.06932,0.06932,0.06932,0.06932,0.06932,0.06932,...,0.047572,0.047474,0.047376,0.047278,0.04718,0.046192,0.045204,0.044216,0.043228,0.04224
131,p99,0.06480,0.06480,0.06480,0.06480,0.06480,0.06480,0.06480,0.06480,0.06480,...,0.060022,0.061944,0.063866,0.065788,0.06771,0.067376,0.067042,0.066708,0.066374,0.06604


,ba,2014,2015,2016,2017,2018,2019,2020,2021,2022,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,p1,0.04576,0.04576,0.04576,0.04576,0.04576,0.04576,0.04576,0.04576,0.04576,...,0.043892,0.043464,0.043036,0.042608,0.04218,0.041894,0.041608,0.041322,0.041036,0.04075
1,p10,0.07322,0.07322,0.07322,0.07322,0.07322,0.07322,0.07322,0.07322,0.07322,...,0.062094,0.061078,0.060062,0.059046,0.05803,0.058426,0.058822,0.059218,0.059614,0.06001
2,p100,0.06496,0.06496,0.06496,0.06496,0.06496,0.06496,0.06496,0.06496,0.06496,...,0.058764,0.059108,0.059452,0.059796,0.06014,0.060182,0.060224,0.060266,0.060308,0.06035
3,p101,0.07489,0.07489,0.07489,0.07489,0.07489,0.07489,0.07489,0.07489,0.07489,...,0.059202,0.058694,0.058186,0.057678,0.05717,0.056828,0.056486,0.056144,0.055802,0.05546
4,p102,0.07141,0.07141,0.07141,0.07141,0.07141,0.07141,0.07141,0.07141,0.07141,...,0.062522,0.061854,0.061186,0.060518,0.05985,0.059346,0.058842,0.058338,0.057834,0.05733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,p96,0.07181,0.07181,0.07181,0.07181,0.07181,0.07181,0.07181,0.07181,0.07181,...,0.054074,0.053138,0.052202,0.051266,0.05033,0.049510,0.048690,0.047870,0.047050,0.04623
129,p97,0.06814,0.06814,0.06814,0.06814,0.06814,0.06814,0.06814,0.06814,0.06814,...,0.058732,0.058104,0.057476,0.056848,0.05622,0.055626,0.055032,0.054438,0.053844,0.05325
130,p98,0.07165,0.07165,0.07165,0.07165,0.07165,0.07165,0.07165,0.07165,0.07165,...,0.051178,0.051796,0.052414,0.053032,0.05365,0.053134,0.052618,0.052102,0.051586,0.05107
131,p99,0.06663,0.06663,0.06663,0.06663,0.06663,0.06663,0.06663,0.06663,0.06663,...,0.062238,0.064846,0.067454,0.070062,0.07267,0.072380,0.072090,0.071800,0.071510,0.07122


,ba,2014,2015,2016,2017,2018,2019,2020,2021,2022,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,p1,0.04929,0.04929,0.04929,0.04929,0.04929,0.04929,0.04929,0.04929,0.04929,...,0.048404,0.048588,0.048772,0.048956,0.04914,0.048622,0.048104,0.047586,0.047068,0.04655
1,p10,0.07328,0.07328,0.07328,0.07328,0.07328,0.07328,0.07328,0.07328,0.07328,...,0.062116,0.060602,0.059088,0.057574,0.05606,0.056304,0.056548,0.056792,0.057036,0.05728
2,p100,0.06584,0.06584,0.06584,0.06584,0.06584,0.06584,0.06584,0.06584,0.06584,...,0.063612,0.063624,0.063636,0.063648,0.06366,0.063948,0.064236,0.064524,0.064812,0.06510
3,p101,0.07377,0.07377,0.07377,0.07377,0.07377,0.07377,0.07377,0.07377,0.07377,...,0.063662,0.062954,0.062246,0.061538,0.06083,0.060158,0.059486,0.058814,0.058142,0.05747
4,p102,0.07082,0.07082,0.07082,0.07082,0.07082,0.07082,0.07082,0.07082,0.07082,...,0.065216,0.064412,0.063608,0.062804,0.06200,0.061390,0.060780,0.060170,0.059560,0.05895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,p96,0.07205,0.07205,0.07205,0.07205,0.07205,0.07205,0.07205,0.07205,0.07205,...,0.059314,0.058788,0.058262,0.057736,0.05721,0.056526,0.055842,0.055158,0.054474,0.05379
129,p97,0.06881,0.06881,0.06881,0.06881,0.06881,0.06881,0.06881,0.06881,0.06881,...,0.061770,0.061100,0.060430,0.059760,0.05909,0.058368,0.057646,0.056924,0.056202,0.05548
130,p98,0.07205,0.07205,0.07205,0.07205,0.07205,0.07205,0.07205,0.07205,0.07205,...,0.060674,0.060128,0.059582,0.059036,0.05849,0.058200,0.057910,0.057620,0.057330,0.05704
131,p99,0.06753,0.06753,0.06753,0.06753,0.06753,0.06753,0.06753,0.06753,0.06753,...,0.067536,0.069792,0.072048,0.074304,0.07656,0.076528,0.076496,0.076464,0.076432,0.07640


## Electricity Rates

In [100]:
### Input Data Processing: Electricity Rates

# Input Folder: N/A
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: N/A
# Data Source: URDB

#--------------------#

# Import the original agent files
agents_original_file_path = os.path.join(original_agent_data_folder, agents_original_file)
agents_original = pd.read_parquet(agents_original_file_path)
display(agents_original)

,gid,pgid,hdf_index,county_id,lat,lon,state_abbr,census_division_abbr,sector_abbr,application,...,census_tract_2020,census_block_group_2010,census_block_group_2020,units_number,garage_carport_type,garage_code_desc,parking_spaces,county_file,val_market,val_transfer
0,5ae8f1f4-db15-4979-923f-cf47adca67cb,9913187,,1929,31.673914,-95.758926,TX,WSC,res,BTM,...,48001950901,3,4800195090130,0,None,None,0,anderson,54011.0,249.0
1,021bfa2f-77c0-4362-bba2-e7508e307ac6,9912423,,1929,31.772570,-95.639069,TX,WSC,res,BTM,...,48001950500,4,4800195050040,0,None,None,0,anderson,43582.0,249.0
2,2c7df7e0-f0cc-4063-b32e-a4280bc13d3f,9913029,,1929,31.892509,-95.476234,TX,WSC,res,"BTM, FOM",...,48001950100,None,4800195010021,0,None,None,0,anderson,190972.0,249.0
3,0e5421b0-6a06-4de2-97bb-d058797de671,9912503,,1929,31.705729,-95.625839,TX,WSC,res,"BTM, FOM",...,48001950901,2,4800195090120,0,None,None,0,anderson,91632.0,249.0
4,077d46b8-5438-42cb-b4e5-227843ba98e9,9913918,,1929,31.729355,-95.604027,TX,WSC,res,BTM,...,48001950800,1,4800195080010,0,None,None,0,anderson,7797.0,249.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8978476,269e905e-167e-4a8e-bbe4-1bcd8a7ce4fb,9416741,,2621,28.712814,-99.831604,TX,WSC,com,"BTM, FOM",...,48507950301,2,4850795030120,0,None,None,0,zavala,228540.0,1499.0
8978477,35343710-6605-4d4e-bd7a-f6e7be58be2f,9417079,,2621,28.672693,-99.835518,TX,WSC,com,"BTM, FOM",...,48507950302,1,4850795030210,0,None,None,0,zavala,298540.0,1499.0
8978478,916cd2b0-a4e9-4655-ba49-e5f1a84833a6,9416572,,2621,28.939903,-99.833992,TX,WSC,ind,"BTM, FOM, Utility",...,48507950200,1,4850795020012,0,None,None,0,zavala,57190.0,1499.0
8978479,99429a41-7aa8-414e-8df4-0d8f3d09e2e9,9416741,,2621,28.692505,-99.809959,TX,WSC,com,"BTM, FOM",...,48507950301,2,4850795030120,0,None,None,0,zavala,23060.0,1499.0


In [101]:
# SUPPORT FUNCTIONS

# FUNCTION 1. -----------------------------------------#

def URDBv8_to_ElectricityRates(urdb_response):
    """
    Formats response from Utility Rate Database API version 8 for use in PySAM
        i.e.
            model = PySAM.UtilityRate5.new()
            rates = PySAM.ResourceTools.URDBv8_to_ElectricityRates(urdb_response)
            model.ElectricityRates.assign(rates)

    This function does the additional processing when these rate features
    are present. 

    :param: urdb_response:
        dictionary with response fields following
        https://openei.org/services/doc/rest/util_rates/?version=8
    :return: dictionary for PySAM.UtilityRate5.UtilityRate5.ElectricityRates
    """
    urdb_data = dict()
    urdb_data['en_electricity_rates'] = 1
    display(urdb_response)

    def try_get_schedule(urdb_name, data_name):
        if urdb_name in urdb_response.keys():
            urdb_data[data_name] = urdb_response[urdb_name]
            for i in range(12):
                for j in range(24):
                    urdb_data[data_name][i][j] += 1

    def try_get_rate_structure(urdb_name, data_name):
        mat = []
        supported_units = {
            "kwh" : 0,
            "kwh/kw" : 1,
            "kwh daily" : 2,
            "kwh/kw daily" : 3,
            "kwh/kva": 4
        }
        if urdb_name in urdb_response.keys():
            structure = urdb_response[urdb_name]
            for i, period in enumerate(structure):
                for j, entry in enumerate(period):
                    rate = entry.get('rate', 0)
                    if 'adj' in entry.keys():
                        rate += entry['adj']
                    tier_max = 1e38
                    if 'max' in entry.keys():
                        tier_max = entry['max']
                    sell = 0
                    if 'sell' in entry.keys():
                        sell = entry['sell']
                    units = 0
                    if 'unit' in entry.keys():
                        try:
                            units = supported_units[entry['unit'].lower()]
                        except KeyError:
                            raise RuntimeError("UtilityRateDatabase error: unrecognized unit in rate structure")
                    mat.append((i + 1, j + 1, tier_max, units, rate, sell))
            urdb_data[data_name] = mat

    def try_get_demand_structure(urdb_name, data_name):
        mat = []
        if urdb_name in urdb_response.keys():
            structure = urdb_response[urdb_name]
            for i, period in enumerate(structure):
                for j, entry in enumerate(period):
                    rate = entry.get('rate', 0)
                    if 'adj' in entry.keys():
                        rate += entry['adj']
                    tier_max = 1e38
                    if 'max' in entry.keys():
                        tier_max = entry['max']
                    if 'unit' in entry.keys():
                        if entry['unit'].lower() != "kW".lower():
                            raise RuntimeError("UtilityRateDatabase error: unrecognized unit in rate structure")
                    mat.append((i + 1, j + 1, tier_max, rate))
            if data_name:
                urdb_data[data_name] = mat
            else:
                return mat

    if "dgrules" in urdb_response.keys():
        rules = urdb_response['dgrules']  # dgrules
        if rules == "Net Metering":
            urdb_data['ur_metering_option'] = 0
        elif rules == "Net Billing Instantaneous":
            urdb_data['ur_metering_option'] = 2
        elif rules == "Net Billing Hourly":
            urdb_data['ur_metering_option'] = 3
        elif rules == "Buy All Sell All":
            urdb_data['ur_metering_option'] = 4
    else:
        # if no metering option provided, assume Net Metering
        urdb_data['ur_metering_option'] = 0

    if 'fixedchargefirstmeter' in urdb_response.keys() and 'fixedchargeunits' in urdb_response.keys():
        fixed_charge = urdb_response['fixedchargefirstmeter']
        fixed_charge_units = urdb_response['fixedchargeunits']
        if fixed_charge_units == "$/day":
            fixed_charge *= 365 / 12
        elif fixed_charge_units == "$/year":
            fixed_charge /= 12
        urdb_data['ur_monthly_fixed_charge'] = fixed_charge

    if 'mincharge' in urdb_response.keys():
        min_charge = urdb_response['mincharge']
        min_charge_units = urdb_response.get('minchargeunits', "$/month")
        if min_charge_units == "$/year":
            urdb_data['ur_annual_min_charge'] = min_charge
        else:
            if min_charge_units == "$/day":
                min_charge *= 365 / 12
            urdb_data['ur_monthly_min_charge'] = min_charge

    try_get_schedule('energyweekdayschedule', 'ur_ec_sched_weekday')
    try_get_schedule('energyweekendschedule', 'ur_ec_sched_weekend')
    try_get_rate_structure('energyratestructure', 'ur_ec_tou_mat')

    try_get_demand_structure('demandratestructure', 'ur_dc_tou_mat')
    try_get_schedule('demandweekdayschedule', 'ur_dc_sched_weekday')
    try_get_schedule('demandweekendschedule', 'ur_dc_sched_weekend')

    flat_demand_structure = try_get_demand_structure('flatdemandstructure', None)

    if 'flatdemandmonths' in urdb_response.keys():
        urdb_data['ur_dc_enable'] = 1
        flat_mat = []
        flat_demand = urdb_response['flatdemandmonths']
        for month, period in enumerate(flat_demand):
            tiers = []
            for r in flat_demand_structure:
                if r[0] == int(period + 1):
                    tiers.append(r)
                    
            if len(tiers) == 0:
                #raise ValueError("flatdemandstructure missing period number ", period)
                print(f"Warning: No flatdemandstructure tiers found for period {period}. Skipping demand charge data for this month.")
                continue # Skip to the next month/period in flat_demand
            for t in tiers:
                month_row = []
                month_row.append(month)
                month_row += [t[i] for i in (1, 2, 3)]
                flat_mat.append(month_row)
        urdb_data['ur_dc_flat_mat'] = flat_mat
    # Fill out an empty flat rate structure if the rate has TOU demand but not flat demand    
    elif "demandratestructure" in urdb_response.keys():
        urdb_data['ur_dc_enable'] = 1
        # Enumerate a dc_flat table with $0/kW in 12 months
        flat_mat = []
        for i in range(0, 12):
            month_mat = [i, 1, 1e38, 0]
            flat_mat.append(month_mat)
        urdb_data['ur_dc_flat_mat'] = flat_mat
    else:
        urdb_data['ur_dc_enable'] = 0

    if urdb_data['ur_dc_enable'] == 1 and "ur_dc_tou_mat" not in urdb_data.keys():
        urdb_data['ur_dc_tou_mat'] = [[1, 1, 1e38, 0], ]
        urdb_data['ur_dc_sched_weekday'] = [[1] * 24 for i in range(12)]
        urdb_data['ur_dc_sched_weekend'] = urdb_data['ur_dc_sched_weekday']

    has_billing_demand = False

    lookback_range = 0
    if 'lookbackrange' in urdb_response.keys():
        lookback_range = urdb_response['lookbackrange']
        urdb_data['ur_billing_demand_lookback_period'] = lookback_range 
    
    lbp = 0
    if 'lookbackpercent' in urdb_response.keys():
        lbp = urdb_response['lookbackpercent'] * 100.0
        
        # Some demand ratchets apply in every month - use this as the default if lookbackmonths is not specified
        lbm = [True] * 12
        if 'lookbackmonths' in urdb_response.keys():
            lbm = urdb_response['lookbackmonths']
        
        lookback_percentages = [ [0, 0] ] * 12
        for i in range(0, len(lookback_percentages)):
            if lbm[i]:
                lookback_percentages[i][0] = lbp

        urdb_data['ur_billing_demand_lookback_percentages'] = lookback_percentages
    
    has_billing_demand = lookback_range > 0 or lbp > 0

    if has_billing_demand:
        urdb_data['ur_enable_billing_demand'] = True
        # Handle variables that aren't currently in URDB but are in SAM
        urdb_data['ur_billing_demand_minimum'] = 0

        # Added these two lines
        if "ur_dc_tou_mat" not in urdb_data.keys():
            urdb_data['ur_dc_tou_mat'] = [[1, 1, 1e38, 0], ]
        
        dc_tou = urdb_data['ur_dc_tou_mat']
        max_period = 0
        for i in range(0, len(dc_tou)):
            if dc_tou[i][0] > max_period:
                max_period = dc_tou[i][0] 
        
        bd_periods = []
        for i in range(0, max_period):
            bd_periods.append([i+1, 1])
        
        urdb_data['ur_dc_billing_demand_periods'] = bd_periods

        print("Billing Demand Notice.\nThis rate includes billing demand adjustments and/or demand ratchets that may not be accurately reflected in the data downloaded from the URDB. Please check the information in the Description under Description and Applicability and review the rate sheet to be sure the billing demand inputs are correct.")
    else:
        urdb_data['ur_enable_billing_demand'] = False

    return urdb_data

# FUNCTION 2. -----------------------------------------#

def get_urdb_rate_data(page, key):
    """
    Using page value and user API key, returns json file containing specified tariff dictionary
    
    page: 'str' : required key for URDB website for specific eia_id tariff 
    key: 'str' : User specific API key for URDB website
    """
    
    # Full API can be viewed at: https://openei.org/services/doc/rest/util_rates/?version=8
    urdb_url = 'https://api.openei.org/utility_rates?version=8&format=json&detail=full&limit=3'
    get_url = urdb_url + '&api_key={api_key}&getpage={page_id}'.format(api_key=key, page_id=page)


    base_filename = "urdb_rate_{}.json".format(page)
    filename = os.path.join(rate_json_data_folder, base_filename)

    if not os.path.isfile(filename):
        print(get_url)
        CA = certifi.where()
        resp = requests.get(get_url, verify=CA)
    
        data = resp.json()
        
        # Cache rate as file
        if "error" not in data:
            with open(filename, 'w') as f:
                f.write(json.dumps(data, sort_keys=True, indent=2, separators=(',', ': ')))
    else:
        with open(filename, 'r') as f:
            data = json.load(f)

    return data

# FUNCTION 3. -----------------------------------------#

# Function to automatically grab each tariff dictionary from URDB website 
def get_tariff_dict(page_list, api_key):
    """
    Takes in list of relevant page labels for URDB site, returns tariff dictionary list that matches
    order of relevant FZ
    
    page_list : list : list of strings for looking up and retrieving tariff dictionaries of interest
    api_key : str : user specific API key for URDB website
    """
    # Empty dictionary for storing returned tariff dictionaries
    tar = {}
    
    # Loop through each page in the page_list
    for page in page_list:
        # Get tariff dictionary
        urdb_response = get_urdb_rate_data(page, api_key)

        if urdb_response["items"]:
        
            # Process tariff dictionary
            rates = URDBv8_to_ElectricityRates(urdb_response["items"][0]) 
            
            # Append tariff dictionary to empty list
            tar[page] = rates

        else:
            tar[page] = None

    return tar

# FUNCTION 4. -----------------------------------------#

# Function to get the URDB rates as a CSV using the API
def get_all_urdb_rates_csv(api_key, sector):
    """
    Polls the Utility Rate Database (URDB) API for all rate data for a given sector
    by paginating through results, and returns them as a single Pandas DataFrame.

    Args:
        api_key (str): Your URDB API key.
        sector (str): The sector to filter rates by (e.g., 'Residential', 'Commercial').

    Returns:
        pd.DataFrame: A DataFrame containing all extracted rate data, or an empty DataFrame
                      if no data is found or an error occurs.
    """

    # URDB API base URL
    base_url = 'https://api.openei.org/utility_rates?'

    all_rates_df = pd.DataFrame() # Initialize an empty DataFrame to store all results
    offset = 0
    limit = 500 # Maximum limit per request as per URDB API documentation

    while True:
        params = {
            'version': 'latest',
            'format': 'csv',
            'api_key': api_key,
            'sector': sector,
            'limit': limit,
            'offset': offset
        }

        print(f"Making API request to: {base_url} with parameters: {params}")

        try:
            response = requests.get(base_url, params=params)
            response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

            # Read the CSV response directly into a Pandas DataFrame
            current_page_df = pd.read_csv(io.StringIO(response.text))

            if not current_page_df.empty:
                print(f"  Retrieved {len(current_page_df)} rates from offset {offset}.")
                all_rates_df = pd.concat([all_rates_df, current_page_df], ignore_index=True)
                offset += limit # Increment offset for the next page
                time.sleep(1) # Add a small delay to be polite to the API server
            else:
                print(f"  No more rates found from offset {offset}. Ending pagination.")
                break # Exit the loop if no more data is returned

        except requests.exceptions.HTTPError as http_err:
            print(f"HTTP error occurred: {http_err} - Response: {response.text}")
            break # Exit on error
        except requests.exceptions.ConnectionError as conn_err:
            print(f"Connection error occurred: {conn_err} - Check your internet connection.")
            break
        except requests.exceptions.Timeout as timeout_err:
            print(f"Timeout error occurred: {timeout_err} - API took too long to respond.")
            break
        except requests.exceptions.RequestException as req_err:
            print(f"An unexpected request error occurred: {req_err}")
            break
        except pd.errors.EmptyDataError:
            print(f"Received empty CSV data from the API at offset {offset}. This might indicate the end of data or an issue.")
            break
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            break
    
    print(f"Pagination complete. Total rates retrieved for sector '{sector}': {len(all_rates_df)}")
    return all_rates_df

In [5]:
# Extract all of the URDB residential rates
residential_rates_all = get_all_urdb_rates_csv(api_key=urdb_api_key, sector='Residential')
residential_rates_all['eiaid'] = residential_rates_all['eiaid'].astype(str)
#display(residential_rates_all)

# Save as csv file
residential_rates_file_path = os.path.join(rate_data_folder, 'residential_rates.csv')
residential_rates_all.to_csv(residential_rates_file_path, index=False)

Making API request to: https://api.openei.org/utility_rates? with parameters: {'version': 'latest', 'format': 'csv', 'api_key': 'Ro9gfEvVHefX5C7TxRSrvi8x5ieLHLBMCAtES70t', 'sector': 'Residential', 'limit': 500, 'offset': 0}
  Retrieved 500 rates from offset 0.
Making API request to: https://api.openei.org/utility_rates? with parameters: {'version': 'latest', 'format': 'csv', 'api_key': 'Ro9gfEvVHefX5C7TxRSrvi8x5ieLHLBMCAtES70t', 'sector': 'Residential', 'limit': 500, 'offset': 500}
  Retrieved 500 rates from offset 500.
Making API request to: https://api.openei.org/utility_rates? with parameters: {'version': 'latest', 'format': 'csv', 'api_key': 'Ro9gfEvVHefX5C7TxRSrvi8x5ieLHLBMCAtES70t', 'sector': 'Residential', 'limit': 500, 'offset': 1000}
  Retrieved 500 rates from offset 1000.
Making API request to: https://api.openei.org/utility_rates? with parameters: {'version': 'latest', 'format': 'csv', 'api_key': 'Ro9gfEvVHefX5C7TxRSrvi8x5ieLHLBMCAtES70t', 'sector': 'Residential', 'limit': 5

In [6]:
# Extract all of the URDB commercial rates
commercial_rates_all = get_all_urdb_rates_csv(api_key=urdb_api_key, sector='Commercial')
commercial_rates_all['eiaid'] = commercial_rates_all['eiaid'].astype(str)
#display(commercial_rates_all)

# Save as csv file
commercial_rates_file_path = os.path.join(rate_data_folder, 'commercial_rates.csv')
commercial_rates_all.to_csv(commercial_rates_file_path, index=False)

Making API request to: https://api.openei.org/utility_rates? with parameters: {'version': 'latest', 'format': 'csv', 'api_key': 'Ro9gfEvVHefX5C7TxRSrvi8x5ieLHLBMCAtES70t', 'sector': 'Commercial', 'limit': 500, 'offset': 0}
  Retrieved 500 rates from offset 0.
Making API request to: https://api.openei.org/utility_rates? with parameters: {'version': 'latest', 'format': 'csv', 'api_key': 'Ro9gfEvVHefX5C7TxRSrvi8x5ieLHLBMCAtES70t', 'sector': 'Commercial', 'limit': 500, 'offset': 500}
  Retrieved 500 rates from offset 500.
Making API request to: https://api.openei.org/utility_rates? with parameters: {'version': 'latest', 'format': 'csv', 'api_key': 'Ro9gfEvVHefX5C7TxRSrvi8x5ieLHLBMCAtES70t', 'sector': 'Commercial', 'limit': 500, 'offset': 1000}
  Retrieved 500 rates from offset 1000.
Making API request to: https://api.openei.org/utility_rates? with parameters: {'version': 'latest', 'format': 'csv', 'api_key': 'Ro9gfEvVHefX5C7TxRSrvi8x5ieLHLBMCAtES70t', 'sector': 'Commercial', 'limit': 500, 

In [116]:
# Combine the residential_rates_all and commercial_rates_all data frames
rates_all = pd.concat([residential_rates_all, commercial_rates_all], ignore_index=True)

# Create a data frame where only the unique combinations of 'name' and 'eiaid' are kept for residential_rates_subset (use the latest entry to keep the most recent rate)
rates_subset_unique = rates_all.drop_duplicates(subset=['name', 'eiaid'], keep='last')

# Create an agents data frame where the urdb labels for matching 'name' ('tariff_name') and 'eiaid' ('eia_id') are copied from rates_subset_unique 
agents = pd.merge(
    agents_original,
    rates_subset_unique[['name', 'eiaid', 'label']], # Select only necessary columns from the right data frame
    left_on=['rate_name', 'eia_id'],
    right_on=['name', 'eiaid'],
    how='left' # Keep all rows from residential_agents_original
)
agents.drop(columns=['eiaid', 'name'], inplace=True)
agents = agents.rename(columns={'label': 'urdb_rate_id_new'}) # Re-name the 'label' column to urdb_rate_id_new 
display(agents)

# Save as csv file
#updated_agents_file_path = os.path.join(updated_agent_data_folder, f'agents_{state}.csv')
#agents.to_csv(updated_agents_file_path, index=False)

,gid,pgid,hdf_index,county_id,lat,lon,state_abbr,census_division_abbr,sector_abbr,application,...,census_block_group_2010,census_block_group_2020,units_number,garage_carport_type,garage_code_desc,parking_spaces,county_file,val_market,val_transfer,urdb_rate_id_new
0,5ae8f1f4-db15-4979-923f-cf47adca67cb,9913187,,1929,31.673914,-95.758926,TX,WSC,res,BTM,...,3,4800195090130,0,None,None,0,anderson,54011.0,249.0,NaN
1,021bfa2f-77c0-4362-bba2-e7508e307ac6,9912423,,1929,31.772570,-95.639069,TX,WSC,res,BTM,...,4,4800195050040,0,None,None,0,anderson,43582.0,249.0,NaN
2,2c7df7e0-f0cc-4063-b32e-a4280bc13d3f,9913029,,1929,31.892509,-95.476234,TX,WSC,res,"BTM, FOM",...,None,4800195010021,0,None,None,0,anderson,190972.0,249.0,NaN
3,0e5421b0-6a06-4de2-97bb-d058797de671,9912503,,1929,31.705729,-95.625839,TX,WSC,res,"BTM, FOM",...,2,4800195090120,0,None,None,0,anderson,91632.0,249.0,NaN
4,077d46b8-5438-42cb-b4e5-227843ba98e9,9913918,,1929,31.729355,-95.604027,TX,WSC,res,BTM,...,1,4800195080010,0,None,None,0,anderson,7797.0,249.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8978476,269e905e-167e-4a8e-bbe4-1bcd8a7ce4fb,9416741,,2621,28.712814,-99.831604,TX,WSC,com,"BTM, FOM",...,2,4850795030120,0,None,None,0,zavala,228540.0,1499.0,539fb91bec4f024bc1dc1f11
8978477,35343710-6605-4d4e-bd7a-f6e7be58be2f,9417079,,2621,28.672693,-99.835518,TX,WSC,com,"BTM, FOM",...,1,4850795030210,0,None,None,0,zavala,298540.0,1499.0,539fb91bec4f024bc1dc1f11
8978478,916cd2b0-a4e9-4655-ba49-e5f1a84833a6,9416572,,2621,28.939903,-99.833992,TX,WSC,ind,"BTM, FOM, Utility",...,1,4850795020012,0,None,None,0,zavala,57190.0,1499.0,NaN
8978479,99429a41-7aa8-414e-8df4-0d8f3d09e2e9,9416741,,2621,28.692505,-99.809959,TX,WSC,com,"BTM, FOM",...,2,4850795030120,0,None,None,0,zavala,23060.0,1499.0,539fb91bec4f024bc1dc1f11


In [118]:
##### CODE FOR TEXAS ONLY #####

# Define the list of values to replace
old_ids = [
    'temp_mongoid_for_ss19_05', 
    'temp_mongoid_for_ss19_04', 
    'temp_mongoid_for_ss19_01', 
    'temp_mongoid_for_ss19_02', 
    'temp_mongoid_for_ss19_03'
]

# Define the new value
new_id = '5cca2042b718b3716b5cf5d0'

# Replace the values
agents['urdb_rate_id'] = agents['urdb_rate_id'].replace(old_ids, new_id)

In [120]:
# Determine the unique 'urdb_rate_id' values for the agents that currently do not have an associated 'urdb_rate_id_new' value
nan_label_mask = agents['urdb_rate_id_new'].isna()
urdb_id_for_nan_labels = agents.loc[nan_label_mask, 'urdb_rate_id']
unique_urdb_id_nan_labels = urdb_id_for_nan_labels.unique()
count_unique_urdb_id = len(unique_urdb_id_nan_labels)
display(unique_urdb_id_nan_labels)
display(count_unique_urdb_id)
condition_to_fill = nan_label_mask & agents['urdb_rate_id'].isin(unique_urdb_id_nan_labels)
agents.loc[condition_to_fill, 'urdb_rate_id_new'] = agents.loc[condition_to_fill, 'urdb_rate_id']

# Map the 'urdb_rate_id' values to the 'urdb_rate_id_new' values
rates_urdb_mapping = agents[['urdb_rate_id', 'urdb_rate_id_new']].copy()
rates_urdb_mapping = rates_urdb_mapping.drop_duplicates(subset=['urdb_rate_id', 'urdb_rate_id_new'], keep='last')

# Save as csv file
urdb_labels_filepath = os.path.join(rate_state_data_folder, f'urdb_labels_{state}.csv')
rates_urdb_mapping.to_csv(urdb_labels_filepath, index=False)

# Extract the 'urdb_rate_id_new' values for the  agents
urdb_labels = rates_urdb_mapping['urdb_rate_id_new']
urdb_labels = urdb_labels.dropna() # drop NA values
urdb_labels = list(set(urdb_labels)) # drop duplicates
urdb_labels.sort() # sort alphabetically
urdb_labels = pd.Series(urdb_labels, name='urdb_rate_id_new')
display(urdb_labels)

# Pull _tariff_dictionary values
tariff_dictionary = get_tariff_dict(list(urdb_labels), urdb_api_key)
display(tariff_dictionary)

array(['5cca2042b718b3716b5cf5d0', '5cd0ad9d5457a3176654e9d6'],
      dtype=object)

2

0     539f6d9dec4f024411ecb7ff
1     539f6fcbec4f024411ecd2e1
2     539f70e5ec4f024411ecdfeb
3     539f72b0ec4f024411ecf41b
4     539f7315ec4f024411ecf8a7
5     539f73d4ec4f024411ed0179
6     539f762dec4f024411ed1c3b
7     539f764dec4f024411ed1d9b
8     539fb3edec4f024bc1dbe351
9     539fb434ec4f024bc1dbe66f
10    539fb48bec4f024bc1dbeac7
11    539fb4c0ec4f024bc1dbed4b
12    539fb773ec4f024bc1dc0c27
13    539fb858ec4f024bc1dc1609
14    539fb8aeec4f024bc1dc1a0d
15    539fb91bec4f024bc1dc1f11
16    539fbaaeec4f024bc1dc3085
17    539fc033ec4f024c27d8999b
18    539fc0c8ec4f024c27d8a055
19    539fc0ecec4f024c27d8a141
20    539fc9d6ec4f024d2f53f566
21    539fcb22ec4f024d2f5402e8
22    53d95b695257a379669598a6
23    54184b4b5257a3473865dec5
24    541b20a75257a3b77adaf905
25    5b3117265457a33c498367bf
26    5b4634835457a3e51da907e9
27    5b5f811a5457a31f19409a7b
28    5b730e305457a33312af0aeb
29    5b74b5995457a3784eaf0aec
30    5b77495f5457a3b064af0aec
31    5cca2042b718b3716b5cf5d0
32    5c

{'approved': True,
 'country': 'USA',
 'demandrateunit': 'kW',
 'demandunits': 'kW',
 'description': '*The General Service Rate Schedule applies to all electric service to permanent installations used primarily for residential purposes. It applies to electric service to a private dwelling and its appurtenances, the major use of which is for lighting, air conditioning, heating and household appliances. Except as otherwise provided, when a majority of the electricity consumed in a dwelling is regularly used in connection with the conduct of a business, the service shall be classified as Commercial or Large Power and billed under the appropriate rate.\n*In addition to the rates and charges described above, members will be billed a Power Cost Pass-Through for all kWh consumed.',
 'eiaid': 3282,
 'enddate': 1364734800,
 'energyratestructure': [[{'rate': 0.024}]],
 'energyweekdayschedule': [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
  

{'approved': True,
 'country': 'USA',
 'demandrateunit': 'kW',
 'demandunits': 'kW',
 'eiaid': 13418,
 'energyratestructure': [[{'rate': 0.0679, 'unit': 'kWh'}],
  [{'rate': 0.0579, 'unit': 'kWh'}]],
 'energyweekdayschedule': [[1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 1, 1, 1, 1, 1, 1

...

{'approved': True,
 'country': 'USA',
 'demandrateunit': 'kW',
 'demandunits': 'kW',
 'description': "This rate is applicable under the regular terms and conditions of the Company for single\r\nfamily residences or individual apartments or appurtenant domestic purposes. This rate\r\nis not applicable to service for common facilities at apartments and other multi-dwelling\r\nunits. Service will be single-phase except that three-phase service may be rendered\r\nhereunder, at Company's option, where such service is available. Where a Customer\r\nhas more than one meter, each meter shall be billed separately. The Customer shall not\r\nresell any energy purchased under this rate schedule or supply energy to another\r\noccupied dwelling. Standby, maintenance, or supplemental service is not applicable\r\nhereunder except in connection with a contract for service pursuant to the Company�s\r\ntariff for Interconnection and Parallel Operation of Distributed Generation (IPODG). For\r\ncustomers r

{'approved': True,
 'country': 'USA',
 'demandrateunit': 'kW',
 'demandunits': 'kW',
 'description': 'This rate is applicable on a voluntary basis for customers who qualify for service under\r\nSchedule RS, under the regular terms and conditions of the Company, for single family\r\nresidences or individual apartments or appurtenant domestic purposes. Where a\r\nCustomer has more than one meter, each meter will be billed separately. The Customer\r\nshall not resell any energy purchased under this rate schedule, or supply energy to\r\nanother occupied dwelling. Standby, maintenance, or supplemental service is not\r\napplicable hereunder except in connection with a contract for service pursuant to the\r\nCompany�s tariff for Interconnection and Parallel Operation of Distributed Generation\r\n(IPODG). For customers receiving service pursuant to IPODG and also requesting\r\nservice under the Standby and Maintenance Service Rider, Schedule SMS, the Billing\r\nDemand as defined in SMS will be

{'approved': True,
 'country': 'USA',
 'demandrateunit': 'kW',
 'demandunits': 'kW',
 'description': 'Applicable to all Customers taking the type of service described in this rate\r\nschedule for all of the Electric Service supplied at one Point of Delivery and\r\nmeasured through one meter for domestic uses associated with the operation\r\nof a single family or multi-family residential installation. Not applicable to\r\ntemporary or shared service.',
 'dgrules': 'Buy All Sell All',
 'eiaid': 5078,
 'energycomments': 'Adjustment = Power Cost Recovery Factor + Securitized Charges Recovery Factor',
 'energyratestructure': [[{'adj': -0.0051,
    'max': 700,
    'rate': 0.1274,
    'unit': 'kWh'}]],
 'energyweekdayschedule': [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

{'approved': True,
 'country': 'USA',
 'demandrateunit': 'kW',
 'demandunits': 'kW',
 'description': 'To residential customers for electric service used for domestic purposes in private residences and separately metered individual apartments when all service is supplied at one point of delivery, and measured through on kilowatt hour meter, where facilities of adequate capacity and suitable voltage are adjacent to the premises to be served. Single-phase motors not to exceed 10 horsepower, individual capacity, may be served under this rate.',
 'eiaid': 17718,
 'energycomments': 'Adjustments= RPS Cost Rider + Fuel and Purchased Power Cost Adjustment Clause + Grid Modernization Rider',
 'energyratestructure': [[{'adj': 0.00258, 'rate': 0.093683, 'unit': 'kWh'}],
  [{'adj': 0.00258, 'rate': 0.11242, 'unit': 'kWh'}]],
 'energyweekdayschedule': [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0, 

{'539f6d9dec4f024411ecb7ff': {'en_electricity_rates': 1,
  'ur_metering_option': 0,
  'ur_monthly_fixed_charge': 22.5,
  'ur_ec_sched_weekday': [[1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [122]:
# Populate the 'tariff_dict' column in the 'agents' data frame
agents['tariff_dict'] = agents['urdb_rate_id_new'].map(tariff_dictionary)
display(agents)

# Replace the values in the 'urdb_rate_id' column with the values in the 'urdb_rate_id_new' column and then remove the 'urdb_rate_id_new' column
agents['urdb_rate_id'] = agents['urdb_rate_id_new']
agents = agents.drop(columns=['urdb_rate_id_new'])

agents['tariff_dict'] = agents['tariff_dict'].apply(
    lambda x: json.dumps(x) if isinstance(x, dict) else x
)

# Save as parquet file
updated_agents_filepath = os.path.join(updated_agent_data_folder, f'agents_{state}.parquet')
agents.to_parquet(updated_agents_filepath)

,gid,pgid,hdf_index,county_id,lat,lon,state_abbr,census_division_abbr,sector_abbr,application,...,census_block_group_2020,units_number,garage_carport_type,garage_code_desc,parking_spaces,county_file,val_market,val_transfer,urdb_rate_id_new,tariff_dict
0,5ae8f1f4-db15-4979-923f-cf47adca67cb,9913187,,1929,31.673914,-95.758926,TX,WSC,res,BTM,...,4800195090130,0,None,None,0,anderson,54011.0,249.0,5cca2042b718b3716b5cf5d0,None
1,021bfa2f-77c0-4362-bba2-e7508e307ac6,9912423,,1929,31.772570,-95.639069,TX,WSC,res,BTM,...,4800195050040,0,None,None,0,anderson,43582.0,249.0,5cca2042b718b3716b5cf5d0,None
2,2c7df7e0-f0cc-4063-b32e-a4280bc13d3f,9913029,,1929,31.892509,-95.476234,TX,WSC,res,"BTM, FOM",...,4800195010021,0,None,None,0,anderson,190972.0,249.0,5cca2042b718b3716b5cf5d0,None
3,0e5421b0-6a06-4de2-97bb-d058797de671,9912503,,1929,31.705729,-95.625839,TX,WSC,res,"BTM, FOM",...,4800195090120,0,None,None,0,anderson,91632.0,249.0,5cca2042b718b3716b5cf5d0,None
4,077d46b8-5438-42cb-b4e5-227843ba98e9,9913918,,1929,31.729355,-95.604027,TX,WSC,res,BTM,...,4800195080010,0,None,None,0,anderson,7797.0,249.0,5cca2042b718b3716b5cf5d0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8978476,269e905e-167e-4a8e-bbe4-1bcd8a7ce4fb,9416741,,2621,28.712814,-99.831604,TX,WSC,com,"BTM, FOM",...,4850795030120,0,None,None,0,zavala,228540.0,1499.0,539fb91bec4f024bc1dc1f11,"{'en_electricity_rates': 1, 'ur_metering_optio..."
8978477,35343710-6605-4d4e-bd7a-f6e7be58be2f,9417079,,2621,28.672693,-99.835518,TX,WSC,com,"BTM, FOM",...,4850795030210,0,None,None,0,zavala,298540.0,1499.0,539fb91bec4f024bc1dc1f11,"{'en_electricity_rates': 1, 'ur_metering_optio..."
8978478,916cd2b0-a4e9-4655-ba49-e5f1a84833a6,9416572,,2621,28.939903,-99.833992,TX,WSC,ind,"BTM, FOM, Utility",...,4850795020012,0,None,None,0,zavala,57190.0,1499.0,5cd0ad9d5457a3176654e9d6,"{'en_electricity_rates': 1, 'ur_metering_optio..."
8978479,99429a41-7aa8-414e-8df4-0d8f3d09e2e9,9416741,,2621,28.692505,-99.809959,TX,WSC,com,"BTM, FOM",...,4850795030120,0,None,None,0,zavala,23060.0,1499.0,539fb91bec4f024bc1dc1f11,"{'en_electricity_rates': 1, 'ur_metering_optio..."


## PV Prices

In [41]:
# Input Data Processing: PV Prices

# Input Folder: pv_prices
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'PV Price Scenario'
# Data Source: ATB_Data

#--------------------#

# Import the ATB data
atb_file_path = os.path.join(atb_data_folder, atb_data_file)
atb_data = pd.ExcelFile(atb_file_path)
print(f'Successfully read ATB data from: {atb_data_file}')

# Assign the individual tabs to separate data frames
pv_dist_com_full = atb_data.parse('Solar - PV Dist. Comm')
pv_dist_res_full = atb_data.parse('Solar - PV Dist. Res')

# Extract the relevant data
pv_dist_com_subset = pv_dist_com_full.iloc[157:284, 10:41] # Commercial
pv_dist_com_subset.columns = pv_dist_com_subset.iloc[0]
pv_dist_com_subset = pv_dist_com_subset[1:]
pv_dist_com_subset.columns = pv_dist_com_subset.columns.astype(float).fillna(0).astype(int).astype(str)
pv_dist_com_subset.columns.values[0] = 0
pv_dist_com_subset.columns.values[1] = 1

pv_dist_res_subset = pv_dist_res_full.iloc[157:284, 10:41] # Residential
pv_dist_res_subset.columns = pv_dist_res_subset.iloc[0]
pv_dist_res_subset = pv_dist_res_subset[1:]
pv_dist_res_subset.columns = pv_dist_res_subset.columns.astype(float).fillna(0).astype(int).astype(str)
pv_dist_res_subset.columns.values[0] = 0
pv_dist_res_subset.columns.values[1] = 1

#display(pv_dist_com_subset)
#display(pv_dist_res_subset)

##### Capital Costs
pv_dist_com_capex = pv_dist_com_subset.iloc[0:3, :] # Commercial
pv_dist_res_capex = pv_dist_res_subset.iloc[0:3, :] # Residential
#display(pv_dist_com_capex)
#display(pv_dist_res_capex)

##### Fixed Operations and Maintenance (O&M) Costs
pv_dist_com_fom = pv_dist_com_subset.iloc[64:67, :] # Commercial
pv_dist_res_fom = pv_dist_res_subset.iloc[64:67, :] # Residential
#display(pv_dist_com_fom)
#display(pv_dist_res_fom)

##### Variable Operations and Maintenance (O&M) Costs
pv_dist_com_vom = pv_dist_com_subset.iloc[96:99, :] # Commercial
pv_dist_res_vom = pv_dist_res_subset.iloc[96:99, :] # Residential
#display(pv_dist_com_vom)
#display(pv_dist_res_vom)

# Initialize empty data frames for the csv output files (conservative, moderate, advanced)
pv_prices_column_names = [
    'year',
    'system_capex_per_kw_res',
    'system_capex_per_kw_com',
    'system_capex_per_kw_ind',
    'system_om_per_kw_res',
    'system_om_per_kw_com',
    'system_om_per_kw_ind',
    'system_variable_om_per_kw_res',
    'system_variable_om_per_kw_com',
    'system_variable_om_per_kw_ind'
]
pv_price_atb24_conservative = pd.DataFrame(columns=pv_prices_column_names)
pv_price_atb24_moderate = pd.DataFrame(columns=pv_prices_column_names)
pv_price_atb24_advanced = pd.DataFrame(columns=pv_prices_column_names)

# Populate the data frames
years = list(range(start_year, end_year + 1))
pv_price_atb24_conservative['year'] = years
pv_price_atb24_moderate['year'] = years
pv_price_atb24_advanced['year'] = years

##### Capital Costs
pv_dist_com_capex_conservative = pv_dist_com_capex[pv_dist_com_capex.iloc[:, 1] == 'Conservative'] # conservative - Commercial and Industrial
pv_dist_com_capex_conservative.columns = pv_dist_com_capex_conservative.columns.astype(int)
pv_dist_com_capex_conservative_values = pv_dist_com_capex_conservative.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_conservative['system_capex_per_kw_com'] = pv_price_atb24_conservative['year'].apply(
    lambda y: pv_dist_com_capex_conservative_values[2022] if y < 2022 else pv_dist_com_capex_conservative_values.get(y, None)
)
pv_price_atb24_conservative['system_capex_per_kw_ind'] = pv_price_atb24_conservative['year'].apply(
    lambda y: pv_dist_com_capex_conservative_values[2022] if y < 2022 else pv_dist_com_capex_conservative_values.get(y, None)
)

pv_dist_res_capex_conservative = pv_dist_res_capex[pv_dist_res_capex.iloc[:, 1] == 'Conservative'] # conservative - Residential
pv_dist_res_capex_conservative.columns = pv_dist_res_capex_conservative.columns.astype(int)
pv_dist_res_capex_conservative_values = pv_dist_res_capex_conservative.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_conservative['system_capex_per_kw_res'] = pv_price_atb24_conservative['year'].apply(
    lambda y: pv_dist_res_capex_conservative_values[2022] if y < 2022 else pv_dist_res_capex_conservative_values.get(y, None)
)

pv_dist_com_capex_moderate = pv_dist_com_capex[pv_dist_com_capex.iloc[:, 1] == 'Moderate'] # moderate - Commercial and Industrial
pv_dist_com_capex_moderate.columns = pv_dist_com_capex_moderate.columns.astype(int)
pv_dist_com_capex_moderate_values = pv_dist_com_capex_moderate.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_moderate['system_capex_per_kw_com'] = pv_price_atb24_moderate['year'].apply(
    lambda y: pv_dist_com_capex_moderate_values[2022] if y < 2022 else pv_dist_com_capex_moderate_values.get(y, None)
)
pv_price_atb24_moderate['system_capex_per_kw_ind'] = pv_price_atb24_moderate['year'].apply(
    lambda y: pv_dist_com_capex_moderate_values[2022] if y < 2022 else pv_dist_com_capex_moderate_values.get(y, None)
)

pv_dist_res_capex_moderate = pv_dist_res_capex[pv_dist_res_capex.iloc[:, 1] == 'Moderate'] # moderate - Residential
pv_dist_res_capex_moderate.columns = pv_dist_res_capex_moderate.columns.astype(int)
pv_dist_res_capex_moderate_values = pv_dist_res_capex_moderate.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_moderate['system_capex_per_kw_res'] = pv_price_atb24_moderate['year'].apply(
    lambda y: pv_dist_res_capex_moderate_values[2022] if y < 2022 else pv_dist_res_capex_moderate_values.get(y, None)
)

pv_dist_com_capex_advanced = pv_dist_com_capex[pv_dist_com_capex.iloc[:, 1] == 'Advanced'] # advanced - Commercial and Industrial
pv_dist_com_capex_advanced.columns = pv_dist_com_capex_advanced.columns.astype(int)
pv_dist_com_capex_advanced_values = pv_dist_com_capex_advanced.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_advanced['system_capex_per_kw_com'] = pv_price_atb24_advanced['year'].apply(
    lambda y: pv_dist_com_capex_advanced_values[2022] if y < 2022 else pv_dist_com_capex_advanced_values.get(y, None)
)
pv_price_atb24_advanced['system_capex_per_kw_ind'] = pv_price_atb24_advanced['year'].apply(
    lambda y: pv_dist_com_capex_advanced_values[2022] if y < 2022 else pv_dist_com_capex_advanced_values.get(y, None)
)

pv_dist_res_capex_advanced = pv_dist_res_capex[pv_dist_res_capex.iloc[:, 1] == 'Advanced'] # advanced - Residential
pv_dist_res_capex_advanced.columns = pv_dist_res_capex_advanced.columns.astype(int)
pv_dist_res_capex_advanced_values = pv_dist_res_capex_advanced.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_advanced['system_capex_per_kw_res'] = pv_price_atb24_advanced['year'].apply(
    lambda y: pv_dist_res_capex_advanced_values[2022] if y < 2022 else pv_dist_res_capex_advanced_values.get(y, None)
)

##### Fixed Operations and Maintenance (O&M) Costs
pv_dist_com_fom_conservative = pv_dist_com_fom[pv_dist_com_fom.iloc[:, 1] == 'Conservative'] # conservative - Commercial and Industrial
pv_dist_com_fom_conservative.columns = pv_dist_com_fom_conservative.columns.astype(int)
pv_dist_com_fom_conservative_values = pv_dist_com_fom_conservative.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_conservative['system_om_per_kw_com'] = pv_price_atb24_conservative['year'].apply(
    lambda y: pv_dist_com_fom_conservative_values[2022] if y < 2022 else pv_dist_com_fom_conservative_values.get(y, None)
)
pv_price_atb24_conservative['system_om_per_kw_ind'] = pv_price_atb24_conservative['year'].apply(
    lambda y: pv_dist_com_fom_conservative_values[2022] if y < 2022 else pv_dist_com_fom_conservative_values.get(y, None)
)

pv_dist_res_fom_conservative = pv_dist_res_fom[pv_dist_res_fom.iloc[:, 1] == 'Conservative'] # conservative - Residential
pv_dist_res_fom_conservative.columns = pv_dist_res_fom_conservative.columns.astype(int)
pv_dist_res_fom_conservative_values = pv_dist_res_fom_conservative.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_conservative['system_om_per_kw_res'] = pv_price_atb24_conservative['year'].apply(
    lambda y: pv_dist_res_fom_conservative_values[2022] if y < 2022 else pv_dist_res_fom_conservative_values.get(y, None)
)

pv_dist_com_fom_moderate = pv_dist_com_fom[pv_dist_com_fom.iloc[:, 1] == 'Moderate'] # moderate - Commercial and Industrial
pv_dist_com_fom_moderate.columns = pv_dist_com_fom_moderate.columns.astype(int)
pv_dist_com_fom_moderate_values = pv_dist_com_fom_moderate.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_moderate['system_om_per_kw_com'] = pv_price_atb24_moderate['year'].apply(
    lambda y: pv_dist_com_fom_moderate_values[2022] if y < 2022 else pv_dist_com_fom_moderate_values.get(y, None)
)
pv_price_atb24_moderate['system_om_per_kw_ind'] = pv_price_atb24_moderate['year'].apply(
    lambda y: pv_dist_com_fom_moderate_values[2022] if y < 2022 else pv_dist_com_fom_moderate_values.get(y, None)
)

pv_dist_res_fom_moderate = pv_dist_res_fom[pv_dist_res_fom.iloc[:, 1] == 'Moderate'] # moderate - Residential
pv_dist_res_fom_moderate.columns = pv_dist_res_fom_moderate.columns.astype(int)
pv_dist_res_fom_moderate_values = pv_dist_res_fom_moderate.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_moderate['system_om_per_kw_res'] = pv_price_atb24_moderate['year'].apply(
    lambda y: pv_dist_res_fom_moderate_values[2022] if y < 2022 else pv_dist_res_fom_moderate_values.get(y, None)
)

pv_dist_com_fom_advanced = pv_dist_com_fom[pv_dist_com_fom.iloc[:, 1] == 'Advanced'] # advanced - Commercial and Industrial
pv_dist_com_fom_advanced.columns = pv_dist_com_fom_advanced.columns.astype(int)
pv_dist_com_fom_advanced_values = pv_dist_com_fom_advanced.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_advanced['system_om_per_kw_com'] = pv_price_atb24_advanced['year'].apply(
    lambda y: pv_dist_com_fom_advanced_values[2022] if y < 2022 else pv_dist_com_fom_advanced_values.get(y, None)
)
pv_price_atb24_advanced['system_om_per_kw_ind'] = pv_price_atb24_advanced['year'].apply(
    lambda y: pv_dist_com_fom_advanced_values[2022] if y < 2022 else pv_dist_com_fom_advanced_values.get(y, None)
)

pv_dist_res_fom_advanced = pv_dist_res_fom[pv_dist_res_fom.iloc[:, 1] == 'Advanced'] # advanced - Residential
pv_dist_res_fom_advanced.columns = pv_dist_res_fom_advanced.columns.astype(int)
pv_dist_res_fom_advanced_values = pv_dist_res_fom_advanced.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_advanced['system_om_per_kw_res'] = pv_price_atb24_advanced['year'].apply(
    lambda y: pv_dist_res_fom_advanced_values[2022] if y < 2022 else pv_dist_res_fom_advanced_values.get(y, None)
)

##### Variable Operations and Maintenance (O&M) Costs
pv_dist_com_vom_conservative = pv_dist_com_vom[pv_dist_com_vom.iloc[:, 1] == 'Conservative'] # conservative - Commercial and Industrial
pv_dist_com_vom_conservative.columns = pv_dist_com_vom_conservative.columns.astype(int)
pv_dist_com_vom_conservative_values = pv_dist_com_vom_conservative.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_conservative['system_variable_om_per_kw_com'] = pv_price_atb24_conservative['year'].apply(
    lambda y: pv_dist_com_vom_conservative_values[2022] if y < 2022 else pv_dist_com_vom_conservative_values.get(y, None)
)
pv_price_atb24_conservative['system_variable_om_per_kw_ind'] = pv_price_atb24_conservative['year'].apply(
    lambda y: pv_dist_com_vom_conservative_values[2022] if y < 2022 else pv_dist_com_vom_conservative_values.get(y, None)
)

pv_dist_res_vom_conservative = pv_dist_res_vom[pv_dist_res_vom.iloc[:, 1] == 'Conservative'] # conservative - Residential
pv_dist_res_vom_conservative.columns = pv_dist_res_vom_conservative.columns.astype(int)
pv_dist_res_vom_conservative_values = pv_dist_res_vom_conservative.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_conservative['system_variable_om_per_kw_res'] = pv_price_atb24_conservative['year'].apply(
    lambda y: pv_dist_res_vom_conservative_values[2022] if y < 2022 else pv_dist_res_vom_conservative_values.get(y, None)
)

pv_dist_com_vom_moderate = pv_dist_com_vom[pv_dist_com_vom.iloc[:, 1] == 'Moderate'] # moderate - Commercial and Industrial
pv_dist_com_vom_moderate.columns = pv_dist_com_vom_moderate.columns.astype(int)
pv_dist_com_vom_moderate_values = pv_dist_com_vom_moderate.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_moderate['system_variable_om_per_kw_com'] = pv_price_atb24_moderate['year'].apply(
    lambda y: pv_dist_com_vom_moderate_values[2022] if y < 2022 else pv_dist_com_vom_moderate_values.get(y, None)
)
pv_price_atb24_moderate['system_variable_om_per_kw_ind'] = pv_price_atb24_moderate['year'].apply(
    lambda y: pv_dist_com_vom_moderate_values[2022] if y < 2022 else pv_dist_com_vom_moderate_values.get(y, None)
)

pv_dist_res_vom_moderate = pv_dist_res_vom[pv_dist_res_vom.iloc[:, 1] == 'Moderate'] # moderate - Residential
pv_dist_res_vom_moderate.columns = pv_dist_res_vom_moderate.columns.astype(int)
pv_dist_res_vom_moderate_values = pv_dist_res_vom_moderate.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_moderate['system_variable_om_per_kw_res'] = pv_price_atb24_moderate['year'].apply(
    lambda y: pv_dist_res_vom_moderate_values[2022] if y < 2022 else pv_dist_res_vom_moderate_values.get(y, None)
)

pv_dist_com_vom_advanced = pv_dist_com_vom[pv_dist_com_vom.iloc[:, 1] == 'Advanced'] # advanced - Commercial and Industrial
pv_dist_com_vom_advanced.columns = pv_dist_com_vom_advanced.columns.astype(int)
pv_dist_com_vom_advanced_values = pv_dist_com_vom_advanced.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_advanced['system_variable_om_per_kw_com'] = pv_price_atb24_advanced['year'].apply(
    lambda y: pv_dist_com_vom_advanced_values[2022] if y < 2022 else pv_dist_com_vom_advanced_values.get(y, None)
)
pv_price_atb24_advanced['system_variable_om_per_kw_ind'] = pv_price_atb24_advanced['year'].apply(
    lambda y: pv_dist_com_vom_advanced_values[2022] if y < 2022 else pv_dist_com_vom_advanced_values.get(y, None)
)

pv_dist_res_vom_advanced = pv_dist_res_vom[pv_dist_res_vom.iloc[:, 1] == 'Advanced'] # advanced - Residential
pv_dist_res_vom_advanced.columns = pv_dist_res_vom_advanced.columns.astype(int)
pv_dist_res_vom_advanced_values = pv_dist_res_vom_advanced.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_advanced['system_variable_om_per_kw_res'] = pv_price_atb24_advanced['year'].apply(
    lambda y: pv_dist_res_vom_advanced_values[2022] if y < 2022 else pv_dist_res_vom_advanced_values.get(y, None)
)

#display(pv_price_atb24_conservative)
#display(pv_price_atb24_moderate)
#display(pv_price_atb24_advanced)

# Save the csv files
output_path = os.path.join(os.getcwd(), '..', 'pv_prices', 'pv_price_atb24_conservative.csv')
pv_price_atb24_conservative.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'pv_prices', 'pv_price_atb24_moderate.csv')
pv_price_atb24_moderate.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'pv_prices', 'pv_price_atb24_advanced.csv')
pv_price_atb24_advanced.to_csv(output_path, index=False)

Successfully read ATB data from: 2024_v3_Workbook.xlsx


## PV Performance

In [3]:
# Input Data Processing: PV Performance

# Input Folder: pv_tech_performance
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'PV Technical Performance Scenario'
# Data Source: ATB_Data

#--------------------#

# PV Panel Density [column names: pv_kw_per_sqft_res, pv_kw_per_sqft_com, pv_kw_per_sqft_ind]
# - This information is not in the ATB so the values from FY19 are kept the same

# PV Panel Degradation [column names: pv_degradation_factor_res, pv_degradation_factor_com, pv_degradation_factor_ind]
# - This information is not in the ATB spreadsheet, but it is on the ATB website in the capacity factor section of residential PV
# - https://atb.nrel.gov/electricity/2024/residential_pv
# - This information was manually updated for the three technology scenarios (conservative, moderate, advanced)
# - Conservative: 0.7% degradation from 2014-2050
# - Moderate: 0.7% degradation from 2014-2019, linear decrease from 0.7% to 0.5% degradation from 2020-2034, 0.5% degradation from 2035-2050
# - Moderate: 0.7% degradation from 2014-2019, linear decrease from 0.7% to 0.2% degradation from 2020-2034, 0.2% degradation from 2035-2050

# Output files
# - pv_tech_performance_atb24_conservative.csv
# - pv_tech_performance_atb24_moderate.csv
# - pv_tech_performance_atb24_advanced.csv

## Battery Prices

In [7]:
# Input Data Processing: Battery Prices

# Input Folder: batt_prices
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'Storage Cost Scenario'
# Data Source: ATB_Data

#--------------------#

# Import the ATB data
atb_file_path = os.path.join(atb_data_folder, atb_data_file)
atb_data = pd.ExcelFile(atb_file_path)
print(f'Successfully read ATB data from: {atb_data_file}')

# Assign the individual tabs to separate data frames
batt_com_full = atb_data.parse('Commercial Battery Storage')
batt_res_full = atb_data.parse('Residential Battery Storage')

# Extract the relevant data
batt_com_subset = batt_com_full.iloc[16:168, 4:66] # Commercial
batt_com_subset.columns = batt_com_subset.iloc[0]
batt_com_subset = batt_com_subset[1:]
batt_com_subset.columns.values[0] = 0
batt_com_subset.columns = batt_com_subset.columns.astype(float).fillna(0).astype(int).astype(str)

batt_res_subset = batt_res_full.iloc[17:113, 4:66] # Residential
batt_res_subset.columns = batt_res_subset.iloc[0]
batt_res_subset = batt_res_subset[1:]
batt_res_subset.columns.values[0] = 0
batt_res_subset.columns = batt_res_subset.columns.astype(float).fillna(0).astype(int).astype(str)

#display(batt_com_subset)
#display(batt_res_subset)

##### Capital Costs
batt_com_capex_kwh = batt_com_subset.iloc[0:3, 0:30] # Commercial - kWh (Assumption: 1800 kW - 72000 kWh battery)
batt_com_capex_kw = batt_com_subset.iloc[6:9, 0:30] # Commercial - kW (Assumption: 1800 kW - 72000 kWh battery)
batt_res_capex_kwh = batt_res_subset.iloc[0:3, 0:30] # Residential - kWh (Assumption: 5 kW - 12.5 kWh battery)
batt_res_capex_kw = batt_res_subset.iloc[6:9, 0:30] # Residential - kW (Assumption: 5 kW - 12.5 kWh battery)

#display(batt_com_capex_kwh)
#display(batt_com_capex_kw)
#display(batt_res_capex_kwh)
#display(batt_res_capex_kw)

##### Operations and Maintenance (O&M) Costs
batt_com_om_kwh = batt_com_subset.iloc[135:145, 0:31] # Commercial - kWh (Assumption: 4-hour battery) | (Assumption: 1800 kW - 72000 kWh battery)
batt_com_om_kwh.columns = batt_com_om_kwh.iloc[0]
batt_com_om_kwh = batt_com_om_kwh[1:]
batt_com_om_kwh.columns.values[0] = 0
batt_com_om_kwh.columns.values[1] = 1
batt_com_om_kwh.columns = batt_com_om_kwh.columns.astype(float).fillna(0).astype(int).astype(str)
batt_com_om_kwh = batt_com_om_kwh.iloc[6:9, :]

batt_com_om_kw = batt_com_subset.iloc[118:128, 0:31] # Commercial - kW (Assumption: 4-hour battery) | (Assumption: 1800 kW - 72000 kWh battery)
batt_com_om_kw.columns = batt_com_om_kw.iloc[0]
batt_com_om_kw = batt_com_om_kw[1:]
batt_com_om_kw.columns.values[0] = 0
batt_com_om_kw.columns.values[1] = 1
batt_com_om_kw.columns = batt_com_om_kwh.columns.astype(float).fillna(0).astype(int).astype(str)
batt_com_om_kw = batt_com_om_kw.iloc[6:9, :]

batt_res_om_kwh = batt_res_subset.iloc[88:92, 0:31] # Residential - kWh (Assumption: 5 kW - 12.5 kWh battery)
batt_res_om_kwh.columns = batt_res_om_kwh.iloc[0]
batt_res_om_kwh = batt_res_om_kwh[1:]
batt_res_om_kwh.columns.values[0] = 0
batt_res_om_kwh.columns.values[1] = 1
batt_res_om_kwh.columns = batt_res_om_kwh.columns.astype(float).fillna(0).astype(int).astype(str)

batt_res_om_kw = batt_res_subset.iloc[80:84, 0:31] # Residential - kW (Assumption: 5 kW - 12.5 kWh battery)
batt_res_om_kw.columns = batt_res_om_kw.iloc[0]
batt_res_om_kw = batt_res_om_kw[1:]
batt_res_om_kw.columns.values[0] = 0
batt_res_om_kw.columns.values[1] = 1
batt_res_om_kw.columns = batt_res_om_kw.columns.astype(float).fillna(0).astype(int).astype(str)

#display(batt_com_om_kwh)
#display(batt_com_om_kw)
#display(batt_res_om_kwh)
#display(batt_res_om_kw)

##### Linear Constant
batt_com_constant = batt_com_subset.iloc[0:3, 31:62] # Commercial
batt_res_constant = batt_res_subset.iloc[0:3, 32:62] # Residential
#display(batt_com_constant)
#display(batt_res_constant)

# Initialize empty data frames for the csv output files (conservative, moderate, advanced)
batt_prices_column_names = [
    'year',
    'batt_capex_per_kwh_res',
    'batt_capex_per_kw_res',
    'batt_capex_per_kwh_nonres',
    'batt_capex_per_kw_nonres',
    'linear_constant_res',
    'linear_constant_nonres',
    'batt_om_per_kw_res',
    'batt_om_per_kwh_res',
    'batt_om_per_kw_nonres',
    'batt_om_per_kwh_nonres',
    'batt_replace_frac_kw',
    'batt_replace_frac_kwh'
]
batt_price_atb24_conservative = pd.DataFrame(columns=batt_prices_column_names)
batt_price_atb24_moderate = pd.DataFrame(columns=batt_prices_column_names)
batt_price_atb24_advanced = pd.DataFrame(columns=batt_prices_column_names)

# Populate the data frames
years = list(range(start_year, end_year + 1))
batt_price_atb24_conservative['year'] = years
batt_price_atb24_moderate['year'] = years
batt_price_atb24_advanced['year'] = years
batt_price_atb24_conservative['batt_replace_frac_kw'] = 0.174
batt_price_atb24_moderate['batt_replace_frac_kw'] = 0.174
batt_price_atb24_advanced['batt_replace_frac_kw'] = 0.174
batt_price_atb24_conservative['batt_replace_frac_kwh'] = 1
batt_price_atb24_moderate['batt_replace_frac_kwh'] = 1
batt_price_atb24_advanced['batt_replace_frac_kwh'] = 1

##### Capital Costs
batt_com_capex_kwh_conservative = batt_com_capex_kwh[batt_com_capex_kwh.iloc[:, 0] == 'Conservative'] # conservative - Nonresidential
batt_com_capex_kwh_conservative.columns = batt_com_capex_kwh_conservative.columns.astype(int)
batt_com_capex_kwh_conservative_values = batt_com_capex_kwh_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['batt_capex_per_kwh_nonres'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_com_capex_kwh_conservative_values[2022] if y < 2022 else batt_com_capex_kwh_conservative_values.get(y, None)
)
batt_com_capex_kw_conservative = batt_com_capex_kw[batt_com_capex_kw.iloc[:, 0] == 'Conservative'] # conservative - Nonresidential
batt_com_capex_kw_conservative.columns = batt_com_capex_kw_conservative.columns.astype(int)
batt_com_capex_kw_conservative_values = batt_com_capex_kw_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['batt_capex_per_kw_nonres'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_com_capex_kw_conservative_values[2022] if y < 2022 else batt_com_capex_kw_conservative_values.get(y, None)
)
batt_res_capex_kwh_conservative = batt_res_capex_kwh[batt_res_capex_kwh.iloc[:, 0] == 'Conservative'] # conservative - Residential
batt_res_capex_kwh_conservative.columns = batt_res_capex_kwh_conservative.columns.astype(int)
batt_res_capex_kwh_conservative_values = batt_res_capex_kwh_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['batt_capex_per_kwh_res'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_res_capex_kwh_conservative_values[2022] if y < 2022 else batt_res_capex_kwh_conservative_values.get(y, None)
)
batt_res_capex_kw_conservative = batt_res_capex_kw[batt_res_capex_kw.iloc[:, 0] == 'Conservative'] # conservative - Residential
batt_res_capex_kw_conservative.columns = batt_res_capex_kw_conservative.columns.astype(int)
batt_res_capex_kw_conservative_values = batt_res_capex_kw_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['batt_capex_per_kw_res'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_res_capex_kw_conservative_values[2022] if y < 2022 else batt_res_capex_kw_conservative_values.get(y, None)
)

batt_com_capex_kwh_moderate = batt_com_capex_kwh[batt_com_capex_kwh.iloc[:, 0] == 'Moderate'] # moderate - Nonresidential
batt_com_capex_kwh_moderate.columns = batt_com_capex_kwh_moderate.columns.astype(int)
batt_com_capex_kwh_moderate_values = batt_com_capex_kwh_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['batt_capex_per_kwh_nonres'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_com_capex_kwh_moderate_values[2022] if y < 2022 else batt_com_capex_kwh_moderate_values.get(y, None)
)
batt_com_capex_kw_moderate = batt_com_capex_kw[batt_com_capex_kw.iloc[:, 0] == 'Moderate'] # moderate - Nonresidential
batt_com_capex_kw_moderate.columns = batt_com_capex_kw_moderate.columns.astype(int)
batt_com_capex_kw_moderate_values = batt_com_capex_kw_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['batt_capex_per_kw_nonres'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_com_capex_kw_moderate_values[2022] if y < 2022 else batt_com_capex_kw_moderate_values.get(y, None)
)
batt_res_capex_kwh_moderate = batt_res_capex_kwh[batt_res_capex_kwh.iloc[:, 0] == 'Moderate'] # moderate - Residential
batt_res_capex_kwh_moderate.columns = batt_res_capex_kwh_moderate.columns.astype(int)
batt_res_capex_kwh_moderate_values = batt_res_capex_kwh_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['batt_capex_per_kwh_res'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_res_capex_kwh_moderate_values[2022] if y < 2022 else batt_res_capex_kwh_moderate_values.get(y, None)
)
batt_res_capex_kw_moderate = batt_res_capex_kw[batt_res_capex_kw.iloc[:, 0] == 'Moderate'] # moderate - Residential
batt_res_capex_kw_moderate.columns = batt_res_capex_kw_moderate.columns.astype(int)
batt_res_capex_kw_moderate_values = batt_res_capex_kw_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['batt_capex_per_kw_res'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_res_capex_kw_moderate_values[2022] if y < 2022 else batt_res_capex_kw_moderate_values.get(y, None)
)

batt_com_capex_kwh_advanced = batt_com_capex_kwh[batt_com_capex_kwh.iloc[:, 0] == 'Advanced'] # advanced - Nonresidential
batt_com_capex_kwh_advanced.columns = batt_com_capex_kwh_advanced.columns.astype(int)
batt_com_capex_kwh_advanced_values = batt_com_capex_kwh_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['batt_capex_per_kwh_nonres'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_com_capex_kwh_advanced_values[2022] if y < 2022 else batt_com_capex_kwh_advanced_values.get(y, None)
)
batt_com_capex_kw_advanced = batt_com_capex_kw[batt_com_capex_kw.iloc[:, 0] == 'Advanced'] # advanced - Nonresidential
batt_com_capex_kw_advanced.columns = batt_com_capex_kw_advanced.columns.astype(int)
batt_com_capex_kw_advanced_values = batt_com_capex_kw_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['batt_capex_per_kw_nonres'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_com_capex_kw_advanced_values[2022] if y < 2022 else batt_com_capex_kw_advanced_values.get(y, None)
)
batt_res_capex_kwh_advanced = batt_res_capex_kwh[batt_res_capex_kwh.iloc[:, 0] == 'Advanced'] # advanced - Residential
batt_res_capex_kwh_advanced.columns = batt_res_capex_kwh_advanced.columns.astype(int)
batt_res_capex_kwh_advanced_values = batt_res_capex_kwh_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['batt_capex_per_kwh_res'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_res_capex_kwh_advanced_values[2022] if y < 2022 else batt_res_capex_kwh_advanced_values.get(y, None)
)
batt_res_capex_kw_advanced = batt_res_capex_kw[batt_res_capex_kw.iloc[:, 0] == 'Advanced'] # advanced - Residential
batt_res_capex_kw_advanced.columns = batt_res_capex_kw_advanced.columns.astype(int)
batt_res_capex_kw_advanced_values = batt_res_capex_kw_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['batt_capex_per_kw_res'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_res_capex_kw_advanced_values[2022] if y < 2022 else batt_res_capex_kw_advanced_values.get(y, None)
)

##### Operations and Maintenance (O&M) Costs
batt_com_om_kwh_conservative = batt_com_om_kwh[batt_com_om_kwh.iloc[:, 1] == 'Conservative'] # conservative - Nonresidential
batt_com_om_kwh_conservative.columns = batt_com_om_kwh_conservative.columns.astype(int)
batt_com_om_kwh_conservative_values = batt_com_om_kwh_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['batt_om_per_kwh_nonres'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_com_om_kwh_conservative_values[2022] if y < 2022 else batt_com_om_kwh_conservative_values.get(y, None)
)
batt_com_om_kw_conservative = batt_com_om_kw[batt_com_om_kw.iloc[:, 1] == 'Conservative'] # conservative - Nonresidential
batt_com_om_kw_conservative.columns = batt_com_om_kw_conservative.columns.astype(int)
batt_com_om_kw_conservative_values = batt_com_om_kw_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['batt_om_per_kw_nonres'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_com_om_kw_conservative_values[2022] if y < 2022 else batt_com_om_kw_conservative_values.get(y, None)
)
batt_res_om_kwh_conservative = batt_res_om_kwh[batt_res_om_kwh.iloc[:, 1] == 'Conservative'] # conservative - Residential
batt_res_om_kwh_conservative.columns = batt_res_om_kwh_conservative.columns.astype(int)
batt_res_om_kwh_conservative_values = batt_res_om_kwh_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['batt_om_per_kwh_res'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_res_om_kwh_conservative_values[2022] if y < 2022 else batt_res_om_kwh_conservative_values.get(y, None)
)
batt_res_om_kw_conservative = batt_res_om_kw[batt_res_om_kw.iloc[:, 1] == 'Conservative'] # conservative - Residential
batt_res_om_kw_conservative.columns = batt_res_om_kw_conservative.columns.astype(int)
batt_res_om_kw_conservative_values = batt_res_om_kw_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['batt_om_per_kw_res'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_res_om_kw_conservative_values[2022] if y < 2022 else batt_res_om_kw_conservative_values.get(y, None)
)

batt_com_om_kwh_moderate = batt_com_om_kwh[batt_com_om_kwh.iloc[:, 1] == 'Moderate'] # moderate - Nonresidential
batt_com_om_kwh_moderate.columns = batt_com_om_kwh_moderate.columns.astype(int)
batt_com_om_kwh_moderate_values = batt_com_om_kwh_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['batt_om_per_kwh_nonres'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_com_om_kwh_moderate_values[2022] if y < 2022 else batt_com_om_kwh_moderate_values.get(y, None)
)
batt_com_om_kw_moderate = batt_com_om_kw[batt_com_om_kw.iloc[:, 1] == 'Moderate'] # moderate - Nonresidential
batt_com_om_kw_moderate.columns = batt_com_om_kw_moderate.columns.astype(int)
batt_com_om_kw_moderate_values = batt_com_om_kw_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['batt_om_per_kw_nonres'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_com_om_kw_moderate_values[2022] if y < 2022 else batt_com_om_kw_moderate_values.get(y, None)
)
batt_res_om_kwh_moderate = batt_res_om_kwh[batt_res_om_kwh.iloc[:, 1] == 'Moderate'] # moderate - Residential
batt_res_om_kwh_moderate.columns = batt_res_om_kwh_moderate.columns.astype(int)
batt_res_om_kwh_moderate_values = batt_res_om_kwh_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['batt_om_per_kwh_res'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_res_om_kwh_moderate_values[2022] if y < 2022 else batt_res_om_kwh_moderate_values.get(y, None)
)
batt_res_om_kw_moderate = batt_res_om_kw[batt_res_om_kw.iloc[:, 1] == 'Moderate'] # moderate - Residential
batt_res_om_kw_moderate.columns = batt_res_om_kw_moderate.columns.astype(int)
batt_res_om_kw_moderate_values = batt_res_om_kw_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['batt_om_per_kw_res'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_res_om_kw_moderate_values[2022] if y < 2022 else batt_res_om_kw_moderate_values.get(y, None)
)

batt_com_om_kwh_advanced = batt_com_om_kwh[batt_com_om_kwh.iloc[:, 1] == 'Advanced'] # advanced - Nonresidential
batt_com_om_kwh_advanced.columns = batt_com_om_kwh_advanced.columns.astype(int)
batt_com_om_kwh_advanced_values = batt_com_om_kwh_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['batt_om_per_kwh_nonres'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_com_om_kwh_advanced_values[2022] if y < 2022 else batt_com_om_kwh_advanced_values.get(y, None)
)
batt_com_om_kw_advanced = batt_com_om_kw[batt_com_om_kw.iloc[:, 1] == 'Advanced'] # advanced - Nonresidential
batt_com_om_kw_advanced.columns = batt_com_om_kw_advanced.columns.astype(int)
batt_com_om_kw_advanced_values = batt_com_om_kw_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['batt_om_per_kw_nonres'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_com_om_kw_advanced_values[2022] if y < 2022 else batt_com_om_kw_advanced_values.get(y, None)
)
batt_res_om_kwh_advanced = batt_res_om_kwh[batt_res_om_kwh.iloc[:, 1] == 'Advanced'] # advanced - Residential
batt_res_om_kwh_advanced.columns = batt_res_om_kwh_advanced.columns.astype(int)
batt_res_om_kwh_advanced_values = batt_res_om_kwh_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['batt_om_per_kwh_res'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_res_om_kwh_advanced_values[2022] if y < 2022 else batt_res_om_kwh_advanced_values.get(y, None)
)
batt_res_om_kw_advanced = batt_res_om_kw[batt_res_om_kw.iloc[:, 1] == 'Advanced'] # advanced - Residential
batt_res_om_kw_advanced.columns = batt_res_om_kw_advanced.columns.astype(int)
batt_res_om_kw_advanced_values = batt_res_om_kw_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['batt_om_per_kw_res'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_res_om_kw_advanced_values[2022] if y < 2022 else batt_res_om_kw_advanced_values.get(y, None)
)

##### Linear Constant
batt_com_constant_conservative = batt_com_constant[batt_com_constant.iloc[:, 0] == 'Conservative'] # conservative - Nonresidential
batt_com_constant_conservative.columns = batt_com_constant_conservative.columns.astype(int)
batt_com_constant_conservative_values = batt_com_constant_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['linear_constant_nonres'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_com_constant_conservative_values[2022] if y < 2022 else batt_com_constant_conservative_values.get(y, None)
)
batt_res_constant_conservative = batt_res_constant[batt_res_constant.iloc[:, 0] == 'Conservative'] # conservative - Residential
batt_res_constant_conservative.columns = batt_res_constant_conservative.columns.astype(int)
batt_res_constant_conservative_values = batt_res_constant_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['linear_constant_res'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_res_constant_conservative_values[2022] if y < 2022 else batt_res_constant_conservative_values.get(y, None)
)

batt_com_constant_moderate = batt_com_constant[batt_com_constant.iloc[:, 0] == 'Moderate'] # moderate - Nonresidential
batt_com_constant_moderate.columns = batt_com_constant_moderate.columns.astype(int)
batt_com_constant_moderate_values = batt_com_constant_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['linear_constant_nonres'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_com_constant_moderate_values[2022] if y < 2022 else batt_com_constant_moderate_values.get(y, None)
)
batt_res_constant_moderate = batt_res_constant[batt_res_constant.iloc[:, 0] == 'Moderate'] # moderate - Residential
batt_res_constant_moderate.columns = batt_res_constant_moderate.columns.astype(int)
batt_res_constant_moderate_values = batt_res_constant_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['linear_constant_res'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_res_constant_moderate_values[2022] if y < 2022 else batt_res_constant_moderate_values.get(y, None)
)

batt_com_constant_advanced = batt_com_constant[batt_com_constant.iloc[:, 0] == 'Advanced'] # advanced - Nonresidential
batt_com_constant_advanced.columns = batt_com_constant_advanced.columns.astype(int)
batt_com_constant_advanced_values = batt_com_constant_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['linear_constant_nonres'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_com_constant_advanced_values[2022] if y < 2022 else batt_com_constant_advanced_values.get(y, None)
)
batt_res_constant_advanced = batt_res_constant[batt_res_constant.iloc[:, 0] == 'Advanced'] # advanced - Residential
batt_res_constant_advanced.columns = batt_res_constant_advanced.columns.astype(int)
batt_res_constant_advanced_values = batt_res_constant_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['linear_constant_res'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_res_constant_advanced_values[2022] if y < 2022 else batt_res_constant_advanced_values.get(y, None)
)

#display(batt_price_atb24_conservative)
#display(batt_price_atb24_moderate)
#display(batt_price_atb24_advanced)

# Save the csv files
output_path = os.path.join(os.getcwd(), '..', 'batt_prices', 'batt_price_atb24_conservative.csv')
batt_price_atb24_conservative.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'batt_prices', 'batt_price_atb24_moderate.csv')
batt_price_atb24_moderate.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'batt_prices', 'batt_price_atb24_advanced.csv')
batt_price_atb24_advanced.to_csv(output_path, index=False)

Successfully read ATB data from: 2024_v3_Workbook.xlsx


16,0,2022,2023,2024,2025,2026,2027,2028,2029,2030,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
17,Advanced,275.160902,338.731345,251.989357,183.97537,177.346721,170.718072,164.089424,157.460775,150.832126,...,119.631597,116.795186,113.958774,111.122362,108.285950,105.449539,102.613127,99.776715,96.940303,94.103892
18,Moderate,275.160902,338.731345,236.696556,225.414849,213.064276,201.79771,191.436626,182.074604,173.471592,...,149.619248,147.450853,145.282459,143.114064,140.945669,138.777274,136.608879,134.440484,132.272089,130.103694
19,Conservative,275.160902,338.731345,298.157058,293.92788,282.932017,271.936154,260.940291,249.944428,238.948564,...,231.442808,230.760467,230.078125,229.395784,228.713443,228.031101,227.348760,226.666418,225.984077,225.301735


16,0,2022,2023,2024,2025,2026,2027,2028,2029,2030,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
23,Advanced,997.149656,1157.490659,772.689749,666.704374,642.682956,618.661539,594.640122,570.618705,546.597288,...,433.530364,423.251553,412.972742,402.693930,392.415119,382.136308,371.857497,361.578685,351.299874,341.021063
24,Moderate,997.149656,1157.490659,904.609776,867.347401,841.727690,816.346943,792.532917,770.904676,749.631799,...,646.557427,637.187029,627.816632,618.446234,609.075837,599.705439,590.335042,580.964644,571.594247,562.223849
25,Conservative,997.149656,1157.490659,1080.48493,1065.158903,1025.311232,985.46356,945.615889,905.768218,865.920547,...,838.720600,836.247877,833.775155,831.302432,828.829710,826.356987,823.884265,821.411542,818.938820,816.466097


17,0,2022,2023,2024,2025,2026,2027,2028,2029,2030,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
18,Advanced,621.82448,447.339184,422.227884,400.744665,386.305800,371.866934,357.428069,342.989203,328.550338,...,260.587732,254.409313,248.230894,242.052476,235.874057,229.695638,223.517220,217.338801,211.160382,204.981963
19,Moderate,621.82448,569.533525,519.630724,471.587198,448.699338,428.485638,411.352509,395.195303,380.602457,...,328.269619,323.512088,318.754558,313.997027,309.239496,304.481966,299.724435,294.966904,290.209373,285.451843
20,Conservative,621.82448,644.855016,649.461123,640.248909,616.297151,592.345393,568.393636,544.441878,520.49012,...,504.140694,502.654383,501.168071,499.681760,498.195448,496.709137,495.222825,493.736514,492.250203,490.763891


17,0,2022,2023,2024,2025,2026,2027,2028,2029,2030,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
24,Advanced,672.907908,484.088491,456.914276,433.666192,418.041161,402.41613,386.791099,371.166068,355.541037,...,281.995243,275.309262,268.623281,261.937300,255.251318,248.565337,241.879356,235.193375,228.507393,221.821412
25,Moderate,672.907908,650.607034,631.779775,615.704463,602.908720,591.257172,580.63149,569.803438,559.894884,...,482.909338,475.910652,468.911966,461.913280,454.914593,447.915907,440.917221,433.918535,426.919849,419.921163
26,Conservative,672.907908,697.830423,702.814926,692.84592,666.926504,641.007088,615.087673,589.168257,563.248841,...,545.556296,543.947882,542.339469,540.731056,539.122643,537.514229,535.905816,534.297403,532.688990,531.080576


## Battery Performance

In [19]:
# Input Data Processing: Battery Performance

# Input Folder: batt_tech_performance
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'Storage Technical Performance Scenario'
# Data Source: ATB_Data

#--------------------#

# Import the ATB data
atb_file_path = os.path.join(atb_data_folder, atb_data_file)
atb_data = pd.ExcelFile(atb_file_path)
print(f'Successfully read ATB data from: {atb_data_file}')

# Assign the individual tabs to separate data frames
batt_com_full = atb_data.parse('Commercial Battery Storage')
batt_res_full = atb_data.parse('Residential Battery Storage')

# Extract the relevant data
##### Efficiency
batt_com_eff = batt_com_full.iloc[186:190, 4:35] # Commercial
batt_com_eff.columns = batt_com_eff.iloc[0]
batt_com_eff = batt_com_eff[1:]
batt_com_eff.columns.values[0] = 0
batt_com_eff.columns.values[1] = 1
batt_com_eff.columns = batt_com_eff.columns.astype(float).fillna(0).astype(int).astype(str)

batt_res_eff = batt_res_full.iloc[122:126, 4:35] # Residential
batt_res_eff.columns = batt_res_eff.iloc[0]
batt_res_eff = batt_res_eff[1:]
batt_res_eff.columns.values[0] = 0
batt_res_eff.columns.values[1] = 1
batt_res_eff.columns = batt_res_eff.columns.astype(float).fillna(0).astype(int).astype(str)

#display(batt_com_eff)
#display(batt_res_eff)

# Initialize empty data frames for the csv output files (conservative, moderate, advanced)
batt_tech_performance_column_names = [
    'year',
    'batt_eff_res',
    'batt_eff_com',
    'batt_eff_ind',
    'batt_lifetime_yrs_res',
    'batt_lifetime_yrs_com',
    'batt_lifetime_yrs_ind',
]
batt_tech_performance_atb24_conservative = pd.DataFrame(columns=batt_tech_performance_column_names)
batt_tech_performance_atb24_moderate = pd.DataFrame(columns=batt_tech_performance_column_names)
batt_tech_performance_atb24_advanced = pd.DataFrame(columns=batt_tech_performance_column_names)

# Populate the data frames
years = list(range(start_year, end_year + 1))
batt_tech_performance_atb24_conservative['year'] = years
batt_tech_performance_atb24_moderate['year'] = years
batt_tech_performance_atb24_advanced['year'] = years
batt_tech_performance_atb24_conservative['batt_lifetime_yrs_res'] = 10
batt_tech_performance_atb24_moderate['batt_lifetime_yrs_res'] = 10
batt_tech_performance_atb24_advanced['batt_lifetime_yrs_res'] = 10
batt_tech_performance_atb24_conservative['batt_lifetime_yrs_com'] = 10
batt_tech_performance_atb24_moderate['batt_lifetime_yrs_com'] = 10
batt_tech_performance_atb24_advanced['batt_lifetime_yrs_com'] = 10
batt_tech_performance_atb24_conservative['batt_lifetime_yrs_ind'] = 10
batt_tech_performance_atb24_moderate['batt_lifetime_yrs_ind'] = 10
batt_tech_performance_atb24_advanced['batt_lifetime_yrs_ind'] = 10

##### Efficiency
batt_com_eff_conservative = batt_com_eff[batt_com_eff.iloc[:, 1] == 'Conservative'] # conservative - Commercial and Industrial
batt_com_eff_conservative.columns = batt_com_eff_conservative.columns.astype(int)
batt_com_eff_conservative_values = batt_com_eff_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_tech_performance_atb24_conservative['batt_eff_com'] = batt_tech_performance_atb24_conservative['year'].apply(
    lambda y: batt_com_eff_conservative_values[2022] if y < 2022 else batt_com_eff_conservative_values.get(y, None)
)
batt_tech_performance_atb24_conservative['batt_eff_ind'] = batt_tech_performance_atb24_conservative['year'].apply(
    lambda y: batt_com_eff_conservative_values[2022] if y < 2022 else batt_com_eff_conservative_values.get(y, None)
)

batt_res_eff_conservative = batt_res_eff[batt_res_eff.iloc[:, 1] == 'Conservative'] # conservative - Residential
batt_res_eff_conservative.columns = batt_res_eff_conservative.columns.astype(int)
batt_res_eff_conservative_values = batt_res_eff_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_tech_performance_atb24_conservative['batt_eff_res'] = batt_tech_performance_atb24_conservative['year'].apply(
    lambda y: batt_res_eff_conservative_values[2022] if y < 2022 else batt_res_eff_conservative_values.get(y, None)
)

batt_com_eff_moderate = batt_com_eff[batt_com_eff.iloc[:, 1] == 'Moderate'] # moderate - Commercial and Industrial
batt_com_eff_moderate.columns = batt_com_eff_moderate.columns.astype(int)
batt_com_eff_moderate_values = batt_com_eff_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_tech_performance_atb24_moderate['batt_eff_com'] = batt_tech_performance_atb24_moderate['year'].apply(
    lambda y: batt_com_eff_moderate_values[2022] if y < 2022 else batt_com_eff_moderate_values.get(y, None)
)
batt_tech_performance_atb24_moderate['batt_eff_ind'] = batt_tech_performance_atb24_moderate['year'].apply(
    lambda y: batt_com_eff_moderate_values[2022] if y < 2022 else batt_com_eff_moderate_values.get(y, None)
)

batt_res_eff_moderate = batt_res_eff[batt_res_eff.iloc[:, 1] == 'Moderate'] # moderate - Residential
batt_res_eff_moderate.columns = batt_res_eff_moderate.columns.astype(int)
batt_res_eff_moderate_values = batt_res_eff_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_tech_performance_atb24_moderate['batt_eff_res'] = batt_tech_performance_atb24_moderate['year'].apply(
    lambda y: batt_res_eff_moderate_values[2022] if y < 2022 else batt_res_eff_moderate_values.get(y, None)
)

batt_com_eff_advanced = batt_com_eff[batt_com_eff.iloc[:, 1] == 'Advanced'] # advanced - Commercial and Industrial
batt_com_eff_advanced.columns = batt_com_eff_advanced.columns.astype(int)
batt_com_eff_advanced_values = batt_com_eff_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_tech_performance_atb24_advanced['batt_eff_com'] = batt_tech_performance_atb24_advanced['year'].apply(
    lambda y: batt_com_eff_advanced_values[2022] if y < 2022 else batt_com_eff_advanced_values.get(y, None)
)
batt_tech_performance_atb24_advanced['batt_eff_ind'] = batt_tech_performance_atb24_advanced['year'].apply(
    lambda y: batt_com_eff_advanced_values[2022] if y < 2022 else batt_com_eff_advanced_values.get(y, None)
)

batt_res_eff_advanced = batt_res_eff[batt_res_eff.iloc[:, 1] == 'Advanced'] # advanced - Residential
batt_res_eff_advanced.columns = batt_res_eff_advanced.columns.astype(int)
batt_res_eff_advanced_values = batt_res_eff_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_tech_performance_atb24_advanced['batt_eff_res'] = batt_tech_performance_atb24_advanced['year'].apply(
    lambda y: batt_res_eff_advanced_values[2022] if y < 2022 else batt_res_eff_advanced_values.get(y, None)
)

#display(batt_tech_performance_atb24_conservative)
#display(batt_tech_performance_atb24_moderate)
#display(batt_tech_performance_atb24_advanced)

# Save the csv files
output_path = os.path.join(os.getcwd(), '..', 'batt_tech_performance', 'batt_tech_performance_atb24_conservative.csv')
batt_tech_performance_atb24_conservative.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'batt_tech_performance', 'batt_tech_performance_atb24_moderate.csv')
batt_tech_performance_atb24_moderate.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'batt_tech_performance', 'batt_tech_performance_atb24_advanced.csv')
batt_tech_performance_atb24_advanced.to_csv(output_path, index=False)

Successfully read ATB data from: 2024_v3_Workbook.xlsx


## PV & Battery Hybrid Prices

In [309]:
# Input Data Processing: PV and Battery Hybrid Prices

# Input Folder: pv_plus_batt_prices
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'PV + Storage Cost Scenario'
# Data Source: ATB_Data

#--------------------#

# Import the ATB data
pv_price_atb24_conservative = pd.read_csv('../pv_prices/pv_price_atb24_conservative.csv')
pv_price_atb24_moderate = pd.read_csv('../pv_prices/pv_price_atb24_moderate.csv')
pv_price_atb24_advanced = pd.read_csv('../pv_prices/pv_price_atb24_advanced.csv')

batt_price_atb24_conservative = pd.read_csv('../batt_prices/batt_price_atb24_conservative.csv')
batt_price_atb24_moderate = pd.read_csv('../batt_prices/batt_price_atb24_moderate.csv')
batt_price_atb24_advanced = pd.read_csv('../batt_prices/batt_price_atb24_advanced.csv')

#display(pv_price_atb24_conservative)
#display(batt_price_atb24_conservative)

# Initialize empty data frames for the csv output files (conservative, moderate, advanced)
pv_plus_batt_prices_column_names = [
    'year',
    'tech',
    'system_capex_per_kw_res',
    'system_capex_per_kw_nonres',
    'batt_capex_per_kwh_res',
    'batt_capex_per_kw_res',
    'batt_capex_per_kwh_nonres',
    'batt_capex_per_kw_nonres',
    'linear_constant_res',
    'linear_constant_nonres',
    'batt_om_per_kw_res',
    'batt_om_per_kwh_res',
    'batt_om_per_kw_nonres',
    'batt_om_per_kwh_nonres',
    'batt_replace_frac_kw',
    'batt_replace_frac_kwh'
]
pv_plus_batt_price_atb24_conservative = pd.DataFrame(columns=pv_plus_batt_prices_column_names)
pv_plus_batt_price_atb24_moderate = pd.DataFrame(columns=pv_plus_batt_prices_column_names)
pv_plus_batt_price_atb24_advanced = pd.DataFrame(columns=pv_plus_batt_prices_column_names)

# Populate the data frames
years = list(range(start_year, end_year + 1))
pv_plus_batt_price_atb24_conservative['year'] = years
pv_plus_batt_price_atb24_moderate['year'] = years
pv_plus_batt_price_atb24_advanced['year'] = years
pv_plus_batt_price_atb24_conservative['batt_replace_frac_kw'] = 0.174
pv_plus_batt_price_atb24_moderate['batt_replace_frac_kw'] = 0.174
pv_plus_batt_price_atb24_advanced['batt_replace_frac_kw'] = 0.174
pv_plus_batt_price_atb24_conservative['batt_replace_frac_kwh'] = 1
pv_plus_batt_price_atb24_moderate['batt_replace_frac_kwh'] = 1
pv_plus_batt_price_atb24_advanced['batt_replace_frac_kwh'] = 1
pv_plus_batt_price_atb24_conservative['tech'] = 'pv,batt'
pv_plus_batt_price_atb24_moderate['tech'] = 'pv,batt'
pv_plus_batt_price_atb24_advanced['tech'] = 'pv,batt'

##### PV Data
pv_plus_batt_price_atb24_conservative['system_capex_per_kw_nonres'] = pv_price_atb24_conservative['system_capex_per_kw_com'].values # Nonresidential
pv_plus_batt_price_atb24_moderate['system_capex_per_kw_nonres'] = pv_price_atb24_moderate['system_capex_per_kw_com'].values
pv_plus_batt_price_atb24_advanced['system_capex_per_kw_nonres'] = pv_price_atb24_advanced['system_capex_per_kw_com'].values

pv_plus_batt_price_atb24_conservative['system_capex_per_kw_res'] = pv_price_atb24_conservative['system_capex_per_kw_res'].values # Residential
pv_plus_batt_price_atb24_moderate['system_capex_per_kw_res'] = pv_price_atb24_moderate['system_capex_per_kw_res'].values
pv_plus_batt_price_atb24_advanced['system_capex_per_kw_res'] = pv_price_atb24_advanced['system_capex_per_kw_res'].values

##### Battery Data
pv_plus_batt_price_atb24_conservative['batt_capex_per_kwh_nonres'] = batt_price_atb24_conservative['batt_capex_per_kwh_nonres'].values # Nonresidential
pv_plus_batt_price_atb24_moderate['batt_capex_per_kwh_nonres'] = batt_price_atb24_moderate['batt_capex_per_kwh_nonres'].values
pv_plus_batt_price_atb24_advanced['batt_capex_per_kwh_nonres'] = batt_price_atb24_advanced['batt_capex_per_kwh_nonres'].values

pv_plus_batt_price_atb24_conservative['batt_capex_per_kwh_res'] = batt_price_atb24_conservative['batt_capex_per_kwh_res'].values # Residential
pv_plus_batt_price_atb24_moderate['batt_capex_per_kwh_res'] = batt_price_atb24_moderate['batt_capex_per_kwh_res'].values
pv_plus_batt_price_atb24_advanced['batt_capex_per_kwh_res'] = batt_price_atb24_advanced['batt_capex_per_kwh_res'].values

pv_plus_batt_price_atb24_conservative['batt_capex_per_kw_nonres'] = batt_price_atb24_conservative['batt_capex_per_kw_nonres'].values # Nonresidential
pv_plus_batt_price_atb24_moderate['batt_capex_per_kw_nonres'] = batt_price_atb24_moderate['batt_capex_per_kw_nonres'].values
pv_plus_batt_price_atb24_advanced['batt_capex_per_kw_nonres'] = batt_price_atb24_advanced['batt_capex_per_kw_nonres'].values

pv_plus_batt_price_atb24_conservative['batt_capex_per_kw_res'] = batt_price_atb24_conservative['batt_capex_per_kw_res'].values # Residential
pv_plus_batt_price_atb24_moderate['batt_capex_per_kw_res'] = batt_price_atb24_moderate['batt_capex_per_kw_res'].values
pv_plus_batt_price_atb24_advanced['batt_capex_per_kw_res'] = batt_price_atb24_advanced['batt_capex_per_kw_res'].values

pv_plus_batt_price_atb24_conservative['batt_om_per_kwh_nonres'] = batt_price_atb24_conservative['batt_om_per_kwh_nonres'].values # Nonresidential
pv_plus_batt_price_atb24_moderate['batt_om_per_kwh_nonres'] = batt_price_atb24_moderate['batt_om_per_kwh_nonres'].values
pv_plus_batt_price_atb24_advanced['batt_om_per_kwh_nonres'] = batt_price_atb24_advanced['batt_om_per_kwh_nonres'].values

pv_plus_batt_price_atb24_conservative['batt_om_per_kwh_res'] = batt_price_atb24_conservative['batt_om_per_kwh_res'].values # Residential
pv_plus_batt_price_atb24_moderate['batt_om_per_kwh_res'] = batt_price_atb24_moderate['batt_om_per_kwh_res'].values
pv_plus_batt_price_atb24_advanced['batt_om_per_kwh_res'] = batt_price_atb24_advanced['batt_om_per_kwh_res'].values

pv_plus_batt_price_atb24_conservative['batt_om_per_kw_nonres'] = batt_price_atb24_conservative['batt_om_per_kw_nonres'].values # Nonresidential
pv_plus_batt_price_atb24_moderate['batt_om_per_kw_nonres'] = batt_price_atb24_moderate['batt_om_per_kw_nonres'].values
pv_plus_batt_price_atb24_advanced['batt_om_per_kw_nonres'] = batt_price_atb24_advanced['batt_om_per_kw_nonres'].values

pv_plus_batt_price_atb24_conservative['batt_om_per_kw_res'] = batt_price_atb24_conservative['batt_om_per_kw_res'].values # Residential
pv_plus_batt_price_atb24_moderate['batt_om_per_kw_res'] = batt_price_atb24_moderate['batt_om_per_kw_res'].values
pv_plus_batt_price_atb24_advanced['batt_om_per_kw_res'] = batt_price_atb24_advanced['batt_om_per_kw_res'].values

pv_plus_batt_price_atb24_conservative['linear_constant_nonres'] = batt_price_atb24_conservative['linear_constant_nonres'].values # Nonresidential
pv_plus_batt_price_atb24_moderate['linear_constant_nonres'] = batt_price_atb24_moderate['linear_constant_nonres'].values
pv_plus_batt_price_atb24_advanced['linear_constant_nonres'] = batt_price_atb24_advanced['linear_constant_nonres'].values

pv_plus_batt_price_atb24_conservative['linear_constant_res'] = batt_price_atb24_conservative['linear_constant_res'].values # Residential
pv_plus_batt_price_atb24_moderate['linear_constant_res'] = batt_price_atb24_moderate['linear_constant_res'].values
pv_plus_batt_price_atb24_advanced['linear_constant_res'] = batt_price_atb24_advanced['linear_constant_res'].values

#display(pv_plus_batt_price_atb24_conservative)
#display(pv_plus_batt_price_atb24_moderate)
#display(pv_plus_batt_price_atb24_advanced)

# Save the csv files
output_path = os.path.join(os.getcwd(), '..', 'pv_plus_batt_prices', 'pv_plus_batt_price_atb24_conservative.csv')
pv_plus_batt_price_atb24_conservative.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'pv_plus_batt_prices', 'pv_plus_batt_price_atb24_moderate.csv')
pv_plus_batt_price_atb24_moderate.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'pv_plus_batt_prices', 'pv_plus_batt_price_atb24_advanced.csv')
pv_plus_batt_price_atb24_advanced.to_csv(output_path, index=False)

## Financial Inputs

In [2]:
# Input Data Processing: Financial Inputs

# Input Folder: financing_terms
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'Financing Scenario'
# Data Source: ATB_Data

#--------------------#

# Import the ATB data
# Financial Scenario Options: R&D, Market
# Financial Scenario Selected: Market
atb_file_path = os.path.join(atb_data_folder, atb_data_file)
atb_data = pd.ExcelFile(atb_file_path)
print(f'Successfully read ATB data from: {atb_data_file}')

# Assign the individual tabs to separate data frames
pv_dist_com_full = atb_data.parse('Solar - PV Dist. Comm')
pv_dist_res_full = atb_data.parse('Solar - PV Dist. Res')

# Extract the relevant data
pv_dist_com_finance = pv_dist_com_full.iloc[56:87, 10:41] # Commercial
pv_dist_com_finance.columns = pv_dist_com_finance.iloc[0]
pv_dist_com_finance = pv_dist_com_finance[1:]
pv_dist_com_finance.columns = pv_dist_com_finance.columns.astype(float).fillna(0).astype(int).astype(str)
pv_dist_com_finance.columns.values[0] = 0
pv_dist_com_finance.columns.values[1] = 1

pv_dist_res_finance = pv_dist_res_full.iloc[56:88, 10:41] # Residential
pv_dist_res_finance.columns = pv_dist_res_finance.iloc[0]
pv_dist_res_finance = pv_dist_res_finance[1:]
pv_dist_res_finance.columns = pv_dist_res_finance.columns.astype(float).fillna(0).astype(int).astype(str)
pv_dist_res_finance.columns.values[0] = 0
pv_dist_res_finance.columns.values[1] = 1

#display(pv_dist_com_finance)
#display(pv_dist_res_finance)

##### Loan Interest Rate (ATB Assumption: 'Calculated Interest Rate Real')
pv_dist_com_loanrate = pv_dist_com_finance.iloc [4:7, :] # Commercial
pv_dist_res_loanrate = pv_dist_res_finance.iloc [4:7, :] # Residential
#display(pv_dist_com_loanrate)
#display(pv_dist_res_loanrate)

##### Real Discount Rate (ATB Assumption: 'WACC Real')
pv_dist_com_discountrate = pv_dist_com_finance.iloc [21:24, :] # Commercial
pv_dist_res_discountrate = pv_dist_res_finance.iloc [21:24, :] # Residential
#display(pv_dist_com_discountrate)
#display(pv_dist_res_discountrate)

##### Tax Rate 
pv_dist_com_taxrate = pv_dist_res_finance.iloc [17:18, :] # Commercial
pv_dist_res_taxrate = pv_dist_res_finance.iloc [17:18, :] # Residential
#display(pv_dist_com_taxrate)
#display(pv_dist_res_taxrate)

# Initialize empty data frames for the csv output files (conservative, moderate, advanced)
financing_column_names = [
    'year',
    'economic_lifetime_yrs',
    'loan_term_yrs_res',
    'loan_interest_rate_res',
    'down_payment_fraction_res',
    'real_discount_rate_res',
    'tax_rate_res',
    'loan_term_yrs_nonres',
    'loan_interest_rate_nonres',
    'down_payment_fraction_nonres',
    'real_discount_rate_nonres',
    'tax_rate_nonres'
]
financing_atb24_conservative = pd.DataFrame(columns=financing_column_names)
financing_atb24_moderate = pd.DataFrame(columns=financing_column_names)
financing_atb24_advanced = pd.DataFrame(columns=financing_column_names)

# Populate the data frames
years = list(range(start_year, end_year + 1))
financing_atb24_conservative['year'] = years
financing_atb24_moderate['year'] = years
financing_atb24_advanced['year'] = years

financing_atb24_conservative['economic_lifetime_yrs'] = 30
financing_atb24_moderate['economic_lifetime_yrs'] = 30
financing_atb24_advanced['economic_lifetime_yrs'] = 30

financing_atb24_conservative['loan_term_yrs_res'] = 30
financing_atb24_moderate['loan_term_yrs_res'] = 30
financing_atb24_advanced['loan_term_yrs_res'] = 30

financing_atb24_conservative['down_payment_fraction_res'] = 1
financing_atb24_moderate['down_payment_fraction_res'] = 1
financing_atb24_advanced['down_payment_fraction_res'] = 1

financing_atb24_conservative['loan_term_yrs_nonres'] = 30
financing_atb24_moderate['loan_term_yrs_nonres'] = 30
financing_atb24_advanced['loan_term_yrs_nonres'] = 30

financing_atb24_conservative['down_payment_fraction_nonres'] = 1
financing_atb24_moderate['down_payment_fraction_nonres'] = 1
financing_atb24_advanced['down_payment_fraction_nonres'] = 1

##### Loan Interest Rate (ATB Assumption: 'Calculated Interest Rate Real')
pv_dist_com_loanrate_conservative = pv_dist_com_loanrate[pv_dist_com_loanrate.iloc[:, 1] == 'Conservative'] # conservative - Nonresidential
pv_dist_com_loanrate_conservative.columns = pv_dist_com_loanrate_conservative.columns.astype(int)
pv_dist_com_loanrate_conservative_values = pv_dist_com_loanrate_conservative.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_conservative['loan_interest_rate_nonres'] = financing_atb24_conservative['year'].apply(
    lambda y: pv_dist_com_loanrate_conservative_values[2022] if y < 2022 else pv_dist_com_loanrate_conservative_values.get(y, None)
)
pv_dist_res_loanrate_conservative = pv_dist_res_loanrate[pv_dist_res_loanrate.iloc[:, 1] == 'Conservative'] # conservative - Residential
pv_dist_res_loanrate_conservative.columns = pv_dist_res_loanrate_conservative.columns.astype(int)
pv_dist_res_loanrate_conservative_values = pv_dist_res_loanrate_conservative.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_conservative['loan_interest_rate_res'] = financing_atb24_conservative['year'].apply(
    lambda y: pv_dist_res_loanrate_conservative_values[2022] if y < 2022 else pv_dist_res_loanrate_conservative_values.get(y, None)
)
pv_dist_com_loanrate_moderate = pv_dist_com_loanrate[pv_dist_com_loanrate.iloc[:, 1] == 'Moderate'] # moderate - Nonresidential
pv_dist_com_loanrate_moderate.columns = pv_dist_com_loanrate_moderate.columns.astype(int)
pv_dist_com_loanrate_moderate_values = pv_dist_com_loanrate_moderate.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_moderate['loan_interest_rate_nonres'] = financing_atb24_moderate['year'].apply(
    lambda y: pv_dist_com_loanrate_moderate_values[2022] if y < 2022 else pv_dist_com_loanrate_moderate_values.get(y, None)
)
pv_dist_res_loanrate_moderate = pv_dist_res_loanrate[pv_dist_res_loanrate.iloc[:, 1] == 'Moderate'] # moderate - Residential
pv_dist_res_loanrate_moderate.columns = pv_dist_res_loanrate_moderate.columns.astype(int)
pv_dist_res_loanrate_moderate_values = pv_dist_res_loanrate_moderate.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_moderate['loan_interest_rate_res'] = financing_atb24_moderate['year'].apply(
    lambda y: pv_dist_res_loanrate_moderate_values[2022] if y < 2022 else pv_dist_res_loanrate_moderate_values.get(y, None)
)
pv_dist_com_loanrate_advanced = pv_dist_com_loanrate[pv_dist_com_loanrate.iloc[:, 1] == 'Advanced'] # advanced - Nonresidential
pv_dist_com_loanrate_advanced.columns = pv_dist_com_loanrate_advanced.columns.astype(int)
pv_dist_com_loanrate_advanced_values = pv_dist_com_loanrate_advanced.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_advanced['loan_interest_rate_nonres'] = financing_atb24_advanced['year'].apply(
    lambda y: pv_dist_com_loanrate_advanced_values[2022] if y < 2022 else pv_dist_com_loanrate_advanced_values.get(y, None)
)
pv_dist_res_loanrate_advanced = pv_dist_res_loanrate[pv_dist_res_loanrate.iloc[:, 1] == 'Advanced'] # advanced - Residential
pv_dist_res_loanrate_advanced.columns = pv_dist_res_loanrate_advanced.columns.astype(int)
pv_dist_res_loanrate_advanced_values = pv_dist_res_loanrate_advanced.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_advanced['loan_interest_rate_res'] = financing_atb24_advanced['year'].apply(
    lambda y: pv_dist_res_loanrate_advanced_values[2022] if y < 2022 else pv_dist_res_loanrate_advanced_values.get(y, None)
)

##### Real Discount Rate (ATB Assumption: 'WACC Real')
pv_dist_com_discountrate_conservative = pv_dist_com_discountrate[pv_dist_com_discountrate.iloc[:, 1] == 'Conservative'] # conservative - Nonresidential
pv_dist_com_discountrate_conservative.columns = pv_dist_com_discountrate_conservative.columns.astype(int)
pv_dist_com_discountrate_conservative_values = pv_dist_com_discountrate_conservative.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_conservative['real_discount_rate_nonres'] = financing_atb24_conservative['year'].apply(
    lambda y: pv_dist_com_discountrate_conservative_values[2022] if y < 2022 else pv_dist_com_discountrate_conservative_values.get(y, None)
)
pv_dist_res_discountrate_conservative = pv_dist_res_discountrate[pv_dist_res_discountrate.iloc[:, 1] == 'Conservative'] # conservative - Residential
pv_dist_res_discountrate_conservative.columns = pv_dist_res_discountrate_conservative.columns.astype(int)
pv_dist_res_discountrate_conservative_values = pv_dist_res_discountrate_conservative.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_conservative['real_discount_rate_res'] = financing_atb24_conservative['year'].apply(
    lambda y: pv_dist_res_discountrate_conservative_values[2022] if y < 2022 else pv_dist_res_discountrate_conservative_values.get(y, None)
)
pv_dist_com_discountrate_moderate = pv_dist_com_discountrate[pv_dist_com_discountrate.iloc[:, 1] == 'Moderate'] # moderate - Nonresidential
pv_dist_com_discountrate_moderate.columns = pv_dist_com_discountrate_moderate.columns.astype(int)
pv_dist_com_discountrate_moderate_values = pv_dist_com_discountrate_moderate.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_moderate['real_discount_rate_nonres'] = financing_atb24_moderate['year'].apply(
    lambda y: pv_dist_com_discountrate_moderate_values[2022] if y < 2022 else pv_dist_com_discountrate_moderate_values.get(y, None)
)
pv_dist_res_discountrate_moderate = pv_dist_res_discountrate[pv_dist_res_discountrate.iloc[:, 1] == 'Moderate'] # moderate - Residential
pv_dist_res_discountrate_moderate.columns = pv_dist_res_discountrate_moderate.columns.astype(int)
pv_dist_res_discountrate_moderate_values = pv_dist_res_discountrate_moderate.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_moderate['real_discount_rate_res'] = financing_atb24_moderate['year'].apply(
    lambda y: pv_dist_res_discountrate_moderate_values[2022] if y < 2022 else pv_dist_res_discountrate_moderate_values.get(y, None)
)
pv_dist_com_discountrate_advanced = pv_dist_com_discountrate[pv_dist_com_discountrate.iloc[:, 1] == 'Advanced'] # advanced - Nonresidential
pv_dist_com_discountrate_advanced.columns = pv_dist_com_discountrate_advanced.columns.astype(int)
pv_dist_com_discountrate_advanced_values = pv_dist_com_discountrate_advanced.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_advanced['real_discount_rate_nonres'] = financing_atb24_advanced['year'].apply(
    lambda y: pv_dist_com_discountrate_advanced_values[2022] if y < 2022 else pv_dist_com_discountrate_advanced_values.get(y, None)
)
pv_dist_res_discountrate_advanced = pv_dist_res_discountrate[pv_dist_res_discountrate.iloc[:, 1] == 'Advanced'] # advanced - Residential
pv_dist_res_discountrate_advanced.columns = pv_dist_res_discountrate_advanced.columns.astype(int)
pv_dist_res_discountrate_advanced_values = pv_dist_res_discountrate_advanced.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_advanced['real_discount_rate_res'] = financing_atb24_advanced['year'].apply(
    lambda y: pv_dist_res_discountrate_advanced_values[2022] if y < 2022 else pv_dist_res_discountrate_advanced_values.get(y, None)
)

##### Tax Rate
pv_dist_com_taxrate_conservative = pv_dist_com_taxrate # conservative - Nonresidential
pv_dist_com_taxrate_conservative.columns = pv_dist_com_taxrate_conservative.columns.astype(int)
pv_dist_com_taxrate_conservative_values = pv_dist_com_taxrate_conservative.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_conservative['tax_rate_nonres'] = financing_atb24_conservative['year'].apply(
    lambda y: pv_dist_com_taxrate_conservative_values[2022] if y < 2022 else pv_dist_com_taxrate_conservative_values.get(y, None)
)
pv_dist_res_taxrate_conservative = pv_dist_res_taxrate # conservative - Residential
pv_dist_res_taxrate_conservative.columns = pv_dist_res_taxrate_conservative.columns.astype(int)
pv_dist_res_taxrate_conservative_values = pv_dist_res_taxrate_conservative.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_conservative['tax_rate_res'] = financing_atb24_conservative['year'].apply(
    lambda y: pv_dist_res_taxrate_conservative_values[2022] if y < 2022 else pv_dist_res_taxrate_conservative_values.get(y, None)
)
pv_dist_com_taxrate_moderate = pv_dist_com_taxrate # moderate - Nonresidential
pv_dist_com_taxrate_moderate.columns = pv_dist_com_taxrate_moderate.columns.astype(int)
pv_dist_com_taxrate_moderate_values = pv_dist_com_taxrate_moderate.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_moderate['tax_rate_nonres'] = financing_atb24_moderate['year'].apply(
    lambda y: pv_dist_com_taxrate_moderate_values[2022] if y < 2022 else pv_dist_com_taxrate_moderate_values.get(y, None)
)
pv_dist_res_taxrate_moderate = pv_dist_res_taxrate # moderate - Residential
pv_dist_res_taxrate_moderate.columns = pv_dist_res_taxrate_moderate.columns.astype(int)
pv_dist_res_taxrate_moderate_values = pv_dist_res_taxrate_moderate.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_moderate['tax_rate_res'] = financing_atb24_moderate['year'].apply(
    lambda y: pv_dist_res_taxrate_moderate_values[2022] if y < 2022 else pv_dist_res_taxrate_moderate_values.get(y, None)
)
pv_dist_com_taxrate_advanced = pv_dist_com_taxrate # advanced - Nonresidential
pv_dist_com_taxrate_advanced.columns = pv_dist_com_taxrate_advanced.columns.astype(int)
pv_dist_com_taxrate_advanced_values = pv_dist_com_taxrate_advanced.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_advanced['tax_rate_nonres'] = financing_atb24_advanced['year'].apply(
    lambda y: pv_dist_com_taxrate_advanced_values[2022] if y < 2022 else pv_dist_com_taxrate_advanced_values.get(y, None)
)
pv_dist_res_taxrate_advanced = pv_dist_res_taxrate # advanced - Residential
pv_dist_res_taxrate_advanced.columns = pv_dist_res_taxrate_advanced.columns.astype(int)
pv_dist_res_taxrate_advanced_values = pv_dist_res_taxrate_advanced.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_advanced['tax_rate_res'] = financing_atb24_advanced['year'].apply(
    lambda y: pv_dist_res_taxrate_advanced_values[2022] if y < 2022 else pv_dist_res_taxrate_advanced_values.get(y, None)
)

#display(financing_atb24_conservative)
#display(financing_atb24_moderate)
#display(financing_atb24_advanced)

# Save the csv files
output_path = os.path.join(os.getcwd(), '..', 'financing_terms', 'financing_atb24_conservative.csv')
financing_atb24_conservative.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'financing_terms', 'financing_atb24_moderate.csv')
financing_atb24_moderate.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'financing_terms', 'financing_atb24_advanced.csv')
financing_atb24_advanced.to_csv(output_path, index=False)

Successfully read ATB data from: 2024_v3_Workbook.xlsx


## Depreciation Schedules

In [119]:
# Input Data Processing: Depreciation Schedules

# Input Folder: depreciation_schedules
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'Depreciation Scenario'
# Data Source: ATB_Data

#--------------------#

# Import the ATB data
atb_file_path = os.path.join(atb_data_folder, atb_data_file)
atb_data = pd.ExcelFile(atb_file_path)
print(f'Successfully read ATB data from: {atb_data_file}')

# Assign the individual tabs to separate data frames
pv_dist_com_full = atb_data.parse('Solar - PV Dist. Comm')
pv_dist_res_full = atb_data.parse('Solar - PV Dist. Res')

# Extract the relevant data
pv_dist_com_depreciation = pv_dist_com_full.iloc[365:386, 10:41] # Commercial
pv_dist_com_depreciation.columns = pv_dist_com_depreciation.iloc[0]
pv_dist_com_depreciation = pv_dist_com_depreciation[1:]
pv_dist_com_depreciation.columns.values[0] = 0
pv_dist_com_depreciation.columns.values[1] = 1
pv_dist_com_depreciation.columns = pv_dist_com_depreciation.columns.astype(float).fillna(0).astype(int).astype(str)

pv_dist_res_depreciation = pv_dist_res_full.iloc[365:386, 10:41] # Residential
pv_dist_res_depreciation.columns = pv_dist_res_depreciation.iloc[0]
pv_dist_res_depreciation = pv_dist_res_depreciation[1:]
pv_dist_res_depreciation.columns.values[0] = 0
pv_dist_res_depreciation.columns.values[1] = 1
pv_dist_res_depreciation.columns = pv_dist_res_depreciation.columns.astype(float).fillna(0).astype(int).astype(str)

##### Conservative
pv_dist_com_depreciation_conservative = pv_dist_com_depreciation.iloc[14:20, :] # Commercial
pv_dist_com_depreciation_conservative = pv_dist_com_depreciation_conservative.T
new_header = pv_dist_com_depreciation_conservative.iloc[0]
pv_dist_com_depreciation_conservative = pv_dist_com_depreciation_conservative[2:]
pv_dist_com_depreciation_conservative.columns = new_header

pv_dist_res_depreciation_conservative = pv_dist_res_depreciation.iloc[14:20, :] # Residential
pv_dist_res_depreciation_conservative = pv_dist_res_depreciation_conservative.T
new_header = pv_dist_res_depreciation_conservative.iloc[0]
pv_dist_res_depreciation_conservative = pv_dist_res_depreciation_conservative[2:]
pv_dist_res_depreciation_conservative.columns = new_header

##### Moderate
pv_dist_com_depreciation_moderate = pv_dist_com_depreciation.iloc[7:13, :] # Commercial
pv_dist_com_depreciation_moderate = pv_dist_com_depreciation_moderate.T
new_header = pv_dist_com_depreciation_moderate.iloc[0]
pv_dist_com_depreciation_moderate = pv_dist_com_depreciation_moderate[2:]
pv_dist_com_depreciation_moderate.columns = new_header

pv_dist_res_depreciation_moderate = pv_dist_res_depreciation.iloc[7:13, :] # Residential
pv_dist_res_depreciation_moderate = pv_dist_res_depreciation_moderate.T
new_header = pv_dist_res_depreciation_moderate.iloc[0]
pv_dist_res_depreciation_moderate = pv_dist_res_depreciation_moderate[2:]
pv_dist_res_depreciation_moderate.columns = new_header

##### Advanced
pv_dist_com_depreciation_advanced = pv_dist_com_depreciation.iloc[0:6, :] # Commercial
pv_dist_com_depreciation_advanced = pv_dist_com_depreciation_advanced.T
new_header = pv_dist_com_depreciation_advanced.iloc[0]
pv_dist_com_depreciation_advanced = pv_dist_com_depreciation_advanced[2:]
pv_dist_com_depreciation_advanced.columns = new_header

pv_dist_res_depreciation_advanced = pv_dist_res_depreciation.iloc[0:6, :] # Residential
pv_dist_res_depreciation_advanced = pv_dist_res_depreciation_advanced.T
new_header = pv_dist_res_depreciation_advanced.iloc[0]
pv_dist_res_depreciation_advanced = pv_dist_res_depreciation_advanced[2:]
pv_dist_res_depreciation_advanced.columns = new_header

#display(pv_dist_com_depreciation)
#display(pv_dist_com_depreciation_conservative)
#display(pv_dist_com_depreciation_moderate)
#display(pv_dist_com_depreciation_advanced)
#display(pv_dist_res_depreciation)
#display(pv_dist_res_depreciation_conservative)
#display(pv_dist_res_depreciation_moderate)
#display(pv_dist_res_depreciation_advanced)

# Initialize empty data frames for the csv output files (conservative, moderate, advanced)
depreciation_column_names = [
    'year',
    'sector_abbr',
    1,
    2,
    3,
    4,
    5,
    6,
]
depreciation_atb24_conservative = pd.DataFrame(columns=depreciation_column_names)
depreciation_atb24_moderate = pd.DataFrame(columns=depreciation_column_names)
depreciation_atb24_advanced = pd.DataFrame(columns=depreciation_column_names)

# Populate the data frames
years = list(range(start_year, end_year + 1))
depreciation_atb24_conservative['year'] = years*3
depreciation_atb24_moderate['year'] = years*3
depreciation_atb24_advanced['year'] = years*3

num_years = end_year - start_year + 1
com_entries = ['com'] * num_years
ind_entries = ['ind'] * num_years
res_entries = ['res'] * num_years
sector_abbr_list = com_entries + ind_entries + res_entries
depreciation_atb24_conservative['sector_abbr'] = sector_abbr_list
depreciation_atb24_moderate['sector_abbr'] = sector_abbr_list
depreciation_atb24_advanced['sector_abbr'] = sector_abbr_list

##### Conservative
pv_dist_com_depreciation_conservative.index = pv_dist_com_depreciation_conservative.index.astype(int)
pv_dist_res_depreciation_conservative.index = pv_dist_res_depreciation_conservative.index.astype(int)
for col in [1, 2, 3, 4, 5, 6]:
    # conservative - Commercial
    com_mask_2014_2021 = (depreciation_atb24_conservative['sector_abbr'] == 'com') & (depreciation_atb24_conservative['year'] >= 2014) & (depreciation_atb24_conservative['year'] <= 2021)
    com_mask_2022_2050 = (depreciation_atb24_conservative['sector_abbr'] == 'com') & (depreciation_atb24_conservative['year'] >= 2022) & (depreciation_atb24_conservative['year'] <= 2050)
    depreciation_atb24_conservative.loc[com_mask_2014_2021, col] = pv_dist_com_depreciation_conservative.loc[2022, col]
    depreciation_atb24_conservative.loc[com_mask_2022_2050, col] = pv_dist_com_depreciation_conservative.loc[depreciation_atb24_conservative.loc[com_mask_2022_2050, 'year'], col].astype(depreciation_atb24_conservative[col].dtype).values
    # conservative - Industrial
    ind_mask_2014_2021 = (depreciation_atb24_conservative['sector_abbr'] == 'ind') & (depreciation_atb24_conservative['year'] >= 2014) & (depreciation_atb24_conservative['year'] <= 2021)
    ind_mask_2022_2050 = (depreciation_atb24_conservative['sector_abbr'] == 'ind') & (depreciation_atb24_conservative['year'] >= 2022) & (depreciation_atb24_conservative['year'] <= 2050)
    depreciation_atb24_conservative.loc[ind_mask_2014_2021, col] = pv_dist_com_depreciation_conservative.loc[2022, col]
    depreciation_atb24_conservative.loc[ind_mask_2022_2050, col] = pv_dist_com_depreciation_conservative.loc[depreciation_atb24_conservative.loc[ind_mask_2022_2050, 'year'], col].astype(depreciation_atb24_conservative[col].dtype).values
    # conservative - Residential
    res_mask_2014_2021 = (depreciation_atb24_conservative['sector_abbr'] == 'res') & (depreciation_atb24_conservative['year'] >= 2014) & (depreciation_atb24_conservative['year'] <= 2021)
    res_mask_2022_2050 = (depreciation_atb24_conservative['sector_abbr'] == 'res') & (depreciation_atb24_conservative['year'] >= 2022) & (depreciation_atb24_conservative['year'] <= 2050)
    depreciation_atb24_conservative.loc[res_mask_2014_2021, col] = pv_dist_res_depreciation_conservative.loc[2022, col]
    depreciation_atb24_conservative.loc[res_mask_2022_2050, col] = pv_dist_res_depreciation_conservative.loc[depreciation_atb24_conservative.loc[res_mask_2022_2050, 'year'], col].astype(depreciation_atb24_conservative[col].dtype).values

##### Moderate
pv_dist_com_depreciation_moderate.index = pv_dist_com_depreciation_moderate.index.astype(int)
pv_dist_res_depreciation_moderate.index = pv_dist_res_depreciation_moderate.index.astype(int)
for col in [1, 2, 3, 4, 5, 6]:
    # moderate - Commercial
    com_mask_2014_2021 = (depreciation_atb24_moderate['sector_abbr'] == 'com') & (depreciation_atb24_moderate['year'] >= 2014) & (depreciation_atb24_moderate['year'] <= 2021)
    com_mask_2022_2050 = (depreciation_atb24_moderate['sector_abbr'] == 'com') & (depreciation_atb24_moderate['year'] >= 2022) & (depreciation_atb24_moderate['year'] <= 2050)
    depreciation_atb24_moderate.loc[com_mask_2014_2021, col] = pv_dist_com_depreciation_moderate.loc[2022, col]
    depreciation_atb24_moderate.loc[com_mask_2022_2050, col] = pv_dist_com_depreciation_moderate.loc[depreciation_atb24_moderate.loc[com_mask_2022_2050, 'year'], col].astype(depreciation_atb24_moderate[col].dtype).values
    # moderate - Industrial
    ind_mask_2014_2021 = (depreciation_atb24_moderate['sector_abbr'] == 'ind') & (depreciation_atb24_moderate['year'] >= 2014) & (depreciation_atb24_moderate['year'] <= 2021)
    ind_mask_2022_2050 = (depreciation_atb24_moderate['sector_abbr'] == 'ind') & (depreciation_atb24_moderate['year'] >= 2022) & (depreciation_atb24_moderate['year'] <= 2050)
    depreciation_atb24_moderate.loc[ind_mask_2014_2021, col] = pv_dist_com_depreciation_moderate.loc[2022, col]
    depreciation_atb24_moderate.loc[ind_mask_2022_2050, col] = pv_dist_com_depreciation_moderate.loc[depreciation_atb24_moderate.loc[ind_mask_2022_2050, 'year'], col].astype(depreciation_atb24_moderate[col].dtype).values
    # moderate - Residential
    res_mask_2014_2021 = (depreciation_atb24_moderate['sector_abbr'] == 'res') & (depreciation_atb24_moderate['year'] >= 2014) & (depreciation_atb24_moderate['year'] <= 2021)
    res_mask_2022_2050 = (depreciation_atb24_moderate['sector_abbr'] == 'res') & (depreciation_atb24_moderate['year'] >= 2022) & (depreciation_atb24_moderate['year'] <= 2050)
    depreciation_atb24_moderate.loc[res_mask_2014_2021, col] = pv_dist_res_depreciation_moderate.loc[2022, col]
    depreciation_atb24_moderate.loc[res_mask_2022_2050, col] = pv_dist_res_depreciation_moderate.loc[depreciation_atb24_moderate.loc[res_mask_2022_2050, 'year'], col].astype(depreciation_atb24_moderate[col].dtype).values

##### Advanced
pv_dist_com_depreciation_advanced.index = pv_dist_com_depreciation_advanced.index.astype(int)
pv_dist_res_depreciation_advanced.index = pv_dist_res_depreciation_advanced.index.astype(int)
for col in [1, 2, 3, 4, 5, 6]:
    # advanced - Commercial
    com_mask_2014_2021 = (depreciation_atb24_advanced['sector_abbr'] == 'com') & (depreciation_atb24_advanced['year'] >= 2014) & (depreciation_atb24_advanced['year'] <= 2021)
    com_mask_2022_2050 = (depreciation_atb24_advanced['sector_abbr'] == 'com') & (depreciation_atb24_advanced['year'] >= 2022) & (depreciation_atb24_advanced['year'] <= 2050)
    depreciation_atb24_advanced.loc[com_mask_2014_2021, col] = pv_dist_com_depreciation_advanced.loc[2022, col]
    depreciation_atb24_advanced.loc[com_mask_2022_2050, col] = pv_dist_com_depreciation_advanced.loc[depreciation_atb24_advanced.loc[com_mask_2022_2050, 'year'], col].astype(depreciation_atb24_advanced[col].dtype).values
    # advanced - Industrial
    ind_mask_2014_2021 = (depreciation_atb24_advanced['sector_abbr'] == 'ind') & (depreciation_atb24_advanced['year'] >= 2014) & (depreciation_atb24_advanced['year'] <= 2021)
    ind_mask_2022_2050 = (depreciation_atb24_advanced['sector_abbr'] == 'ind') & (depreciation_atb24_advanced['year'] >= 2022) & (depreciation_atb24_advanced['year'] <= 2050)
    depreciation_atb24_advanced.loc[ind_mask_2014_2021, col] = pv_dist_com_depreciation_advanced.loc[2022, col]
    depreciation_atb24_advanced.loc[ind_mask_2022_2050, col] = pv_dist_com_depreciation_advanced.loc[depreciation_atb24_advanced.loc[ind_mask_2022_2050, 'year'], col].astype(depreciation_atb24_advanced[col].dtype).values
    # advanced - Residential
    res_mask_2014_2021 = (depreciation_atb24_advanced['sector_abbr'] == 'res') & (depreciation_atb24_advanced['year'] >= 2014) & (depreciation_atb24_advanced['year'] <= 2021)
    res_mask_2022_2050 = (depreciation_atb24_advanced['sector_abbr'] == 'res') & (depreciation_atb24_advanced['year'] >= 2022) & (depreciation_atb24_advanced['year'] <= 2050)
    depreciation_atb24_advanced.loc[res_mask_2014_2021, col] = pv_dist_res_depreciation_advanced.loc[2022, col]
    depreciation_atb24_advanced.loc[res_mask_2022_2050, col] = pv_dist_res_depreciation_advanced.loc[depreciation_atb24_advanced.loc[res_mask_2022_2050, 'year'], col].astype(depreciation_atb24_advanced[col].dtype).values

#display(depreciation_atb24_conservative)
#display(depreciation_atb24_moderate)
#display(depreciation_atb24_advanced)

# Save the csv files
output_path = os.path.join(os.getcwd(), '..', 'depreciation_schedules', 'depreciation_atb24_conservative.csv')
depreciation_atb24_conservative.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'depreciation_schedules', 'depreciation_atb24_moderate.csv')
depreciation_atb24_moderate.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'depreciation_schedules', 'depreciation_atb24_advanced.csv')
depreciation_atb24_advanced.to_csv(output_path, index=False)

Successfully read ATB data from: 2024_v3_Workbook.xlsx


## Carbon Intensities

In [ ]:
# Input Data Processing: Carbon Intensities

# Input Folder: carbon_intensities
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'Carbon Intensity Scenario'
# Data Source: Cambium_Data

#--------------------#

# Import the Cambium data
cambium_workbook_path = os.path.join(cambium_data_folder, cambium_data_workbook)
cambium_data = pd.ExcelFile(cambium_workbook_path)
print(f'Successfully read Cambium data from: {cambium_data_workbook}')

with xw.App() as app:
    cambium_data_workbook = app.books.open(cambium_workbook_path)
    cambium_data_sheet = cambium_data_workbook.sheets('Levelized LRMER')

    # Set the universal cell values
    cambium_data_sheet.range('D7').value = 'CO2e' # Emission
    cambium_data_sheet.range('D8').value = 'Combined' # Emission stage
    cambium_data_sheet.range('D10').value = '20' # Evaluation period (years) -- Cambium default
    cambium_data_sheet.range('D11').value = '0.03' # Discount rate (real) -- Cambium default
    cambium_data_sheet.range('D13').value = '100-year (AR6)' # Global Warming Potentials -- Cambium default
    cambium_data_sheet.range('D14').value = 'End-use' # Location -- Cambium default
    
    # --- First Scenario (Mid-case) ---
    cambium_data_sheet.range('D12').value = 'Mid-case' # Scenario
    
    # 2025
    cambium_data_sheet.range('D9').value = '2025'
    lrmer_2025_midcase = cambium_data_sheet.range('B349:W9109').value
    headers = cambium_data_sheet.range('B349:W9109').value[0]
    lrmer_2025_midcase = pd.DataFrame(lrmer_2025_midcase[1:], columns=headers)
    lrmer_2025_midcase['year'] = 2025

    # 2030
    cambium_data_sheet.range('D9').value = '2030'
    lrmer_2030_midcase = cambium_data_sheet.range('B349:W9109').value
    headers = cambium_data_sheet.range('B349:W9109').value[0]
    lrmer_2030_midcase = pd.DataFrame(lrmer_2030_midcase[1:], columns=headers)
    lrmer_2030_midcase['year'] = 2030

    # 2035
    cambium_data_sheet.range('D9').value = '2035'
    lrmer_2035_midcase = cambium_data_sheet.range('B349:W9109').value
    headers = cambium_data_sheet.range('B349:W9109').value[0]
    lrmer_2035_midcase = pd.DataFrame(lrmer_2035_midcase[1:], columns=headers)
    lrmer_2035_midcase['year'] = 2035

    # 2040
    cambium_data_sheet.range('D9').value = '2040'
    lrmer_2040_midcase = cambium_data_sheet.range('B349:W9109').value
    headers = cambium_data_sheet.range('B349:W9109').value[0]
    lrmer_2040_midcase = pd.DataFrame(lrmer_2040_midcase[1:], columns=headers)
    lrmer_2040_midcase['year'] = 2040

    # 2045
    cambium_data_sheet.range('D9').value = '2045'
    lrmer_2045_midcase = cambium_data_sheet.range('B349:W9109').value
    headers = cambium_data_sheet.range('B349:W9109').value[0]
    lrmer_2045_midcase = pd.DataFrame(lrmer_2045_midcase[1:], columns=headers)
    lrmer_2045_midcase['year'] = 2045

    # 2050
    cambium_data_sheet.range('D9').value = '2050'
    lrmer_2050_midcase = cambium_data_sheet.range('B349:W9109').value
    headers = cambium_data_sheet.range('B349:W9109').value[0]
    lrmer_2050_midcase = pd.DataFrame(lrmer_2050_midcase[1:], columns=headers)
    lrmer_2050_midcase['year'] = 2050

    # --- Second Scenario (Low RE Costs) ---
    cambium_data_sheet.range('D12').value = 'Low RE Costs' # Scenario
    
    # 2025
    cambium_data_sheet.range('D9').value = '2025'
    lrmer_2025_lowrecosts = cambium_data_sheet.range('B349:W9109').value
    headers = cambium_data_sheet.range('B349:W9109').value[0]
    lrmer_2025_lowrecosts = pd.DataFrame(lrmer_2025_lowrecosts[1:], columns=headers)
    lrmer_2025_lowrecosts['year'] = 2025

    # 2030
    cambium_data_sheet.range('D9').value = '2030'
    lrmer_2030_lowrecosts = cambium_data_sheet.range('B349:W9109').value
    headers = cambium_data_sheet.range('B349:W9109').value[0]
    lrmer_2030_lowrecosts = pd.DataFrame(lrmer_2030_lowrecosts[1:], columns=headers)
    lrmer_2030_lowrecosts['year'] = 2030

    # 2035
    cambium_data_sheet.range('D9').value = '2035'
    lrmer_2035_lowrecosts = cambium_data_sheet.range('B349:W9109').value
    headers = cambium_data_sheet.range('B349:W9109').value[0]
    lrmer_2035_lowrecosts = pd.DataFrame(lrmer_2035_lowrecosts[1:], columns=headers)
    lrmer_2035_lowrecosts['year'] = 2035

    # 2040
    cambium_data_sheet.range('D9').value = '2040'
    lrmer_2040_lowrecosts = cambium_data_sheet.range('B349:W9109').value
    headers = cambium_data_sheet.range('B349:W9109').value[0]
    lrmer_2040_lowrecosts = pd.DataFrame(lrmer_2040_lowrecosts[1:], columns=headers)
    lrmer_2040_lowrecosts['year'] = 2040

    # 2045
    cambium_data_sheet.range('D9').value = '2045'
    lrmer_2045_lowrecosts = cambium_data_sheet.range('B349:W9109').value
    headers = cambium_data_sheet.range('B349:W9109').value[0]
    lrmer_2045_lowrecosts = pd.DataFrame(lrmer_2045_lowrecosts[1:], columns=headers)
    lrmer_2045_lowrecosts['year'] = 2045

    # 2050
    cambium_data_sheet.range('D9').value = '2050'
    lrmer_2050_lowrecosts = cambium_data_sheet.range('B349:W9109').value
    headers = cambium_data_sheet.range('B349:W9109').value[0]
    lrmer_2050_lowrecosts = pd.DataFrame(lrmer_2050_lowrecosts[1:], columns=headers)
    lrmer_2050_lowrecosts['year'] = 2050

    # --- Third Scenario (High RE Costs) ---
    cambium_data_sheet.range('D12').value = 'Low RE Costs' # Scenario
    
    # 2025
    cambium_data_sheet.range('D9').value = '2025'
    lrmer_2025_highrecosts = cambium_data_sheet.range('B349:W9109').value
    headers = cambium_data_sheet.range('B349:W9109').value[0]
    lrmer_2025_highrecosts = pd.DataFrame(lrmer_2025_highrecosts[1:], columns=headers)
    lrmer_2025_highrecosts['year'] = 2025

    # 2030
    cambium_data_sheet.range('D9').value = '2030'
    lrmer_2030_highrecosts = cambium_data_sheet.range('B349:W9109').value
    headers = cambium_data_sheet.range('B349:W9109').value[0]
    lrmer_2030_highrecosts = pd.DataFrame(lrmer_2030_highrecosts[1:], columns=headers)
    lrmer_2030_highrecosts['year'] = 2030

    # 2035
    cambium_data_sheet.range('D9').value = '2035'
    lrmer_2035_highrecosts = cambium_data_sheet.range('B349:W9109').value
    headers = cambium_data_sheet.range('B349:W9109').value[0]
    lrmer_2035_highrecosts = pd.DataFrame(lrmer_2035_highrecosts[1:], columns=headers)
    lrmer_2035_highrecosts['year'] = 2035

    # 2040
    cambium_data_sheet.range('D9').value = '2040'
    lrmer_2040_highrecosts = cambium_data_sheet.range('B349:W9109').value
    headers = cambium_data_sheet.range('B349:W9109').value[0]
    lrmer_2040_highrecosts = pd.DataFrame(lrmer_2040_highrecosts[1:], columns=headers)
    lrmer_2040_highrecosts['year'] = 2040

    # 2045
    cambium_data_sheet.range('D9').value = '2045'
    lrmer_2045_highrecosts = cambium_data_sheet.range('B349:W9109').value
    headers = cambium_data_sheet.range('B349:W9109').value[0]
    lrmer_2045_highrecosts = pd.DataFrame(lrmer_2045_highrecosts[1:], columns=headers)
    lrmer_2045_highrecosts['year'] = 2045

    # 2050
    cambium_data_sheet.range('D9').value = '2050'
    lrmer_2050_highrecosts = cambium_data_sheet.range('B349:W9109').value
    headers = cambium_data_sheet.range('B349:W9109').value[0]
    lrmer_2050_highrecosts = pd.DataFrame(lrmer_2050_highrecosts[1:], columns=headers)
    lrmer_2050_highrecosts['year'] = 2050

# Combine the years into one data frame for each scenario
lrmer_midcase_list = [lrmer_2025_midcase, lrmer_2030_midcase, lrmer_2035_midcase, lrmer_2040_midcase, lrmer_2045_midcase, lrmer_2050_midcase]
lrmer_lowrecosts_list = [lrmer_2025_lowrecosts, lrmer_2030_lowrecosts, lrmer_2035_lowrecosts, lrmer_2040_lowrecosts, lrmer_2045_lowrecosts, lrmer_2050_lowrecosts]
lrmer_highrecosts_list = [lrmer_2025_highrecosts, lrmer_2030_highrecosts, lrmer_2035_highrecosts, lrmer_2040_highrecosts, lrmer_2045_highrecosts, lrmer_2050_highrecosts]
lrmer_cambium25_midcase = pd.concat(lrmer_midcase_list, ignore_index=True)
lrmer_cambium25_lowrecosts = pd.concat(lrmer_lowrecosts_list, ignore_index=True)
lrmer_cambium25_highrecosts = pd.concat(lrmer_highrecosts_list, ignore_index=True)

# Drop columns from the data frame for each scenario
columns_to_drop = ['Hour of the day', 'Day of the year', 'Month']
lrmer_cambium25_midcase.drop(columns=columns_to_drop, inplace=True)
lrmer_cambium25_lowrecosts.drop(columns=columns_to_drop, inplace=True)
lrmer_cambium25_highrecosts.drop(columns=columns_to_drop, inplace=True)

# Rename columns in the data frame for each scenario
lrmer_cambium25_midcase.rename(columns={'Hour of the year': 'hour'}, inplace=True)
lrmer_cambium25_lowrecosts.rename(columns={'Hour of the year': 'hour'}, inplace=True)
lrmer_cambium25_highrecosts.rename(columns={'Hour of the year': 'hour'}, inplace=True)

# Re-order the columns in the data frame for each scenario
new_column_order = ['hour', 'year', 'CAISO', 'ERCOT', 'FRCC', 'ISONE', 'MISO Central', 'MISO North', 'MISO South', 'Northern Grid East', 'Northern Grid South', 'Northern Grid West', 'NYISO', 'PJM East', 'PJM West', 'SERTP', 'SPP North', 'SPP South', 'West Connect North', 'West Connect South']
lrmer_cambium25_midcase = lrmer_cambium25_midcase[new_column_order]
lrmer_cambium25_lowrecosts = lrmer_cambium25_lowrecosts[new_column_order]
lrmer_cambium25_highrecosts = lrmer_cambium25_highrecosts[new_column_order]

In [ ]:
# Set the variables for all scenarios

# Early Years
early_years = list(range(start_year, base_year+1))

# Filler Years
filler_years = [2026, 2027, 2028, 2029, 2031, 2032, 2033, 2034, 2036, 2037, 2038, 2039, 2041, 2042, 2043, 2044, 2046, 2047, 2048, 2049]

# --- First Scenario (Mid-case) ---
rows_2025_midcase = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2025].copy()

# Early Years
early_years_df = []
for year in early_years:
    early_years_new_df = rows_2025_midcase.copy()
    early_years_new_df['year'] = year
    early_years_df.append(early_years_new_df)

# Filler Years
filler_years_df = []
for year in filler_years:
    filler_years_new_df = rows_2025_midcase.copy()
    filler_years_new_df['year'] = year
    # Set other columns (except 'year' and 'hour') to 'N/A'
    cols_to_fill = [col for col in filler_years_new_df.columns if col not in ['year', 'hour']]
    filler_years_new_df[cols_to_fill] = 'N/A'
    filler_years_df.append(filler_years_new_df)

lrmer_cambium25_midcase = pd.concat([lrmer_cambium25_midcase] + early_years_df + filler_years_df, ignore_index=True)
lrmer_cambium25_midcase = lrmer_cambium25_midcase.sort_values(by=['year', 'hour'])

# --- Second Scenario (Low RE Costs) ---
rows_2025_lowrecosts = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2025].copy()

# Early Years
early_years_df = []
for year in early_years:
    early_years_new_df = rows_2025_lowrecosts.copy()
    early_years_new_df['year'] = year
    early_years_df.append(early_years_new_df)

# Filler Years
filler_years_df = []
for year in filler_years:
    filler_years_new_df = rows_2025_lowrecosts.copy()
    filler_years_new_df['year'] = year
    # Set other columns (except 'year' and 'hour') to 'N/A'
    cols_to_fill = [col for col in filler_years_new_df.columns if col not in ['year', 'hour']]
    filler_years_new_df[cols_to_fill] = 'N/A'
    filler_years_df.append(filler_years_new_df)

lrmer_cambium25_lowrecosts = pd.concat([lrmer_cambium25_lowrecosts] + early_years_df + filler_years_df, ignore_index=True)
lrmer_cambium25_lowrecosts = lrmer_cambium25_lowrecosts.sort_values(by=['year', 'hour'])

# --- Third Scenario (High RE Costs) ---
rows_2025_highrecosts = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2025].copy()

# Early Years
early_years_df = []
for year in early_years:
    early_years_new_df = rows_2025_highrecosts.copy()
    early_years_new_df['year'] = year
    early_years_df.append(early_years_new_df)

# Filler Years
filler_years_df = []
for year in filler_years:
    filler_years_new_df = rows_2025_highrecosts.copy()
    filler_years_new_df['year'] = year
    # Set other columns (except 'year' and 'hour') to 'N/A'
    cols_to_fill = [col for col in filler_years_new_df.columns if col not in ['year', 'hour']]
    filler_years_new_df[cols_to_fill] = 'N/A'
    filler_years_df.append(filler_years_new_df)

lrmer_cambium25_highrecosts = pd.concat([lrmer_cambium25_highrecosts] + early_years_df + filler_years_df, ignore_index=True)
lrmer_cambium25_highrecosts = lrmer_cambium25_highrecosts.sort_values(by=['year', 'hour'])

In [31]:
# --- First Scenario (Mid-case) ---
lrmer_cambium25_midcase_2025 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2025].copy()
lrmer_cambium25_midcase_2025 = lrmer_cambium25_midcase_2025.reset_index(drop=True)
lrmer_cambium25_midcase_2026 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2026].copy()
lrmer_cambium25_midcase_2026 = lrmer_cambium25_midcase_2026.reset_index(drop=True)
lrmer_cambium25_midcase_2027 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2027].copy()
lrmer_cambium25_midcase_2027 = lrmer_cambium25_midcase_2027.reset_index(drop=True)
lrmer_cambium25_midcase_2028 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2028].copy()
lrmer_cambium25_midcase_2028 = lrmer_cambium25_midcase_2028.reset_index(drop=True)
lrmer_cambium25_midcase_2029 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2029].copy()
lrmer_cambium25_midcase_2029 = lrmer_cambium25_midcase_2029.reset_index(drop=True)
lrmer_cambium25_midcase_2030 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2030].copy()
lrmer_cambium25_midcase_2030 = lrmer_cambium25_midcase_2030.reset_index(drop=True)
lrmer_cambium25_midcase_2031 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2031].copy()
lrmer_cambium25_midcase_2031 = lrmer_cambium25_midcase_2031.reset_index(drop=True)
lrmer_cambium25_midcase_2032 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2032].copy()
lrmer_cambium25_midcase_2032 = lrmer_cambium25_midcase_2032.reset_index(drop=True)
lrmer_cambium25_midcase_2033 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2033].copy()
lrmer_cambium25_midcase_2033 = lrmer_cambium25_midcase_2033.reset_index(drop=True)
lrmer_cambium25_midcase_2034 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2034].copy()
lrmer_cambium25_midcase_2034 = lrmer_cambium25_midcase_2034.reset_index(drop=True)
lrmer_cambium25_midcase_2035 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2035].copy()
lrmer_cambium25_midcase_2035 = lrmer_cambium25_midcase_2035.reset_index(drop=True)
lrmer_cambium25_midcase_2036 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2036].copy()
lrmer_cambium25_midcase_2036 = lrmer_cambium25_midcase_2036.reset_index(drop=True)
lrmer_cambium25_midcase_2037 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2037].copy()
lrmer_cambium25_midcase_2037 = lrmer_cambium25_midcase_2037.reset_index(drop=True)
lrmer_cambium25_midcase_2038 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2038].copy()
lrmer_cambium25_midcase_2038 = lrmer_cambium25_midcase_2038.reset_index(drop=True)
lrmer_cambium25_midcase_2039 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2039].copy()
lrmer_cambium25_midcase_2039 = lrmer_cambium25_midcase_2039.reset_index(drop=True)
lrmer_cambium25_midcase_2040 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2040].copy()
lrmer_cambium25_midcase_2040 = lrmer_cambium25_midcase_2040.reset_index(drop=True)
lrmer_cambium25_midcase_2041 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2041].copy()
lrmer_cambium25_midcase_2041 = lrmer_cambium25_midcase_2041.reset_index(drop=True)
lrmer_cambium25_midcase_2042 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2042].copy()
lrmer_cambium25_midcase_2042 = lrmer_cambium25_midcase_2042.reset_index(drop=True)
lrmer_cambium25_midcase_2043 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2043].copy()
lrmer_cambium25_midcase_2043 = lrmer_cambium25_midcase_2043.reset_index(drop=True)
lrmer_cambium25_midcase_2044 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2044].copy()
lrmer_cambium25_midcase_2044 = lrmer_cambium25_midcase_2044.reset_index(drop=True)
lrmer_cambium25_midcase_2045 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2045].copy()
lrmer_cambium25_midcase_2045 = lrmer_cambium25_midcase_2045.reset_index(drop=True)
lrmer_cambium25_midcase_2046 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2046].copy()
lrmer_cambium25_midcase_2046 = lrmer_cambium25_midcase_2046.reset_index(drop=True)
lrmer_cambium25_midcase_2047 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2047].copy()
lrmer_cambium25_midcase_2047 = lrmer_cambium25_midcase_2047.reset_index(drop=True)
lrmer_cambium25_midcase_2048 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2048].copy()
lrmer_cambium25_midcase_2048 = lrmer_cambium25_midcase_2048.reset_index(drop=True)
lrmer_cambium25_midcase_2049 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2049].copy()
lrmer_cambium25_midcase_2049 = lrmer_cambium25_midcase_2049.reset_index(drop=True)
lrmer_cambium25_midcase_2050 = lrmer_cambium25_midcase[lrmer_cambium25_midcase['year'] == 2050].copy()
lrmer_cambium25_midcase_2050 = lrmer_cambium25_midcase_2050.reset_index(drop=True)

cols_to_modify = [col for col in lrmer_cambium25_midcase_2025.columns if col not in ['hour', 'year']]

#display(lrmer_cambium25_midcase_2025)
#display(lrmer_cambium25_midcase_2030)
#display(lrmer_cambium25_midcase_2035)
#display(lrmer_cambium25_midcase_2040)
#display(lrmer_cambium25_midcase_2045)
#display(lrmer_cambium25_midcase_2050)

# Fill in the 2026-2029 values
lrmer_cambium25_midcase_2025_2030_change = lrmer_cambium25_midcase_2025.copy()
lrmer_cambium25_midcase_2025_2030_change[cols_to_modify] = (lrmer_cambium25_midcase_2030[cols_to_modify] - lrmer_cambium25_midcase_2025[cols_to_modify]) / 5
#display(lrmer_cambium25_midcase_2025_2030_change)
lrmer_cambium25_midcase_2026[cols_to_modify] = lrmer_cambium25_midcase_2025[cols_to_modify] + lrmer_cambium25_midcase_2025_2030_change[cols_to_modify]
lrmer_cambium25_midcase_2027[cols_to_modify] = lrmer_cambium25_midcase_2026[cols_to_modify] + lrmer_cambium25_midcase_2025_2030_change[cols_to_modify]
lrmer_cambium25_midcase_2028[cols_to_modify] = lrmer_cambium25_midcase_2027[cols_to_modify] + lrmer_cambium25_midcase_2025_2030_change[cols_to_modify]
lrmer_cambium25_midcase_2029[cols_to_modify] = lrmer_cambium25_midcase_2028[cols_to_modify] + lrmer_cambium25_midcase_2025_2030_change[cols_to_modify]

# Add the 2026-2029 values back into the main data frame
for col in cols_to_modify:
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2026, col] = lrmer_cambium25_midcase_2026[col].values
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2027, col] = lrmer_cambium25_midcase_2027[col].values
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2028, col] = lrmer_cambium25_midcase_2028[col].values
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2029, col] = lrmer_cambium25_midcase_2029[col].values

# Fill in the 2031-2034 values
lrmer_cambium25_midcase_2030_2035_change = lrmer_cambium25_midcase_2030.copy()
lrmer_cambium25_midcase_2030_2035_change[cols_to_modify] = (lrmer_cambium25_midcase_2035[cols_to_modify] - lrmer_cambium25_midcase_2030[cols_to_modify]) / 5
#display(lrmer_cambium25_midcase_2030_2035_change)
lrmer_cambium25_midcase_2031[cols_to_modify] = lrmer_cambium25_midcase_2030[cols_to_modify] + lrmer_cambium25_midcase_2030_2035_change[cols_to_modify]
lrmer_cambium25_midcase_2032[cols_to_modify] = lrmer_cambium25_midcase_2031[cols_to_modify] + lrmer_cambium25_midcase_2030_2035_change[cols_to_modify]
lrmer_cambium25_midcase_2033[cols_to_modify] = lrmer_cambium25_midcase_2032[cols_to_modify] + lrmer_cambium25_midcase_2030_2035_change[cols_to_modify]
lrmer_cambium25_midcase_2034[cols_to_modify] = lrmer_cambium25_midcase_2033[cols_to_modify] + lrmer_cambium25_midcase_2030_2035_change[cols_to_modify]

# Add the 2031-2034 values back into the main data frame
for col in cols_to_modify:
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2031, col] = lrmer_cambium25_midcase_2031[col].values
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2032, col] = lrmer_cambium25_midcase_2032[col].values
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2033, col] = lrmer_cambium25_midcase_2033[col].values
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2034, col] = lrmer_cambium25_midcase_2034[col].values

# Fill in the 2036-2039 values
lrmer_cambium25_midcase_2035_2040_change = lrmer_cambium25_midcase_2035.copy()
lrmer_cambium25_midcase_2035_2040_change[cols_to_modify] = (lrmer_cambium25_midcase_2040[cols_to_modify] - lrmer_cambium25_midcase_2035[cols_to_modify]) / 5
#display(lrmer_cambium25_midcase_2035_2040_change)
lrmer_cambium25_midcase_2036[cols_to_modify] = lrmer_cambium25_midcase_2035[cols_to_modify] + lrmer_cambium25_midcase_2035_2040_change[cols_to_modify]
lrmer_cambium25_midcase_2037[cols_to_modify] = lrmer_cambium25_midcase_2036[cols_to_modify] + lrmer_cambium25_midcase_2035_2040_change[cols_to_modify]
lrmer_cambium25_midcase_2038[cols_to_modify] = lrmer_cambium25_midcase_2037[cols_to_modify] + lrmer_cambium25_midcase_2035_2040_change[cols_to_modify]
lrmer_cambium25_midcase_2039[cols_to_modify] = lrmer_cambium25_midcase_2038[cols_to_modify] + lrmer_cambium25_midcase_2035_2040_change[cols_to_modify]

# Add the 2036-2039 values back into the main data frame
for col in cols_to_modify:
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2036, col] = lrmer_cambium25_midcase_2036[col].values
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2037, col] = lrmer_cambium25_midcase_2037[col].values
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2038, col] = lrmer_cambium25_midcase_2038[col].values
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2039, col] = lrmer_cambium25_midcase_2039[col].values

# Fill in the 2041-2044 values
lrmer_cambium25_midcase_2040_2045_change = lrmer_cambium25_midcase_2040.copy()
lrmer_cambium25_midcase_2040_2045_change[cols_to_modify] = (lrmer_cambium25_midcase_2045[cols_to_modify] - lrmer_cambium25_midcase_2040[cols_to_modify]) / 5
#display(lrmer_cambium25_midcase_2040_2045_change)
lrmer_cambium25_midcase_2041[cols_to_modify] = lrmer_cambium25_midcase_2040[cols_to_modify] + lrmer_cambium25_midcase_2040_2045_change[cols_to_modify]
lrmer_cambium25_midcase_2042[cols_to_modify] = lrmer_cambium25_midcase_2041[cols_to_modify] + lrmer_cambium25_midcase_2040_2045_change[cols_to_modify]
lrmer_cambium25_midcase_2043[cols_to_modify] = lrmer_cambium25_midcase_2042[cols_to_modify] + lrmer_cambium25_midcase_2040_2045_change[cols_to_modify]
lrmer_cambium25_midcase_2044[cols_to_modify] = lrmer_cambium25_midcase_2043[cols_to_modify] + lrmer_cambium25_midcase_2040_2045_change[cols_to_modify]

# Add the 2041-2044 values back into the main data frame
for col in cols_to_modify:
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2041, col] = lrmer_cambium25_midcase_2041[col].values
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2042, col] = lrmer_cambium25_midcase_2042[col].values
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2043, col] = lrmer_cambium25_midcase_2043[col].values
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2044, col] = lrmer_cambium25_midcase_2044[col].values

# Fill in the 2046-2049 values
lrmer_cambium25_midcase_2045_2050_change = lrmer_cambium25_midcase_2045.copy()
lrmer_cambium25_midcase_2045_2050_change[cols_to_modify] = (lrmer_cambium25_midcase_2050[cols_to_modify] - lrmer_cambium25_midcase_2045[cols_to_modify]) / 5
#display(lrmer_cambium25_midcase_2045_2050_change)
lrmer_cambium25_midcase_2046[cols_to_modify] = lrmer_cambium25_midcase_2045[cols_to_modify] + lrmer_cambium25_midcase_2045_2050_change[cols_to_modify]
lrmer_cambium25_midcase_2047[cols_to_modify] = lrmer_cambium25_midcase_2046[cols_to_modify] + lrmer_cambium25_midcase_2045_2050_change[cols_to_modify]
lrmer_cambium25_midcase_2048[cols_to_modify] = lrmer_cambium25_midcase_2047[cols_to_modify] + lrmer_cambium25_midcase_2045_2050_change[cols_to_modify]
lrmer_cambium25_midcase_2049[cols_to_modify] = lrmer_cambium25_midcase_2048[cols_to_modify] + lrmer_cambium25_midcase_2045_2050_change[cols_to_modify]

# Add the 2046-2049 values back into the main data frame
for col in cols_to_modify:
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2046, col] = lrmer_cambium25_midcase_2046[col].values
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2047, col] = lrmer_cambium25_midcase_2047[col].values
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2048, col] = lrmer_cambium25_midcase_2048[col].values
    lrmer_cambium25_midcase.loc[lrmer_cambium25_midcase['year'] == 2049, col] = lrmer_cambium25_midcase_2049[col].values
    
# Save the csv files
csv_output_path = os.path.join(os.getcwd(), '..', 'carbon_intensities', 'lrmer_cambium25_midcase.csv')
lrmer_cambium25_midcase.to_csv(csv_output_path, index=False)
display(lrmer_cambium25_midcase)

# Save the zip files
zip_output_path = os.path.join(os.getcwd(), '..', 'carbon_intensities', 'lrmer_cambium25_midcase.zip') # Save ZIP in the same directory as CSV
with zipfile.ZipFile(zip_output_path, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
    # zf.write(source_path, arcname=name_inside_zip)
    # arcname specifies the name of the file *inside* the zip archive
    zf.write(csv_output_path, arcname='lrmer_cambium25_midcase.csv')

# Save individual csv files for each region
common_cols = ['hour', 'year']
subregion_specific_cols = [col for col in lrmer_cambium25_midcase.columns if col not in common_cols]
for subregion_col_name in subregion_specific_cols:
    new_df_for_subregion = lrmer_cambium25_midcase[common_cols + [subregion_col_name]].copy()
    cleaned_subregion_name = subregion_col_name.lower().replace(' ', '')
    csv_filename = f'lrmer_cambium25_midcase_{cleaned_subregion_name}.csv'
    csv_output_path = os.path.join(os.getcwd(), '..', 'carbon_intensities', csv_filename)
    new_df_for_subregion.to_csv(csv_output_path, index=False)

,hour,year,CAISO,ERCOT,FRCC,ISONE,MISO Central,MISO North,MISO South,Northern Grid East,Northern Grid South,Northern Grid West,NYISO,PJM East,PJM West,SERTP,SPP North,SPP South,West Connect North,West Connect South
52560,0.0,2014,265.98279,222.235735,449.715836,162.898088,238.627375,260.24279,328.350043,263.459874,325.379452,121.865996,138.026886,298.710999,296.775904,341.227794,323.010509,297.467328,246.598538,303.11481
52561,1.0,2014,275.306713,212.250255,451.52206,152.764978,242.927463,263.397842,328.376441,263.85975,334.295643,119.806746,126.561451,295.505304,284.984504,331.680594,322.837477,296.831469,239.463051,311.72375
52562,2.0,2014,277.466526,216.096185,455.423269,149.689116,244.288716,257.086289,329.744976,261.023751,327.54531,123.42091,120.791188,298.498772,281.409268,350.521421,328.482652,299.942588,248.077503,305.761102
52563,3.0,2014,265.879652,213.427297,457.633794,155.921928,251.66369,245.406036,324.462281,269.202539,327.666118,116.911113,129.309841,292.681837,289.164071,343.51752,324.121953,295.549883,262.25006,312.519338
52564,4.0,2014,263.433818,218.695246,454.224272,159.624773,259.981015,262.063119,339.680728,269.272063,314.797623,121.534171,119.152102,303.615807,301.056908,337.195876,330.620266,306.467579,249.348077,308.777333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52555,8755.0,2050,64.60571,117.16275,303.04177,52.13809,126.39879,245.37758,283.15733,254.68478,238.81222,84.37509,0.0,282.69334,113.72678,240.08804,99.08243,134.38117,271.59119,205.31132
52556,8756.0,2050,67.26736,134.86609,309.67094,49.02553,130.11531,236.21465,288.69437,244.33304,231.85027,72.57626,0.0,277.44156,103.96576,234.01251,100.30593,130.85396,288.79892,206.25138
52557,8757.0,2050,64.63253,144.69574,316.97759,44.81679,120.15147,223.83072,293.04246,250.95625,241.23538,71.1468,0.0,271.11512,100.08805,237.09634,97.51125,140.96577,288.3274,207.92554
52558,8758.0,2050,65.12789,143.90468,321.91199,40.71947,112.2869,228.25759,275.54654,244.82815,232.33371,70.92892,0.0,264.57833,105.74886,253.62386,94.42849,144.47801,281.85187,207.35104


In [33]:
# --- Second Scenario (Low RE Costs) ---
lrmer_cambium25_lowrecosts_2025 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2025].copy()
lrmer_cambium25_lowrecosts_2025 = lrmer_cambium25_lowrecosts_2025.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2026 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2026].copy()
lrmer_cambium25_lowrecosts_2026 = lrmer_cambium25_lowrecosts_2026.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2027 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2027].copy()
lrmer_cambium25_lowrecosts_2027 = lrmer_cambium25_lowrecosts_2027.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2028 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2028].copy()
lrmer_cambium25_lowrecosts_2028 = lrmer_cambium25_lowrecosts_2028.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2029 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2029].copy()
lrmer_cambium25_lowrecosts_2029 = lrmer_cambium25_lowrecosts_2029.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2030 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2030].copy()
lrmer_cambium25_lowrecosts_2030 = lrmer_cambium25_lowrecosts_2030.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2031 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2031].copy()
lrmer_cambium25_lowrecosts_2031 = lrmer_cambium25_lowrecosts_2031.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2032 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2032].copy()
lrmer_cambium25_lowrecosts_2032 = lrmer_cambium25_lowrecosts_2032.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2033 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2033].copy()
lrmer_cambium25_lowrecosts_2033 = lrmer_cambium25_lowrecosts_2033.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2034 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2034].copy()
lrmer_cambium25_lowrecosts_2034 = lrmer_cambium25_lowrecosts_2034.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2035 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2035].copy()
lrmer_cambium25_lowrecosts_2035 = lrmer_cambium25_lowrecosts_2035.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2036 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2036].copy()
lrmer_cambium25_lowrecosts_2036 = lrmer_cambium25_lowrecosts_2036.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2037 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2037].copy()
lrmer_cambium25_lowrecosts_2037 = lrmer_cambium25_lowrecosts_2037.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2038 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2038].copy()
lrmer_cambium25_lowrecosts_2038 = lrmer_cambium25_lowrecosts_2038.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2039 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2039].copy()
lrmer_cambium25_lowrecosts_2039 = lrmer_cambium25_lowrecosts_2039.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2040 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2040].copy()
lrmer_cambium25_lowrecosts_2040 = lrmer_cambium25_lowrecosts_2040.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2041 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2041].copy()
lrmer_cambium25_lowrecosts_2041 = lrmer_cambium25_lowrecosts_2041.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2042 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2042].copy()
lrmer_cambium25_lowrecosts_2042 = lrmer_cambium25_lowrecosts_2042.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2043 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2043].copy()
lrmer_cambium25_lowrecosts_2043 = lrmer_cambium25_lowrecosts_2043.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2044 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2044].copy()
lrmer_cambium25_lowrecosts_2044 = lrmer_cambium25_lowrecosts_2044.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2045 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2045].copy()
lrmer_cambium25_lowrecosts_2045 = lrmer_cambium25_lowrecosts_2045.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2046 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2046].copy()
lrmer_cambium25_lowrecosts_2046 = lrmer_cambium25_lowrecosts_2046.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2047 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2047].copy()
lrmer_cambium25_lowrecosts_2047 = lrmer_cambium25_lowrecosts_2047.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2048 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2048].copy()
lrmer_cambium25_lowrecosts_2048 = lrmer_cambium25_lowrecosts_2048.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2049 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2049].copy()
lrmer_cambium25_lowrecosts_2049 = lrmer_cambium25_lowrecosts_2049.reset_index(drop=True)
lrmer_cambium25_lowrecosts_2050 = lrmer_cambium25_lowrecosts[lrmer_cambium25_lowrecosts['year'] == 2050].copy()
lrmer_cambium25_lowrecosts_2050 = lrmer_cambium25_lowrecosts_2050.reset_index(drop=True)

cols_to_modify = [col for col in lrmer_cambium25_lowrecosts_2025.columns if col not in ['hour', 'year']]

#display(lrmer_cambium25_lowrecosts_2025)
#display(lrmer_cambium25_lowrecosts_2030)
#display(lrmer_cambium25_lowrecosts_2035)
#display(lrmer_cambium25_lowrecosts_2040)
#display(lrmer_cambium25_lowrecosts_2045)
#display(lrmer_cambium25_lowrecosts_2050)

# Fill in the 2026-2029 values
lrmer_cambium25_lowrecosts_2025_2030_change = lrmer_cambium25_lowrecosts_2025.copy()
lrmer_cambium25_lowrecosts_2025_2030_change[cols_to_modify] = (lrmer_cambium25_lowrecosts_2030[cols_to_modify] - lrmer_cambium25_lowrecosts_2025[cols_to_modify]) / 5
#display(lrmer_cambium25_lowrecosts_2025_2030_change)
lrmer_cambium25_lowrecosts_2026[cols_to_modify] = lrmer_cambium25_lowrecosts_2025[cols_to_modify] + lrmer_cambium25_lowrecosts_2025_2030_change[cols_to_modify]
lrmer_cambium25_lowrecosts_2027[cols_to_modify] = lrmer_cambium25_lowrecosts_2026[cols_to_modify] + lrmer_cambium25_lowrecosts_2025_2030_change[cols_to_modify]
lrmer_cambium25_lowrecosts_2028[cols_to_modify] = lrmer_cambium25_lowrecosts_2027[cols_to_modify] + lrmer_cambium25_lowrecosts_2025_2030_change[cols_to_modify]
lrmer_cambium25_lowrecosts_2029[cols_to_modify] = lrmer_cambium25_lowrecosts_2028[cols_to_modify] + lrmer_cambium25_lowrecosts_2025_2030_change[cols_to_modify]

# Add the 2026-2029 values back into the main data frame
for col in cols_to_modify:
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2026, col] = lrmer_cambium25_lowrecosts_2026[col].values
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2027, col] = lrmer_cambium25_lowrecosts_2027[col].values
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2028, col] = lrmer_cambium25_lowrecosts_2028[col].values
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2029, col] = lrmer_cambium25_lowrecosts_2029[col].values

# Fill in the 2031-2034 values
lrmer_cambium25_lowrecosts_2030_2035_change = lrmer_cambium25_lowrecosts_2030.copy()
lrmer_cambium25_lowrecosts_2030_2035_change[cols_to_modify] = (lrmer_cambium25_lowrecosts_2035[cols_to_modify] - lrmer_cambium25_lowrecosts_2030[cols_to_modify]) / 5
#display(lrmer_cambium25_lowrecosts_2030_2035_change)
lrmer_cambium25_lowrecosts_2031[cols_to_modify] = lrmer_cambium25_lowrecosts_2030[cols_to_modify] + lrmer_cambium25_lowrecosts_2030_2035_change[cols_to_modify]
lrmer_cambium25_lowrecosts_2032[cols_to_modify] = lrmer_cambium25_lowrecosts_2031[cols_to_modify] + lrmer_cambium25_lowrecosts_2030_2035_change[cols_to_modify]
lrmer_cambium25_lowrecosts_2033[cols_to_modify] = lrmer_cambium25_lowrecosts_2032[cols_to_modify] + lrmer_cambium25_lowrecosts_2030_2035_change[cols_to_modify]
lrmer_cambium25_lowrecosts_2034[cols_to_modify] = lrmer_cambium25_lowrecosts_2033[cols_to_modify] + lrmer_cambium25_lowrecosts_2030_2035_change[cols_to_modify]

# Add the 2031-2034 values back into the main data frame
for col in cols_to_modify:
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2031, col] = lrmer_cambium25_lowrecosts_2031[col].values
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2032, col] = lrmer_cambium25_lowrecosts_2032[col].values
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2033, col] = lrmer_cambium25_lowrecosts_2033[col].values
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2034, col] = lrmer_cambium25_lowrecosts_2034[col].values

# Fill in the 2036-2039 values
lrmer_cambium25_lowrecosts_2035_2040_change = lrmer_cambium25_lowrecosts_2035.copy()
lrmer_cambium25_lowrecosts_2035_2040_change[cols_to_modify] = (lrmer_cambium25_lowrecosts_2040[cols_to_modify] - lrmer_cambium25_lowrecosts_2035[cols_to_modify]) / 5
#display(lrmer_cambium25_lowrecosts_2035_2040_change)
lrmer_cambium25_lowrecosts_2036[cols_to_modify] = lrmer_cambium25_lowrecosts_2035[cols_to_modify] + lrmer_cambium25_lowrecosts_2035_2040_change[cols_to_modify]
lrmer_cambium25_lowrecosts_2037[cols_to_modify] = lrmer_cambium25_lowrecosts_2036[cols_to_modify] + lrmer_cambium25_lowrecosts_2035_2040_change[cols_to_modify]
lrmer_cambium25_lowrecosts_2038[cols_to_modify] = lrmer_cambium25_lowrecosts_2037[cols_to_modify] + lrmer_cambium25_lowrecosts_2035_2040_change[cols_to_modify]
lrmer_cambium25_lowrecosts_2039[cols_to_modify] = lrmer_cambium25_lowrecosts_2038[cols_to_modify] + lrmer_cambium25_lowrecosts_2035_2040_change[cols_to_modify]

# Add the 2036-2039 values back into the main data frame
for col in cols_to_modify:
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2036, col] = lrmer_cambium25_lowrecosts_2036[col].values
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2037, col] = lrmer_cambium25_lowrecosts_2037[col].values
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2038, col] = lrmer_cambium25_lowrecosts_2038[col].values
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2039, col] = lrmer_cambium25_lowrecosts_2039[col].values

# Fill in the 2041-2044 values
lrmer_cambium25_lowrecosts_2040_2045_change = lrmer_cambium25_lowrecosts_2040.copy()
lrmer_cambium25_lowrecosts_2040_2045_change[cols_to_modify] = (lrmer_cambium25_lowrecosts_2045[cols_to_modify] - lrmer_cambium25_lowrecosts_2040[cols_to_modify]) / 5
#display(lrmer_cambium25_lowrecosts_2040_2045_change)
lrmer_cambium25_lowrecosts_2041[cols_to_modify] = lrmer_cambium25_lowrecosts_2040[cols_to_modify] + lrmer_cambium25_lowrecosts_2040_2045_change[cols_to_modify]
lrmer_cambium25_lowrecosts_2042[cols_to_modify] = lrmer_cambium25_lowrecosts_2041[cols_to_modify] + lrmer_cambium25_lowrecosts_2040_2045_change[cols_to_modify]
lrmer_cambium25_lowrecosts_2043[cols_to_modify] = lrmer_cambium25_lowrecosts_2042[cols_to_modify] + lrmer_cambium25_lowrecosts_2040_2045_change[cols_to_modify]
lrmer_cambium25_lowrecosts_2044[cols_to_modify] = lrmer_cambium25_lowrecosts_2043[cols_to_modify] + lrmer_cambium25_lowrecosts_2040_2045_change[cols_to_modify]

# Add the 2041-2044 values back into the main data frame
for col in cols_to_modify:
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2041, col] = lrmer_cambium25_lowrecosts_2041[col].values
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2042, col] = lrmer_cambium25_lowrecosts_2042[col].values
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2043, col] = lrmer_cambium25_lowrecosts_2043[col].values
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2044, col] = lrmer_cambium25_lowrecosts_2044[col].values

# Fill in the 2046-2049 values
lrmer_cambium25_lowrecosts_2045_2050_change = lrmer_cambium25_lowrecosts_2045.copy()
lrmer_cambium25_lowrecosts_2045_2050_change[cols_to_modify] = (lrmer_cambium25_lowrecosts_2050[cols_to_modify] - lrmer_cambium25_lowrecosts_2045[cols_to_modify]) / 5
#display(lrmer_cambium25_lowrecosts_2045_2050_change)
lrmer_cambium25_lowrecosts_2046[cols_to_modify] = lrmer_cambium25_lowrecosts_2045[cols_to_modify] + lrmer_cambium25_lowrecosts_2045_2050_change[cols_to_modify]
lrmer_cambium25_lowrecosts_2047[cols_to_modify] = lrmer_cambium25_lowrecosts_2046[cols_to_modify] + lrmer_cambium25_lowrecosts_2045_2050_change[cols_to_modify]
lrmer_cambium25_lowrecosts_2048[cols_to_modify] = lrmer_cambium25_lowrecosts_2047[cols_to_modify] + lrmer_cambium25_lowrecosts_2045_2050_change[cols_to_modify]
lrmer_cambium25_lowrecosts_2049[cols_to_modify] = lrmer_cambium25_lowrecosts_2048[cols_to_modify] + lrmer_cambium25_lowrecosts_2045_2050_change[cols_to_modify]

# Add the 2046-2049 values back into the main data frame
for col in cols_to_modify:
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2046, col] = lrmer_cambium25_lowrecosts_2046[col].values
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2047, col] = lrmer_cambium25_lowrecosts_2047[col].values
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2048, col] = lrmer_cambium25_lowrecosts_2048[col].values
    lrmer_cambium25_lowrecosts.loc[lrmer_cambium25_lowrecosts['year'] == 2049, col] = lrmer_cambium25_lowrecosts_2049[col].values
    
# Save the csv files
csv_output_path = os.path.join(os.getcwd(), '..', 'carbon_intensities', 'lrmer_cambium25_lowrecosts.csv')
lrmer_cambium25_lowrecosts.to_csv(csv_output_path, index=False)
display(lrmer_cambium25_lowrecosts)

# Save the zip files
zip_output_path = os.path.join(os.getcwd(), '..', 'carbon_intensities', 'lrmer_cambium25_lowrecosts.zip') # Save ZIP in the same directory as CSV
with zipfile.ZipFile(zip_output_path, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
    # zf.write(source_path, arcname=name_inside_zip)
    # arcname specifies the name of the file *inside* the zip archive
    zf.write(csv_output_path, arcname='lrmer_cambium25_lowrecosts.csv')

# Save individual csv files for each region
common_cols = ['hour', 'year']
subregion_specific_cols = [col for col in lrmer_cambium25_lowrecosts.columns if col not in common_cols]
for subregion_col_name in subregion_specific_cols:
    new_df_for_subregion = lrmer_cambium25_lowrecosts[common_cols + [subregion_col_name]].copy()
    cleaned_subregion_name = subregion_col_name.lower().replace(' ', '')
    csv_filename = f'lrmer_cambium25_lowrecosts_{cleaned_subregion_name}.csv' 
    csv_output_path = os.path.join(os.getcwd(), '..', 'carbon_intensities', csv_filename)
    new_df_for_subregion.to_csv(csv_output_path, index=False)

,hour,year,CAISO,ERCOT,FRCC,ISONE,MISO Central,MISO North,MISO South,Northern Grid East,Northern Grid South,Northern Grid West,NYISO,PJM East,PJM West,SERTP,SPP North,SPP South,West Connect North,West Connect South
52560,0.0,2014,250.909537,259.657884,361.790447,173.727856,289.570349,287.205881,273.361654,308.548517,337.784248,116.394147,136.510658,296.043003,298.684358,314.390326,294.7375,303.869325,268.3889,340.087962
52561,1.0,2014,250.570514,254.533574,386.441601,167.030741,299.000711,285.498372,274.763185,305.949988,339.789114,112.35259,126.916944,288.886588,296.709373,306.288738,310.102832,313.135536,269.112244,343.315599
52562,2.0,2014,256.945144,246.507638,386.808658,165.701043,304.399572,291.915664,268.29218,305.352316,341.677354,115.731454,130.793522,290.71044,293.33604,305.082826,313.882388,301.343976,265.266767,346.120288
52563,3.0,2014,258.871992,242.608934,390.997891,168.780619,307.631736,273.531061,264.158001,308.589644,339.597711,116.883474,123.88453,297.160986,295.519089,307.95283,310.638617,299.882245,268.853684,346.894729
52564,4.0,2014,254.855677,244.971937,386.691205,167.823366,296.848063,287.19998,275.348482,313.555234,339.963566,120.038867,124.939538,300.507918,298.348743,305.200823,317.623383,303.576868,272.463297,340.585472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52555,8755.0,2050,51.19803,67.79848,359.93487,46.11893,80.10502,229.72281,132.71015,220.77958,204.35338,47.72289,0.0,283.8647,129.1351,173.26215,60.36667,75.21548,184.01546,241.3622
52556,8756.0,2050,50.95333,96.54271,363.04147,43.00985,70.89764,227.63541,137.20531,236.66083,190.65357,45.56281,0.0,284.8584,131.97009,168.92336,64.58783,79.41255,181.49826,247.73475
52557,8757.0,2050,56.4739,107.59304,383.61838,43.61415,76.85529,211.76012,137.70365,228.11603,197.62446,46.26413,0.0,269.63377,105.98726,161.89146,79.14558,83.68735,182.82772,256.87516
52558,8758.0,2050,54.08352,130.79584,400.43373,38.80061,77.31043,203.67922,150.0745,207.07952,190.18801,33.41878,0.0,271.58243,118.67153,171.99254,85.24959,104.27759,167.19117,237.50685


In [39]:
# --- Third Scenario (High RE Costs) ---
lrmer_cambium25_highrecosts_2025 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2025].copy()
lrmer_cambium25_highrecosts_2025 = lrmer_cambium25_highrecosts_2025.reset_index(drop=True)
lrmer_cambium25_highrecosts_2026 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2026].copy()
lrmer_cambium25_highrecosts_2026 = lrmer_cambium25_highrecosts_2026.reset_index(drop=True)
lrmer_cambium25_highrecosts_2027 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2027].copy()
lrmer_cambium25_highrecosts_2027 = lrmer_cambium25_highrecosts_2027.reset_index(drop=True)
lrmer_cambium25_highrecosts_2028 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2028].copy()
lrmer_cambium25_highrecosts_2028 = lrmer_cambium25_highrecosts_2028.reset_index(drop=True)
lrmer_cambium25_highrecosts_2029 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2029].copy()
lrmer_cambium25_highrecosts_2029 = lrmer_cambium25_highrecosts_2029.reset_index(drop=True)
lrmer_cambium25_highrecosts_2030 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2030].copy()
lrmer_cambium25_highrecosts_2030 = lrmer_cambium25_highrecosts_2030.reset_index(drop=True)
lrmer_cambium25_highrecosts_2031 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2031].copy()
lrmer_cambium25_highrecosts_2031 = lrmer_cambium25_highrecosts_2031.reset_index(drop=True)
lrmer_cambium25_highrecosts_2032 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2032].copy()
lrmer_cambium25_highrecosts_2032 = lrmer_cambium25_highrecosts_2032.reset_index(drop=True)
lrmer_cambium25_highrecosts_2033 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2033].copy()
lrmer_cambium25_highrecosts_2033 = lrmer_cambium25_highrecosts_2033.reset_index(drop=True)
lrmer_cambium25_highrecosts_2034 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2034].copy()
lrmer_cambium25_highrecosts_2034 = lrmer_cambium25_highrecosts_2034.reset_index(drop=True)
lrmer_cambium25_highrecosts_2035 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2035].copy()
lrmer_cambium25_highrecosts_2035 = lrmer_cambium25_highrecosts_2035.reset_index(drop=True)
lrmer_cambium25_highrecosts_2036 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2036].copy()
lrmer_cambium25_highrecosts_2036 = lrmer_cambium25_highrecosts_2036.reset_index(drop=True)
lrmer_cambium25_highrecosts_2037 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2037].copy()
lrmer_cambium25_highrecosts_2037 = lrmer_cambium25_highrecosts_2037.reset_index(drop=True)
lrmer_cambium25_highrecosts_2038 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2038].copy()
lrmer_cambium25_highrecosts_2038 = lrmer_cambium25_highrecosts_2038.reset_index(drop=True)
lrmer_cambium25_highrecosts_2039 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2039].copy()
lrmer_cambium25_highrecosts_2039 = lrmer_cambium25_highrecosts_2039.reset_index(drop=True)
lrmer_cambium25_highrecosts_2040 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2040].copy()
lrmer_cambium25_highrecosts_2040 = lrmer_cambium25_highrecosts_2040.reset_index(drop=True)
lrmer_cambium25_highrecosts_2041 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2041].copy()
lrmer_cambium25_highrecosts_2041 = lrmer_cambium25_highrecosts_2041.reset_index(drop=True)
lrmer_cambium25_highrecosts_2042 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2042].copy()
lrmer_cambium25_highrecosts_2042 = lrmer_cambium25_highrecosts_2042.reset_index(drop=True)
lrmer_cambium25_highrecosts_2043 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2043].copy()
lrmer_cambium25_highrecosts_2043 = lrmer_cambium25_highrecosts_2043.reset_index(drop=True)
lrmer_cambium25_highrecosts_2044 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2044].copy()
lrmer_cambium25_highrecosts_2044 = lrmer_cambium25_highrecosts_2044.reset_index(drop=True)
lrmer_cambium25_highrecosts_2045 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2045].copy()
lrmer_cambium25_highrecosts_2045 = lrmer_cambium25_highrecosts_2045.reset_index(drop=True)
lrmer_cambium25_highrecosts_2046 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2046].copy()
lrmer_cambium25_highrecosts_2046 = lrmer_cambium25_highrecosts_2046.reset_index(drop=True)
lrmer_cambium25_highrecosts_2047 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2047].copy()
lrmer_cambium25_highrecosts_2047 = lrmer_cambium25_highrecosts_2047.reset_index(drop=True)
lrmer_cambium25_highrecosts_2048 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2048].copy()
lrmer_cambium25_highrecosts_2048 = lrmer_cambium25_highrecosts_2048.reset_index(drop=True)
lrmer_cambium25_highrecosts_2049 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2049].copy()
lrmer_cambium25_highrecosts_2049 = lrmer_cambium25_highrecosts_2049.reset_index(drop=True)
lrmer_cambium25_highrecosts_2050 = lrmer_cambium25_highrecosts[lrmer_cambium25_highrecosts['year'] == 2050].copy()
lrmer_cambium25_highrecosts_2050 = lrmer_cambium25_highrecosts_2050.reset_index(drop=True)

cols_to_modify = [col for col in lrmer_cambium25_highrecosts_2025.columns if col not in ['hour', 'year']]

#display(lrmer_cambium25_highrecosts_2025)
#display(lrmer_cambium25_highrecosts_2030)
#display(lrmer_cambium25_highrecosts_2035)
#display(lrmer_cambium25_highrecosts_2040)
#display(lrmer_cambium25_highrecosts_2045)
#display(lrmer_cambium25_highrecosts_2050)

# Fill in the 2026-2029 values
lrmer_cambium25_highrecosts_2025_2030_change = lrmer_cambium25_highrecosts_2025.copy()
lrmer_cambium25_highrecosts_2025_2030_change[cols_to_modify] = (lrmer_cambium25_highrecosts_2030[cols_to_modify] - lrmer_cambium25_highrecosts_2025[cols_to_modify]) / 5
#display(lrmer_cambium25_highrecosts_2025_2030_change)
lrmer_cambium25_highrecosts_2026[cols_to_modify] = lrmer_cambium25_highrecosts_2025[cols_to_modify] + lrmer_cambium25_highrecosts_2025_2030_change[cols_to_modify]
lrmer_cambium25_highrecosts_2027[cols_to_modify] = lrmer_cambium25_highrecosts_2026[cols_to_modify] + lrmer_cambium25_highrecosts_2025_2030_change[cols_to_modify]
lrmer_cambium25_highrecosts_2028[cols_to_modify] = lrmer_cambium25_highrecosts_2027[cols_to_modify] + lrmer_cambium25_highrecosts_2025_2030_change[cols_to_modify]
lrmer_cambium25_highrecosts_2029[cols_to_modify] = lrmer_cambium25_highrecosts_2028[cols_to_modify] + lrmer_cambium25_highrecosts_2025_2030_change[cols_to_modify]

# Add the 2026-2029 values back into the main data frame
for col in cols_to_modify:
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2026, col] = lrmer_cambium25_highrecosts_2026[col].values
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2027, col] = lrmer_cambium25_highrecosts_2027[col].values
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2028, col] = lrmer_cambium25_highrecosts_2028[col].values
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2029, col] = lrmer_cambium25_highrecosts_2029[col].values

# Fill in the 2031-2034 values
lrmer_cambium25_highrecosts_2030_2035_change = lrmer_cambium25_highrecosts_2030.copy()
lrmer_cambium25_highrecosts_2030_2035_change[cols_to_modify] = (lrmer_cambium25_highrecosts_2035[cols_to_modify] - lrmer_cambium25_highrecosts_2030[cols_to_modify]) / 5
#display(lrmer_cambium25_highrecosts_2030_2035_change)
lrmer_cambium25_highrecosts_2031[cols_to_modify] = lrmer_cambium25_highrecosts_2030[cols_to_modify] + lrmer_cambium25_highrecosts_2030_2035_change[cols_to_modify]
lrmer_cambium25_highrecosts_2032[cols_to_modify] = lrmer_cambium25_highrecosts_2031[cols_to_modify] + lrmer_cambium25_highrecosts_2030_2035_change[cols_to_modify]
lrmer_cambium25_highrecosts_2033[cols_to_modify] = lrmer_cambium25_highrecosts_2032[cols_to_modify] + lrmer_cambium25_highrecosts_2030_2035_change[cols_to_modify]
lrmer_cambium25_highrecosts_2034[cols_to_modify] = lrmer_cambium25_highrecosts_2033[cols_to_modify] + lrmer_cambium25_highrecosts_2030_2035_change[cols_to_modify]

# Add the 2031-2034 values back into the main data frame
for col in cols_to_modify:
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2031, col] = lrmer_cambium25_highrecosts_2031[col].values
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2032, col] = lrmer_cambium25_highrecosts_2032[col].values
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2033, col] = lrmer_cambium25_highrecosts_2033[col].values
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2034, col] = lrmer_cambium25_highrecosts_2034[col].values

# Fill in the 2036-2039 values
lrmer_cambium25_highrecosts_2035_2040_change = lrmer_cambium25_highrecosts_2035.copy()
lrmer_cambium25_highrecosts_2035_2040_change[cols_to_modify] = (lrmer_cambium25_highrecosts_2040[cols_to_modify] - lrmer_cambium25_highrecosts_2035[cols_to_modify]) / 5
#display(lrmer_cambium25_highrecosts_2035_2040_change)
lrmer_cambium25_highrecosts_2036[cols_to_modify] = lrmer_cambium25_highrecosts_2035[cols_to_modify] + lrmer_cambium25_highrecosts_2035_2040_change[cols_to_modify]
lrmer_cambium25_highrecosts_2037[cols_to_modify] = lrmer_cambium25_highrecosts_2036[cols_to_modify] + lrmer_cambium25_highrecosts_2035_2040_change[cols_to_modify]
lrmer_cambium25_highrecosts_2038[cols_to_modify] = lrmer_cambium25_highrecosts_2037[cols_to_modify] + lrmer_cambium25_highrecosts_2035_2040_change[cols_to_modify]
lrmer_cambium25_highrecosts_2039[cols_to_modify] = lrmer_cambium25_highrecosts_2038[cols_to_modify] + lrmer_cambium25_highrecosts_2035_2040_change[cols_to_modify]

# Add the 2036-2039 values back into the main data frame
for col in cols_to_modify:
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2036, col] = lrmer_cambium25_highrecosts_2036[col].values
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2037, col] = lrmer_cambium25_highrecosts_2037[col].values
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2038, col] = lrmer_cambium25_highrecosts_2038[col].values
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2039, col] = lrmer_cambium25_highrecosts_2039[col].values

# Fill in the 2041-2044 values
lrmer_cambium25_highrecosts_2040_2045_change = lrmer_cambium25_highrecosts_2040.copy()
lrmer_cambium25_highrecosts_2040_2045_change[cols_to_modify] = (lrmer_cambium25_highrecosts_2045[cols_to_modify] - lrmer_cambium25_highrecosts_2040[cols_to_modify]) / 5
#display(lrmer_cambium25_highrecosts_2040_2045_change)
lrmer_cambium25_highrecosts_2041[cols_to_modify] = lrmer_cambium25_highrecosts_2040[cols_to_modify] + lrmer_cambium25_highrecosts_2040_2045_change[cols_to_modify]
lrmer_cambium25_highrecosts_2042[cols_to_modify] = lrmer_cambium25_highrecosts_2041[cols_to_modify] + lrmer_cambium25_highrecosts_2040_2045_change[cols_to_modify]
lrmer_cambium25_highrecosts_2043[cols_to_modify] = lrmer_cambium25_highrecosts_2042[cols_to_modify] + lrmer_cambium25_highrecosts_2040_2045_change[cols_to_modify]
lrmer_cambium25_highrecosts_2044[cols_to_modify] = lrmer_cambium25_highrecosts_2043[cols_to_modify] + lrmer_cambium25_highrecosts_2040_2045_change[cols_to_modify]

# Add the 2041-2044 values back into the main data frame
for col in cols_to_modify:
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2041, col] = lrmer_cambium25_highrecosts_2041[col].values
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2042, col] = lrmer_cambium25_highrecosts_2042[col].values
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2043, col] = lrmer_cambium25_highrecosts_2043[col].values
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2044, col] = lrmer_cambium25_highrecosts_2044[col].values

# Fill in the 2046-2049 values
lrmer_cambium25_highrecosts_2045_2050_change = lrmer_cambium25_highrecosts_2045.copy()
lrmer_cambium25_highrecosts_2045_2050_change[cols_to_modify] = (lrmer_cambium25_highrecosts_2050[cols_to_modify] - lrmer_cambium25_highrecosts_2045[cols_to_modify]) / 5
#display(lrmer_cambium25_highrecosts_2045_2050_change)
lrmer_cambium25_highrecosts_2046[cols_to_modify] = lrmer_cambium25_highrecosts_2045[cols_to_modify] + lrmer_cambium25_highrecosts_2045_2050_change[cols_to_modify]
lrmer_cambium25_highrecosts_2047[cols_to_modify] = lrmer_cambium25_highrecosts_2046[cols_to_modify] + lrmer_cambium25_highrecosts_2045_2050_change[cols_to_modify]
lrmer_cambium25_highrecosts_2048[cols_to_modify] = lrmer_cambium25_highrecosts_2047[cols_to_modify] + lrmer_cambium25_highrecosts_2045_2050_change[cols_to_modify]
lrmer_cambium25_highrecosts_2049[cols_to_modify] = lrmer_cambium25_highrecosts_2048[cols_to_modify] + lrmer_cambium25_highrecosts_2045_2050_change[cols_to_modify]

# Add the 2046-2049 values back into the main data frame
for col in cols_to_modify:
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2046, col] = lrmer_cambium25_highrecosts_2046[col].values
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2047, col] = lrmer_cambium25_highrecosts_2047[col].values
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2048, col] = lrmer_cambium25_highrecosts_2048[col].values
    lrmer_cambium25_highrecosts.loc[lrmer_cambium25_highrecosts['year'] == 2049, col] = lrmer_cambium25_highrecosts_2049[col].values
    
# Save the csv files
csv_output_path = os.path.join(os.getcwd(), '..', 'carbon_intensities', 'lrmer_cambium25_highrecosts.csv')
lrmer_cambium25_highrecosts.to_csv(csv_output_path, index=False)
display(lrmer_cambium25_highrecosts)

# Save the zip files
zip_output_path = os.path.join(os.getcwd(), '..', 'carbon_intensities', 'lrmer_cambium25_highrecosts.zip') # Save ZIP in the same directory as CSV
with zipfile.ZipFile(zip_output_path, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
    # zf.write(source_path, arcname=name_inside_zip)
    # arcname specifies the name of the file *inside* the zip archive
    zf.write(csv_output_path, arcname='lrmer_cambium25_highrecosts.csv')

# Save individual csv files for each region
common_cols = ['hour', 'year']
subregion_specific_cols = [col for col in lrmer_cambium25_highrecosts.columns if col not in common_cols]
for subregion_col_name in subregion_specific_cols:
    new_df_for_subregion = lrmer_cambium25_highrecosts[common_cols + [subregion_col_name]].copy()
    cleaned_subregion_name = subregion_col_name.lower().replace(' ', '')
    csv_filename = f'lrmer_cambium25_highrecosts_{cleaned_subregion_name}.csv' 
    csv_output_path = os.path.join(os.getcwd(), '..', 'carbon_intensities', csv_filename)
    new_df_for_subregion.to_csv(csv_output_path, index=False)

,hour,year,CAISO,ERCOT,FRCC,ISONE,MISO Central,MISO North,MISO South,Northern Grid East,Northern Grid South,Northern Grid West,NYISO,PJM East,PJM West,SERTP,SPP North,SPP South,West Connect North,West Connect South
52560,0.0,2014,250.909537,259.657884,361.790447,173.727856,289.570349,287.205881,273.361654,308.548517,337.784248,116.394147,136.510658,296.043003,298.684358,314.390326,294.7375,303.869325,268.3889,340.087962
52561,1.0,2014,250.570514,254.533574,386.441601,167.030741,299.000711,285.498372,274.763185,305.949988,339.789114,112.35259,126.916944,288.886588,296.709373,306.288738,310.102832,313.135536,269.112244,343.315599
52562,2.0,2014,256.945144,246.507638,386.808658,165.701043,304.399572,291.915664,268.29218,305.352316,341.677354,115.731454,130.793522,290.71044,293.33604,305.082826,313.882388,301.343976,265.266767,346.120288
52563,3.0,2014,258.871992,242.608934,390.997891,168.780619,307.631736,273.531061,264.158001,308.589644,339.597711,116.883474,123.88453,297.160986,295.519089,307.95283,310.638617,299.882245,268.853684,346.894729
52564,4.0,2014,254.855677,244.971937,386.691205,167.823366,296.848063,287.19998,275.348482,313.555234,339.963566,120.038867,124.939538,300.507918,298.348743,305.200823,317.623383,303.576868,272.463297,340.585472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52555,8755.0,2050,51.19803,67.79848,359.93487,46.11893,80.10502,229.72281,132.71015,220.77958,204.35338,47.72289,0.0,283.8647,129.1351,173.26215,60.36667,75.21548,184.01546,241.3622
52556,8756.0,2050,50.95333,96.54271,363.04147,43.00985,70.89764,227.63541,137.20531,236.66083,190.65357,45.56281,0.0,284.8584,131.97009,168.92336,64.58783,79.41255,181.49826,247.73475
52557,8757.0,2050,56.4739,107.59304,383.61838,43.61415,76.85529,211.76012,137.70365,228.11603,197.62446,46.26413,0.0,269.63377,105.98726,161.89146,79.14558,83.68735,182.82772,256.87516
52558,8758.0,2050,54.08352,130.79584,400.43373,38.80061,77.31043,203.67922,150.0745,207.07952,190.18801,33.41878,0.0,271.58243,118.67153,171.99254,85.24959,104.27759,167.19117,237.50685
